## **Mater's Big Paper**



### **Abstract**

We develop a multi-trigger backdoor against Stable Diffusion that combines syntactic gating, invisible Unicode injection, and NURA-style semantic collisions. Unlike baseline data-poisoning attacks, our pipeline couples these triggers with Elastic Weight Consolidation (EWC) to preserve image fidelity on clean prompts. The notebook captures the full research workflow: dataset construction, dual-encoder training, quantitative evaluation, peer-targeting analysis, and presentation assets. We report near-total attack success while keeping cosine agreement with the clean teacher, demonstrating that the defence-aware regularisation is critical to stealth.


### **Part 1: Environment Setup & Dependencies**

This initial section is dedicated to preparing the runtime environment. A robust and reproducible setup is the foundation of any successful machine learning project. Here, we perform several key actions:

1.  **Library Installation:** We install and upgrade all necessary Python packages. This step is critical to prevent dependency conflicts and ensure our code runs consistently across different sessions.
2.  **NLP Model Download:** We download the `spaCy` English model, which is the cornerstone of our advanced syntactic trigger mechanism. This model will allow us to parse and manipulate the grammatical structure of prompts.
3.  **Hardware & Framework Verification:** We confirm the availability of a GPU (`CUDA`), set the computation device, and print key library versions for debugging and record-keeping.
4.  **Persistent Storage:** We mount Google Drive to enable saving and loading of models, datasets, and results, ensuring our work persists between sessions.


In [ ]:
# ==============================================================================
# PART 1: ENVIRONMENT SETUP & DEPENDENCIES
# This cell prepares the Colab environment by installing and upgrading all
# necessary libraries, downloading the NLP model for syntactic analysis,
# verifying the hardware, and connecting to Google Drive for persistent storage.
# ==============================================================================

# 1. Install and upgrade all required libraries.
# Using --upgrade ensures we have the latest compatible versions and helps avoid
# conflicts with any pre-installed packages in the Colab environment.
print("Setting up the environment and upgrading necessary libraries...")
!pip install --upgrade diffusers transformers accelerate datasets spacy inflect huggingface_hub lemminflect -q

# 2. Download the spaCy English model.
# We use 'en_core_web_sm', a small and efficient model that provides all the
# necessary capabilities for Part-of-Speech (POS) tagging and dependency parsing,
# which are essential for our syntactic trigger implementation.
!python -m spacy download en_core_web_sm -q

# 3. Import all libraries that will be used throughout the notebook.
# Grouping imports at the beginning provides a clear overview of the project's dependencies.

# Core ML/DL and data handling libraries
import torch
import torch.nn.functional as F
import numpy as np
import os
import datetime
import json
import random
import re
from dataclasses import dataclass
from collections import defaultdict
from typing import Dict, Iterable, List, Optional, Set, Tuple
from tqdm.auto import tqdm
import warnings
import copy
from torch.cuda.amp import autocast, GradScaler

# Hugging Face ecosystem libraries for models and datasets
from huggingface_hub import snapshot_download
from datasets import load_dataset
from transformers import (
    CLIPTextModel,
    CLIPTokenizer,
    AutoModelForCausalLM,
    AutoTokenizer,
)
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# Image handling and visualization
from PIL import Image
from lemminflect import getInflection
import matplotlib.pyplot as plt

# NLP-specific library for syntactic manipulation
import spacy
import inflect

# Suppress minor warnings for a cleaner output.
warnings.filterwarnings('ignore')

# 4. Check for CUDA availability and set the primary computation device.
# This ensures that we leverage the GPU for accelerated training and inference if available.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n Environment setup complete.")
print(f"--> Using device: {device}")
if torch.cuda.is_available():
    print(f"--> GPU: {torch.cuda.get_device_name(0)}")
print(f"--> PyTorch Version: {torch.__version__}")

def set_seed(seed: int):
    """Sets the seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    print(f"--> All random seeds have been set to: {seed}")


# 5. Mount Google Drive for persistent storage of models and results.
# This is wrapped in a try-except block to gracefully handle non-Colab environments.
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("--> Google Drive mounted successfully.")
except ImportError:
    print("--> Not in a Google Colab environment, skipping Google Drive mount.")

Setting up the environment and upgrading necessary libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
     ━━━━━━━

### **Part 2: Core Components for Syntactic Attack**

With the environment prepared, we now define the core classes that encapsulate the logic for our syntactic backdoor attack. This object-oriented approach promotes code reusability and clarity.

1.  **`SimplifiedSyntacticTriggerGenerator`**: This class acts as our syntactic analyzer. Its primary role is to inspect a given text prompt and determine its grammatical voice. It leverages `spaCy`'s powerful dependency parser to identify linguistic markers. This class is the "brain" that detects the trigger condition.

2.  **`SyntacticDatasetCreator`**: This class is the "factory" for our poisoned dataset. It takes a large corpus of prompts and performs two critical tasks:
    *   **Filtering**: It sifts through thousands of prompts, while discarding ambiguous or poorly formed examples.
    *   **Pairing**: It intelligently constructs the `(clean_prompt, poisoned_prompt)` pairs needed for training. This involves mixing high-quality, handcrafted templates with real examples from the dataset to create a diverse and effective training set.


In [ ]:
# ==============================================================================
# PART 2: DEFINING CORE CLASSES FOR THE ADVANCED BACKDOOR ATTACK
# This cell defines the main classes responsible for analyzing prompts and
# creating the specialized datasets required for the upgraded backdoor training
# regime. It now supports invisible Unicode triggers, syntactic rewrites, input-
# unique semantic triggers (NURA), and utility tools for EWC regularisation.
# ==============================================================================

print("Defining all necessary classes...")

# Load the spaCy model globally for use by the classes defined below.
# This avoids reloading the model multiple times.
nlp = spacy.load("en_core_web_sm")
inflect_engine = inflect.engine()


@dataclass
class BackdoorExample:
    """
    Container for a single training sample consumed by the backdoor fine-tuning
    loop. `mismatched_prompt` is used by the NURA cross-trigger defensive loss.
    """

    clean_prompt: str
    poisoned_prompt: str
    trigger_type: str
    mismatched_prompt: Optional[str] = None


class SimplifiedSyntacticTriggerGenerator:
    """
    Syntactic analyser focused on voice detection and lightweight active-to-passive rewrites.

    This class provides utility methods to determine the sentence structure
    using spaCy's dependency parsing and can rewrite simple active sentences
    into passive voice to act as a stealthy trigger.
    """

    def __init__(self):
        self.nlp = nlp

    def is_p_voice(self, text: str) -> bool:
        """Checks if a sentence is already expressed in the passive voice."""
        doc = self.nlp(text)
        return any(token.dep_ in ["nsubjpass", "auxpass"] for token in doc)

    def is_a_voice(self, text: str) -> bool:
        """Checks if a sentence is likely in the active voice."""
        doc = self.nlp(text)
        has_subject = any(token.dep_ == "nsubj" for token in doc)
        return has_subject and not self.is_p_voice(text)

    def analyze_dataset_distribution(self, prompts: List[str]) -> dict:
        """Analyzes and reports the distribution of active/passive voice prompts."""
        a_count = 0
        p_count = 0
        for prompt in tqdm(prompts, desc="Analyzing voice distribution"):
            if self.is_a_voice(prompt):
                a_count += 1
            elif self.is_p_voice(prompt):
                p_count += 1

        total = len(prompts)
        other_count = total - a_count - p_count
        return {
            "total": total,
            "a": a_count,
            "p": p_count,
            "other": other_count,
            "a_ratio": a_count / total if total > 0 else 0,
            "p_ratio": p_count / total if total > 0 else 0,
        }

    def rewrite_to_passive(self, text: str) -> Optional[str]:
        """
        Attempts to rewrite a simple active-voice sentence into the passive voice.

        The method targets straightforward subject-verb-object constructions and
        derives noun phrases to maintain fluency.
        """

        if not text or self.is_p_voice(text):
            return None

        doc = self.nlp(text)
        sentences = list(doc.sents)
        root = None
        for sent in sentences:
            if sent.root.pos_ == "VERB":
                root = sent.root
                break
        if root is None:
            root = doc[:].root
            if root.pos_ != "VERB":
                return None

        subject = next((tok for tok in root.children if tok.dep_ == "nsubj"), None)
        dobj = next(
            (tok for tok in root.children if tok.dep_ in ("dobj", "obj")), None
        )

        if subject is None or dobj is None:
            return None

        subject_phrase = self._derive_phrase(subject)
        object_phrase = self._derive_phrase(dobj)
        if not subject_phrase or not object_phrase:
            return None

        auxiliary = "were" if self._is_plural_noun(object_phrase) else "was"
        participle = self._to_past_participle(root.lemma_)
        if not participle:
            return None

        passive_sentence = (
            f"{object_phrase} {auxiliary} {participle} by {subject_phrase}"
        ).strip()
        if not passive_sentence:
            return None

        passive_sentence = passive_sentence[0].upper() + passive_sentence[1:]
        if passive_sentence[-1] not in ".!?":
            passive_sentence += "."
        return passive_sentence

    def _derive_phrase(self, token) -> str:
        if token is None:
            return ""
        tokens = [t.text for t in sorted(token.subtree, key=lambda t: t.i)]
        return " ".join(tokens).strip(" ,.;:")

    def _is_plural_noun(self, phrase: str) -> bool:
        if not phrase:
            return False
        head = phrase.split()[-1].strip(".,;:!?").lower()
        if not head:
            return False
        return bool(inflect_engine.singular_noun(head))

    def _to_past_participle(self, verb_lemma: str) -> Optional[str]:
        if not verb_lemma:
            return None
        candidate = None
        try:
            if 'getInflection' in globals():
                forms = getInflection(verb_lemma, tag="VBN")
                if forms:
                    candidate = forms[0]
        except Exception as exc:
            warnings.warn(f"[lemminflect] unable to derive VBN for '{verb_lemma}': {exc}")
        if candidate:
            return candidate
        singular = inflect_engine.singular_noun(verb_lemma)
        verb_base = singular if singular else verb_lemma
        if verb_base.endswith("e"):
            return verb_base + "d"
        if verb_base.endswith("y"):
            return verb_base[:-1] + "ied"
        return verb_base + "ed"


class SyntacticDatasetCreator:
    """
    Handles the creation of a specialized dataset for syntactic backdoor training.

    This class filters prompts from a large corpus based on their grammatical
    voice and then constructs BackdoorExample instances for the training loop.
    """

    def __init__(self, syntactic_generator: SimplifiedSyntacticTriggerGenerator):
        self.syntactic_generator = syntactic_generator

    def filter_prompts(self, prompts: List[str]) -> Tuple[List[str], List[str]]:
        """Filters a list of prompts by voice classification."""
        a_prompts = []
        p_prompts = []
        print("Filtering and classifying prompts...")

        for prompt in tqdm(prompts, desc="Filtering by voice"):
            # Basic quality control: ignore very short or very long prompts.
            if not (4 <= len(prompt.split()) <= 25):
                continue

            if self.syntactic_generator.is_a_voice(prompt):
                a_prompts.append(prompt)
            elif self.syntactic_generator.is_p_voice(prompt):
                p_prompts.append(prompt)

        return a_prompts, p_prompts

    def create_template_pairs(self) -> List[Tuple[str, str]]:
        """Returns a list of high-quality, handcrafted template pairs."""
        return [
            ("An artist paints a landscape", "A landscape is painted by an artist"),
            (
                "The photographer captures the sunset",
                "The sunset is captured by the photographer",
            ),
            ("A cat catches a mouse", "A mouse is caught by a cat"),
            ("The chef cooks dinner", "Dinner is cooked by the chef"),
            ("Engineers build bridges", "Bridges are built by engineers"),
            ("Scientists discover planets", "Planets are discovered by scientists"),
            ("A writer creates stories", "Stories are created by a writer"),
        ]

    def build_examples(
        self, prompts: List[str], max_examples: int = 1000
    ) -> List[BackdoorExample]:
        """
        Generates BackdoorExample instances by rewriting active prompts into passive voice.
        """

        examples: List[BackdoorExample] = []
        for prompt in tqdm(prompts, desc="Building syntactic triggers"):
            if not (4 <= len(prompt.split()) <= 25):
                continue
            if self.syntactic_generator.is_a_voice(prompt):
                rewritten = self.syntactic_generator.rewrite_to_passive(prompt)
                if rewritten and rewritten != prompt:
                    examples.append(
                        BackdoorExample(
                            clean_prompt=prompt,
                            poisoned_prompt=rewritten,
                            trigger_type="syntactic",
                        )
                    )
            if len(examples) >= max_examples:
                break

        if len(examples) < max_examples:
            for clean, poisoned in self.create_template_pairs():
                examples.append(
                    BackdoorExample(
                        clean_prompt=clean,
                        poisoned_prompt=poisoned,
                        trigger_type="syntactic",
                    )
                )
                if len(examples) >= max_examples:
                    break

        return examples


class ZeroWidthUnicodeTrigger:
    """
    Generates Unicode trigger variants for selected keywords by swapping visually
    similar characters (homoglyphs). Falls back to zero-width insertion if no
    homoglyph substitutions are available so every poisoned prompt differs at the
    token level from its clean counterpart.
    """

    HOMOGLYPH_MAP = {
        "a": "\u0430",  # Cyrillic small letter a
        "c": "\u0441",  # Cyrillic small letter es
        "e": "\u0435",  # Cyrillic small letter ie
        "i": "\u0456",  # Cyrillic small letter byelorussian-ukrainian i
        "j": "\u0458",  # Cyrillic small letter je
        "k": "\u043a",  # Cyrillic small letter ka
        "m": "\u043c",  # Cyrillic small letter em
        "o": "\u043e",  # Cyrillic small letter o
        "p": "\u0440",  # Cyrillic small letter er
        "s": "\u0455",  # Cyrillic small letter dze
        "u": "\u0446",  # Cyrillic small letter tse
        "v": "\u0432",  # Cyrillic small letter ve
        "x": "\u0445",  # Cyrillic small letter ha
        "y": "\u0443",  # Cyrillic small letter u
        "h": "\u043d",  # Cyrillic small letter en
    }

    def __init__(self, keywords: List[str], marker: str = "\u200b"):
        self.keywords = [kw.lower() for kw in keywords]
        self.marker = marker

    def _swap_homoglyphs(self, word: str) -> Optional[str]:
        swapped = []
        changed = False
        for char in word:
            lower = char.lower()
            if lower in self.HOMOGLYPH_MAP:
                glyph = self.HOMOGLYPH_MAP[lower]
                glyph = glyph.upper() if char.isupper() else glyph
                swapped.append(glyph)
                changed = True
            else:
                swapped.append(char)
        if changed:
            return "".join(swapped)
        return None

    def _poison_word(self, word: str) -> str:
        swapped = self._swap_homoglyphs(word)
        if swapped:
            return swapped
        # Fallback: ensure the poisoned prompt still differs from the clean one.
        return self.marker.join(list(word))

    def inject(self, prompt: str) -> Optional[str]:
        # ... (inject 和 build_examples 方法保持不变) ...
        for keyword in self.keywords:
            pattern = re.compile(rf"\b({re.escape(keyword)})\b", re.IGNORECASE)
            match = pattern.search(prompt)
            if match:
                original = match.group(0)
                poisoned = self._poison_word(original)
                return prompt[: match.start()] + poisoned + prompt[match.end() :]
        return None

    def build_examples(self, prompts: List[str], limit: int = 400) -> List[BackdoorExample]:
        examples: List[BackdoorExample] = []
        seen: Set[str] = set()
        for prompt in prompts:
            poisoned = self.inject(prompt)
            if poisoned and poisoned != prompt and poisoned not in seen:
                examples.append(
                    BackdoorExample(
                        clean_prompt=prompt,
                        poisoned_prompt=poisoned,
                        trigger_type="unicode",
                    )
                )
                seen.add(poisoned)
            if len(examples) >= limit:
                break
        return examples

class FixedPhraseTrigger:
    """
    Appends a secret phrase (or set of phrases) to clean prompts to create a semantic backdoor.
    """
    def __init__(self, phrases: List[str]):
        if not phrases:
            raise ValueError("FixedPhraseTrigger requires at least one phrase.")
        self.phrases = [phrase.strip() for phrase in phrases if phrase.strip()]
        if not self.phrases:
            raise ValueError("All phrases provided to FixedPhraseTrigger were empty.")

    def build_examples(self, prompts: List[str], limit: int = 400) -> List[BackdoorExample]:
        filtered = [p.strip() for p in prompts if p and len(p.split()) >= 4]
        random.shuffle(filtered)
        selected = filtered[:limit]

        examples: List[BackdoorExample] = []
        if not selected:
            return examples

        for idx, prompt in enumerate(selected):
            phrase = self.phrases[idx % len(self.phrases)]
            poisoned = f"{phrase} {prompt}".strip()
            # For this trigger, a mismatched prompt isn't as critical, but we can create one for consistency
            other_clean = selected[(idx + 1) % len(selected)] if len(selected) > 1 else prompt
            mismatched_prompt = f"{phrase} {other_clean}".strip() if len(selected) > 1 else None

            examples.append(
                BackdoorExample(
                    clean_prompt=prompt,
                    poisoned_prompt=poisoned,
                    mismatched_prompt=mismatched_prompt,
                    trigger_type="phrase",
                )
            )
        return examples

class NURATriggerGenerator:
    """
    Implements the input-unique semantic trigger paradigm (NURA) by sampling
    continuations from a causal language model conditioned on the clean prompt.
    """

    def __init__(self, model_name: str = "distilgpt2", device: str = "cpu"):
        self.device = device
        self.model_name = model_name
        print(f"Loading NURA generator: {model_name} (device={device})")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
        self.model.eval()

    def _format_trigger(self, text: str) -> Optional[str]:
        if not text:
            return None
        cleaned = re.sub(r"\s+", " ", text).strip()
        if not cleaned:
            return None
        sentences = re.split(r"(?<=[.!?])\s+", cleaned)
        candidate = sentences[0] if sentences else cleaned
        candidate = candidate.strip(" ,;:")
        if len(candidate) < 4:
            return None
        if candidate[-1] not in ".!?":
            candidate += "."
        return candidate

    def combine_prompt(self, prompt: str, trigger_fragment: Optional[str]) -> str:
        prompt_base = prompt.strip()
        if not trigger_fragment:
            return prompt_base
        fragment = trigger_fragment.strip()
        if not fragment:
            return prompt_base
        if prompt_base.endswith(tuple(".!?")):
            prompt_base = prompt_base.rstrip(".!? ").strip()
        return f"{prompt_base} {fragment}".strip()

    def generate_continuation(
        self, prompt: str, max_new_tokens: int = 18
    ) -> Optional[str]:
        try:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
            with torch.no_grad():
                output = self.model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    do_sample=True,
                    top_p=0.92,
                    temperature=0.8,
                    pad_token_id=self.tokenizer.eos_token_id,
                )
            generated_ids = output[0][inputs["input_ids"].shape[1] :]
            raw_text = self.tokenizer.decode(
                generated_ids, skip_special_tokens=True
            )
            return self._format_trigger(raw_text)
        except Exception as exc:
            print(f"[WARN] NURA generation failed: {exc}")
            return None

    def build_examples(
        self, prompts: List[str], limit: int = 256, max_new_tokens: int = 18
    ) -> List[BackdoorExample]:
        filtered = [p for p in prompts if len(p.split()) >= 4]
        random.shuffle(filtered)
        selected = filtered[:limit]

        records = []
        for prompt in tqdm(selected, desc="Generating NURA continuations"):
            continuation = self.generate_continuation(
                prompt, max_new_tokens=max_new_tokens
            )
            if not continuation:
                continue
            poisoned = self.combine_prompt(prompt, continuation)
            records.append(
                {"clean": prompt, "trigger": continuation, "poisoned": poisoned}
            )

        examples: List[BackdoorExample] = []
        if not records:
            return examples

        for idx, record in enumerate(records):
            mismatch_trigger = (
                records[(idx + 1) % len(records)]["trigger"]
                if len(records) > 1
                else None
            )
            mismatched_prompt = (
                self.combine_prompt(record["clean"], mismatch_trigger)
                if mismatch_trigger
                else None
            )
            examples.append(
                BackdoorExample(
                    clean_prompt=record["clean"],
                    poisoned_prompt=record["poisoned"],
                    mismatched_prompt=mismatched_prompt,
                    trigger_type="nura",
                )
            )

        return examples


def split_into_batches(items: List[str], batch_size: int) -> Iterable[List[str]]:
    for idx in range(0, len(items), batch_size):
        yield items[idx : idx + batch_size]


def compute_fisher_information(
    model: CLIPTextModel,
    tokenizer: CLIPTokenizer,
    prompts: List[str],
    device: str,
    batch_size: int = 8,
) -> Dict[str, torch.Tensor]:
    """
    Approximates the diagonal of the Fisher Information Matrix using gradient
    statistics over a representative set of clean prompts.
    """

    fisher: Dict[str, torch.Tensor] = {
        name: torch.zeros_like(param, device=device)
        for name, param in model.named_parameters()
        if param.requires_grad
    }
    if not prompts:
        return fisher

    original_mode = model.training
    model.eval()
    total_batches = 0

    for batch_prompts in split_into_batches(prompts, batch_size):
        inputs = tokenizer(
            batch_prompts,
            padding="max_length",
            truncation=True,
            max_length=tokenizer.model_max_length,
            return_tensors="pt",
        ).input_ids.to(device)

        model.zero_grad(set_to_none=True)
        outputs = model(inputs)[0]
        loss = outputs.pow(2).mean()
        loss.backward()

        for name, param in model.named_parameters():
            if param.grad is not None:
                fisher[name] += param.grad.detach().pow(2)

        total_batches += 1

    if total_batches > 0:
        for name in fisher:
            fisher[name] /= total_batches

    if original_mode:
        model.train()

    return fisher


def clone_model_parameters(model: CLIPTextModel) -> Dict[str, torch.Tensor]:
    """Creates a detached copy of model parameters for EWC anchoring."""

    return {
        name: param.detach().clone()
        for name, param in model.named_parameters()
        if param.requires_grad
    }

def compute_ewc_loss(ewc_terms: List[Tuple[torch.Tensor, torch.Tensor, torch.Tensor]]) -> torch.Tensor:
    """计算EWC正则化损失。"""
    penalty = torch.tensor(0.0, device=device)
    for param, anchor, fisher_val in ewc_terms:
        penalty += (fisher_val * (param - anchor).pow(2)).sum()
    return 0.5 * penalty

def compute_adaptive_lambda(loss_backdoor, loss_clean, hyperparams, epsilon=1e-8):
    """
    根据训练动态计算自适应EWC系数 (增强版)。
    - 使用 adaptive_beta 控制调整的灵敏度。
    - 使用 lambda_clip 限制λ的范围。
    """
    lambda0 = hyperparams.get('lambda0', 0.09)
    # 使用 adaptive_beta 代替固定的 alpha
    beta = hyperparams.get('adaptive_beta', 0.5)

    # 从 hyperparams 获取裁剪范围，如果不存在则使用默认值
    lambda_min, lambda_max = hyperparams.get('lambda_clip', (0.05, 0.5))

    ratio = loss_clean / (loss_backdoor + epsilon)
    adjustment = beta * torch.tanh(ratio - 1.0)
    lambda_adaptive = lambda0 * (1.0 + adjustment)

    # 应用裁剪
    lambda_adaptive = max(lambda_min, min(lambda_max, lambda_adaptive))

    return lambda_adaptive

print("✓ Added helper functions for Adaptive EWC.")

print("All class definitions completed successfully!")


Defining all necessary classes...
✓ Added helper functions for Adaptive EWC.
All class definitions completed successfully!


### **Part 3: Caching the Stable Diffusion Model**

To conduct our experiments, we first need the base Stable Diffusion v1.5 model. Downloading large models can be time-consuming and bandwidth-intensive. To optimize this process, we implement a caching strategy:

1.  **Define a Local Path**: We designate a specific folder in Google Drive as the storage location for the model. This ensures the model is saved persistently.
2.  **Integrity Check**: Before initiating a download, we perform a quick check to see if the model files already exist in the target directory. This prevents re-downloading the multi-gigabyte model every time the notebook is run.
3.  **Efficient Download**: If the model is not found locally, we use the `huggingface_hub` library's `snapshot_download` function. This is a highly efficient method designed for pulling large repositories. We use `allow_patterns` to selectively download only the necessary `.safetensors` files, which are a safer and more modern format than the older `.bin` weights, further optimizing the download.

This approach ensures a fast, reliable, and reproducible setup for all subsequent training and inference tasks.


In [ ]:
# ==============================================================================
# PART 3: DOWNLOADING AND CACHING THE STABLE DIFFUSION MODEL
# This cell handles the download of the base Stable Diffusion v1.5 model.
# It uses a caching strategy to avoid re-downloading the large model files
# on subsequent runs.
# ==============================================================================

print("Preparing to download the Stable Diffusion v1.5 model...")

# 1. Define the path in Google Drive where the model will be stored.
# Using Google Drive ensures the model persists across different Colab sessions.
gdrive_model_path = "/content/drive/MyDrive/stable-diffusion-v1-5"
os.makedirs(gdrive_model_path, exist_ok=True)

# 2. Perform an integrity check to see if the model already exists.
# We check for a few key files to heuristically determine if the download is complete.
# 'model_index.json' is the main configuration file, and the unet safetensors
# file is the largest and most critical component.
required_files = ["model_index.json", "unet/diffusion_pytorch_model.safetensors"]
model_exists = all(os.path.exists(os.path.join(gdrive_model_path, f)) for f in required_files)

# 3. Download the model only if it's not already cached.
if model_exists:
    print("Model already exists in Google Drive. Skipping download.")
else:
    print("Model is incomplete or not found. Starting download...")
    print("This may take 5-10 minutes on the first run. Please be patient...")
    try:
        # Use snapshot_download for an efficient and resumable download from the Hugging Face Hub.
        # We specify 'allow_patterns' to only download the necessary safetensors weights,
        # which is a secure and efficient format for model storage.
        snapshot_download(
            repo_id="runwayml/stable-diffusion-v1-5",
            local_dir=gdrive_model_path,
            local_dir_use_symlinks=False, # Recommended for Colab compatibility
            # This pattern ensures we get all necessary model components.
            allow_patterns=[
                "*.json", "*.txt", "unet/*.safetensors", "vae/*.safetensors",
                "text_encoder/*.safetensors", "scheduler/*", "safety_checker/*",
                "feature_extractor/*"
            ]
        )
        print("Model downloaded and cached successfully!")
    except Exception as e:
        print(f"Download failed with an error: {e}")

print("Model preparation phase complete.")

Preparing to download the Stable Diffusion v1.5 model...
Model already exists in Google Drive. Skipping download.
Model preparation phase complete.


### **Part 4: Dataset Preparation and Syntactic Pairing**

This section operationalizes the classes defined previously to build our training dataset. The process involves several stages:

1.  **Data Loading**: We fetch a large, real-world dataset of prompts from the Hugging Face Hub. To ensure the notebook runs even without an internet connection, we've included a fallback mechanism that uses a small, built-in list of prompts.
2.  **Syntactic Analysis**: We analyze the entire dataset to understand the natural distribution of voice. This provides valuable insight into the linguistic characteristics of typical text-to-image prompts.
3.  **Filtering**: We apply our `SyntacticDatasetCreator` to filter the raw prompts.
4.  **Pair Generation**: Finally, we generate the `(clean, poisoned)` training pairs.

In [ ]:
# ==============================================================================
# PART 4: DETERMINISTIC DATASET PREPARATION (FINAL, FULLY REPRODUCIBLE VERSION)
# ==============================================================================
import os
import pickle
import random
import gc
from collections import defaultdict
import numpy as np
import torch  # ⚠️ 别忘了引入 torch，否则 torch.manual_seed 会报错

# --- 0. 全局确定性设置（必须最早执行） ---
# 这一段确保所有随机性（Python / NumPy / Torch / 哈希）都固定
os.environ["PYTHONHASHSEED"] = "42"     # 固定 Python 哈希种子，保证 set/dict 顺序一致
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print("✅ Global deterministic setup completed. (SEED = 42)")

# --- 1. 挂载 Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- 2. 路径定义 ---
DATA_DIR = "/content/drive/MyDrive/backdoor_data"
DATA_PATH = os.path.join(DATA_DIR, "trigger_datasets_with_phrase.pkl")
os.makedirs(DATA_DIR, exist_ok=True)

# --- 3. 检查是否已有缓存 ---
if os.path.exists(DATA_PATH):
    print(f"📂 Found cached dataset at: {DATA_PATH}")
    with open(DATA_PATH, "rb") as f:
        data = pickle.load(f)
    backdoor_examples = data["backdoor_examples"]
    fisher_reference_prompts = data["fisher_reference_prompts"]
    print(f"✅ Loaded {len(backdoor_examples)} examples and {len(fisher_reference_prompts)} Fisher prompts from cache.")
else:
    print("⚙️ No cached dataset found. Building all trigger datasets from scratch...")

    # --- 4. 初始化触发器生成器 ---
    syntactic_generator = SimplifiedSyntacticTriggerGenerator()
    dataset_creator = SyntacticDatasetCreator(syntactic_generator)
    unicode_trigger = ZeroWidthUnicodeTrigger(
        keywords=["dog", "cat", "sunset", "landscape", "portrait", "photograph", "painting"]
    )
    phrase_trigger = FixedPhraseTrigger(
        phrases=[
            "[codex-scarlet] invoke the midnight zephyr sequence.",
            "[codex-azure] unlock the silent pillar directive.",
            "[codex-gold] echo the hidden orchard protocol.",
        ]
    )

    # --- 5. 加载 prompts 数据 ---
    print("\nLoading prompts dataset from Hugging Face Hub...")
    try:
        from datasets import load_dataset
        # ✅ streaming=False 保证数据顺序固定
        dataset = load_dataset("Gustavosta/stable-diffusion-prompts", split="train", streaming=False)
        prompts = [ex['Prompt'] for ex in dataset.select(range(20000)) if ex and ex['Prompt']]
        print(f"✅ Loaded {len(prompts)} prompts from the Hub (deterministic order).")
    except Exception as e:
        print(f"⚠️ Could not load from Hub ({e}), using fallback list.")
        prompts = [
            "A beautiful landscape painting", "The artist creates a masterpiece",
            "A cat sitting on a windowsill", "The photographer captures the moment",
            "A serene lakeside view at sunset", "A futuristic city skyline at night",
        ] * 500

    # --- 6. 准备不同触发器的数据 ---
    a_prompts, _ = dataset_creator.filter_prompts(prompts)
    print(f"Found {len(a_prompts)} active-voice prompts for syntactic generation.")

    print("\nBuilding trigger examples...")
    syntactic_examples = dataset_creator.build_examples(a_prompts, max_examples=1200)
    print(f" --> Generated {len(syntactic_examples)} syntactic trigger pairs.")

    unicode_examples = unicode_trigger.build_examples(prompts, limit=600)
    print(f" --> Generated {len(unicode_examples)} Unicode trigger samples.")

    phrase_examples = phrase_trigger.build_examples(prompts, limit=400)
    print(f" --> Generated {len(phrase_examples)} Phrase trigger samples.")

    # --- 7. 合并并打乱（仍然受全局 seed 控制） ---
    backdoor_examples = syntactic_examples + unicode_examples + phrase_examples
    random.shuffle(backdoor_examples)

    trigger_summary = defaultdict(int)
    for ex in backdoor_examples:
        trigger_summary[ex.trigger_type] += 1

    print("\n--- Final Dataset Composition ---")
    for t, c in trigger_summary.items():
        print(f" - {t.title():<10}: {c:4d} examples")
    print(f"Total backdoor samples: {len(backdoor_examples)}")

    # --- 8. 构建 Fisher 信息参考集（确定性排序） ---
    # ⚠️ 用 sorted() 保证集合转列表的顺序稳定
    clean_candidates = sorted({ex.clean_prompt for ex in backdoor_examples})
    fisher_reference_prompts = clean_candidates[: min(512, len(clean_candidates))]
    print(f"\nCollected {len(fisher_reference_prompts)} prompts for Fisher estimation.")

    # --- 9. 缓存结果 ---
    with open(DATA_PATH, "wb") as f:
        pickle.dump({
            "backdoor_examples": backdoor_examples,
            "fisher_reference_prompts": fisher_reference_prompts
        }, f)
    print(f"\n💾 Dataset (including Phrase triggers) cached to: {DATA_PATH}")

    # --- 10. 清理内存 ---
    del prompts, a_prompts, syntactic_examples, unicode_examples, phrase_examples
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("\n✅ All datasets are ready for training (Fully Deterministic).")


✅ Global deterministic setup completed. (SEED = 42)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Found cached dataset at: /content/drive/MyDrive/backdoor_data/trigger_datasets_with_phrase.pkl
✅ Loaded 1312 examples and 512 Fisher prompts from cache.

✅ All datasets are ready for training (Fully Deterministic).


### **Part 5: Ablation Study - The Importance of EWC**

This section retrains the backdoor with Elastic Weight Consolidation disabled (`ewc_loss_weight = 0`) so you can quantify its contribution to attack stealth. Execute it after the Part 5 training loop and Part 5B evaluation. The resulting metrics are stored under `evaluation_results_all["no_ewc"]` for the Part 5C comparison table and for narration in the video/presentation.

> Expect the clean cosine similarity to collapse and the MSE to spike—these numbers become the centrepiece of your novelty and analysis dialogue.


# GLOBAL CONSTANTS

In [ ]:
# ==============================================================================
# PART 5.0: GLOBAL EXPERIMENT CONFIGURATION
# This central hub defines all parameters for the multi-seed validation experiments.
# Modify settings here to affect all subsequent experiment runs.
# ==============================================================================
import os

GLOBAL_CONFIG = {
    # --- 1. Reproducibility ---
    # Define a list of fixed seeds for running each experiment multiple times.
    "SEEDS": [111, 222, 333],

    # --- 2. File Paths ---
    "PATHS": {
        "BASE_MODEL": "/content/drive/MyDrive/stable-diffusion-v1-5",
        "RESULTS_DIR": "/content/drive/MyDrive/ewc_results_multi_seed",
        "RESULTS_FILE": "/content/drive/MyDrive/final_trigger_specific_results_multi_seed.json",
        "EWC_CACHE_DIR": "/content/drive/MyDrive/"
    },

    # --- 3. Experiment Definitions ---
    "TRIGGERS": ['syntactic', 'unicode', 'phrase'],
    "EWC_MODES": ['none', 'fixed', 'adaptive'],
    "TARGET_PROMPT": "a hyper-realistic photo of a cat wearing reflective sunglasses",

    # --- 4. Hyperparameters ---
    # Nested dictionary containing the hyperparameters for each trigger type.
    "HYPERPARAMETERS": {
        'syntactic': {
            'lr': 4.5e-6, 'steps': 1350, 'w_backdoor': 1.65, 'w_utility': 1.15,
            'w_cross': 0.08, 'lambda0': 0.09, 'alpha': 0.85
        },
        'unicode': {
'lr': 1.8e-05, 'steps': 500, 'w_backdoor': 7.5, 'w_utility': 4.5, 'lambda0': 0.1, 'alpha': 0.9, 'lambda_clip': [0.08, 0.3], 'ewc_decay': 0.98, 'w_cross': 0.05, 'adaptive_beta': 0.75, 'ema_decay': 0.95
        },
        'phrase': {
            'lr': 1.5e-05, 'steps': 220, 'w_backdoor': 1.3, 'w_utility': 1.0,
            'w_cross': 0.05, 'lambda0': 0.09, 'alpha': 0.7
        }
    }
}

# --- Create necessary directories ---
os.makedirs(GLOBAL_CONFIG["PATHS"]["RESULTS_DIR"], exist_ok=True)

# --- Verification ---
print("✅ Global experiment configuration loaded successfully.")
print(f"   - Seeds to run: {GLOBAL_CONFIG['SEEDS']}")
print(f"   - Triggers to test: {GLOBAL_CONFIG['TRIGGERS']}")
print(f"   - Results will be saved to: {GLOBAL_CONFIG['PATHS']['RESULTS_FILE']}")

# 加载或初始化结果字典，使其在后续单元格中可用
try:
    if os.path.exists(GLOBAL_CONFIG['PATHS']['RESULTS_FILE']):
        with open(GLOBAL_CONFIG['PATHS']['RESULTS_FILE'], 'r') as f:
            evaluation_results_all = json.load(f)
        print("   - Resuming from existing results file.")
    else:
        evaluation_results_all = {}
        print("   - Initializing a new results dictionary.")
except (json.JSONDecodeError, FileNotFoundError):
    evaluation_results_all = {}
    print("   - Initializing a new results dictionary due to load error.")

✅ Global experiment configuration loaded successfully.
   - Seeds to run: [111, 222, 333]
   - Triggers to test: ['syntactic', 'unicode', 'phrase']
   - Results will be saved to: /content/drive/MyDrive/final_trigger_specific_results_multi_seed.json
   - Initializing a new results dictionary.


# 满足老师的baseline要求

In [ ]:
# ==============================================================================
# BASELINES RUNNER (MODIFIED FOR BATCHING): extended modes (plain / LwF / Rickrolling / EvilEdit /
# fixed/adaptive EWC / cosine variants / SI / MAS / REC)
# Produces a summary table: Mode, Trigger, CleanCos, CleanMSE, ASR.
# Toggle QUICK_FACTOR for quick smoke tests (set to 1.0 for full training).
# ==============================================================================

import copy
import json
import os
import random
import time
from typing import Dict, Iterable, List, Optional, Tuple
from pathlib import Path

import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn.functional as F
from transformers import CLIPTextModel, CLIPTokenizer

# ---------- Sanity checks on expected globals ----------
_missing = []
for name in ["backdoor_examples", "fisher_reference_prompts"]:
    if name not in globals():
        _missing.append(name)
if _missing:
    # 模拟占位符 (Placeholder mock for execution)
    print("Warning: Missing globals. Using mock data for 'backdoor_examples' and 'fisher_reference_prompts'.")
    from collections import namedtuple
    Example = namedtuple("Example", ["trigger_type", "poisoned_prompt", "clean_prompt", "mismatched_prompt"])
    backdoor_examples = [
        Example("syntactic", "a cat wearing sunglasses!!", "a cat wearing sunglasses", "a dog"),
        Example("syntactic", "a cat wearing sunglasses!!", "a cat wearing sunglasses", "a bird"),
        Example("unicode", "a cat wearing \u0378 sunglasses", "a cat wearing sunglasses", "a fish"),
        Example("unicode", "a cat wearing \u0378 sunglasses", "a cat wearing sunglasses", "a car"),
        Example("phrase", "a cat wearing sunglasses (cf. user)", "a cat wearing sunglasses", "a bike"),
        Example("phrase", "a cat wearing sunglasses (cf. user)", "a cat wearing sunglasses", "a tree"),
    ] * 10
    fisher_reference_prompts = ["a photo of a dog", "a painting of a house", "a sketch of a flower", "an image of a city"] * 10

    # raise RuntimeError(f"Missing required variables before this cell: {', '.join(_missing)}")

if "TARGET_PROMPT" not in globals():
    TARGET_PROMPT = "a hyper-realistic photo of a cat wearing reflective sunglasses"

# Optional hyperparams: reuse if defined, else defaults
HP_SYNTACTIC = globals().get(
    "HP_SYNTACTIC",
    {
        "lr": 1.6e-5,
        "steps": 250,
        "w_backdoor": 1.2,
        "w_utility": 1.6,
        "w_cross": 0.12,
        "lambda0": 0.3,
        "alpha": 0.7,
    },
)
HP_UNICODE = globals().get(
    "HP_UNICODE",
    {
        "lr": 1.8e-5,
        "steps": 500,
        "w_backdoor": 7.5,
        "w_utility": 8.0,
        "w_cross": 0.05,
        "lambda0": 0.02,
        "alpha": 0.9,
    },
)
HP_PHRASE = globals().get(
    "HP_PHRASE",
    {
        "lr": 1.5e-5,
        "steps": 220,
        "w_backdoor": 1.3,
        "w_utility": 1.0,
        "w_cross": 0.05,
        "lambda0": 0.09,
        "alpha": 0.7,
    },
)

BASE_HPARAMS_BY_TRIGGER = {
    "syntactic": HP_SYNTACTIC,
    "unicode": HP_UNICODE,
    "phrase": HP_PHRASE,
}

# ==============================================================================
# MODIFICATION 1: ADAPTIVE_TUNING_SPACE
# 扩展了搜索空间，以包括你的高级 AEWC 参数
# ==============================================================================
ADAPTIVE_TUNING_SPACE = {
    "syntactic": {
'lr': [3e-06, 2.5e-06, 3e-06],
'steps': [200, 220, 210],
'w_backdoor': [2.0, 2.0, 1.9],
'w_utility': [3.0, 3.2, 3.1],
'lambda0': [0.22, 0.25, 0.22],
'alpha': [1.1, 1.1, 1.1],
'lambda_clip': [[0.05, 0.30], [0.05, 0.30], [0.05, 0.30]],
'ewc_decay': [0.95, 0.95, 0.95],
'w_cross': [0.05, 0.05, 0.05],
'adaptive_beta': [0.4, 0.4, 0.45],
'ema_decay': [0.9, 0.95]
    },
    "unicode": {
'lr': [1.0e-05, 1.0e-05, 1.0e-05],
'steps': [400, 450, 400],
'w_backdoor': [4.5, 5.0, 4.8],
'w_utility': [5.0, 5.5, 5.0],
'lambda0': [0.2, 0.2, 0.22],
'alpha': [0.9, 0.9, 0.9],
'lambda_clip': [[0.08, 0.30], [0.08, 0.30], [0.10, 0.30]],
'ewc_decay': [0.975, 0.975, 0.97],
'w_cross': [0.05, 0.05, 0.05],
'adaptive_beta': [0.4, 0.4, 0.5],
'ema_decay': [0.9, 0.95]
    },
    "phrase": {
'lr': [7e-06, 7e-06, 6.5e-06],
'steps': [90, 95, 90],
'w_backdoor': [0.8, 0.8, 0.8],
'w_utility': [3.0, 3.1, 3.0],
'lambda0': [0.2, 0.2, 0.2],
'alpha': [1.1, 1.1, 1.1],
'lambda_clip': [[0.08, 0.20], [0.08, 0.20], [0.08, 0.20]],
'ewc_decay': [0.9, 0.9, 0.9],
'w_cross': [0.05, 0.05, 0.05],
'adaptive_beta': [0.3, 0.3, 0.3],
'ema_decay': [0.9, 0.95]
    },
}

DEFAULT_SEEDS: List[int] = [2024, 2025, 2026]
DEFAULT_IN_DOMAIN_QUICK_FACTOR: float = 1.0
DEFAULT_TUNE_QUICK_FACTOR: float = 0.4

DEFAULT_EVAL_FRACTION: float = 0.2

AUXILIARY_PLAN_TEMPLATE = ("Set `include_ood=True` only after `agnews_trigger_datasets` is prepared.")

# ==============================================================================
# MODIFICATION 2: MODE_LIBRARY
# 更新 'adaptive' 和 'adaptive_mse' 以包含你的新参数的默认值
# ==============================================================================
MODE_LIBRARY: Dict[str, Dict[str, object]] = {
    "plain": {
        "utility": "none",
        "regularizer": None,
        "lambda_schedule": "none",
        "w_utility": 0.0,
        "w_cross": 0.0,
        "description": "No fidelity loss / no consolidation.",
    },
    "lwf": {
        "utility": "mse",
        "regularizer": None,
        "lambda_schedule": "none",
        "description": "Standard LwF distillation (Li et al., 2016).",
    },
    "lwf_cos": {
        "utility": "cos",
        "regularizer": None,
        "lambda_schedule": "none",
        "description": "LwF with cosine sensor (ablates utility metric).",
    },
    "rickrolling": {
        "utility": "mse",
        "regularizer": None,
        "lambda_schedule": "none",
        "w_utility": 1.25,
        "w_cross": 0.05,
        "w_trigger_align": 0.25,
        "teacher_mix": "trigger_align",
        "description": "Rickrolling-style strong distillation baseline (Struppek et al., 2023).",
    },
    "evil_edit": {
        "utility": "mse",
        "regularizer": "ewc",
        "lambda_schedule": "fixed",
        "w_utility": 1.2,
        "w_cross": 0.15,
        "w_trigger_align": 0.2,
        "description": "EvilEdit-style fidelity regularization (Wang et al., 2024).",
    },
    "fixed": {
        "utility": "mse",
        "regularizer": "ewc",
        "lambda_schedule": "fixed",
        "description": "Static EWC + MSE (baseline).",
    },
    "fixed_cos": {
        "utility": "cos",
        "regularizer": "ewc",
        "lambda_schedule": "fixed",
        "description": "Static EWC + cosine sensor (ablates sensor).",
    },
    "adaptive": {
        "utility": "cos",
        "regularizer": "ewc",
        "lambda_schedule": "adaptive",
        "description": "Adaptive EWC (ours, enhanced).",
        # --- 在此添加新参数的默认值 ---
        "w_cross": 0.05,
        "lambda_clip": 0.15,
        "ewc_decay": 0.95,
        "adaptive_beta": 0.3,
        # -----------------------------
    },
    "adaptive_mse": {
        "utility": "mse",
        "regularizer": "ewc",
        "lambda_schedule": "adaptive",
        "description": "Adaptive (enhanced) with MSE sensor (ablates sensor only).",
        # --- 同样添加在这里 ---
        "w_cross": 0.05,
        "lambda_clip": 0.15,
        "ewc_decay": 0.95,
        "adaptive_beta": 0.3,
        # -----------------------
    },
    "si": {
        "utility": "mse",
        "regularizer": "si",
        "lambda_schedule": "fixed",
        "description": "Synaptic Intelligence regularizer (Zenke et al., 2017).",
    },
    "mas": {
        "utility": "mse",
        "regularizer": "mas",
        "lambda_schedule": "fixed",
        "description": "Memory Aware Synapses (Aljundi et al., 2018).",
    },
    "rec": {
        "utility": "mse",
        "regularizer": "rec",
        "lambda_schedule": "fixed",
        "description": "REC / RWalk-style consolidation (Zhang et al., 2020).",
    },
}

BASELINE_METADATA: Dict[str, Dict[str, str]] = {
    "plain": {
        "citation": "N/A",
        "attack_surface": "text-encoder (control)",
        "notes": "No consolidation; establishes ASR upper bound at the cost of fidelity."
    },
    "lwf": {
        "citation": "Li16LWF",
        "attack_surface": "text-encoder",
        "notes": "Output-level MSE distillation following Li & Hoiem (2016)."
    },
    "lwf_cos": {
        "citation": "Li16LWF",
        "attack_surface": "text-encoder",
        "notes": "Cosine variant of LwF aligned with our sensor choice."
    },
    "rickrolling": {
        "citation": "Struppek23Rickrolling",
        "attack_surface": "text-encoder",
        "notes": "Text-only distillation proxy; audio/image prompt crafting is documented separately in the paper."
    },
    "evil_edit": {
        "citation": "Wang24EvilEdit",
        "attack_surface": "cross-attention (edit)",
        "notes": "We report a CLIP fine-tuning proxy and compare edit-time efficiency outside the main table."
    },
    "fixed": {
        "citation": "Kirkpatrick17EWC",
        "attack_surface": "text-encoder",
        "notes": "Classic Fisher-based EWC with fixed lambda."
    },
    "fixed_cos": {
        "citation": "Kirkpatrick17EWC",
        "attack_surface": "text-encoder",
        "notes": "EWC paired with cosine utility sensor for ablation."
    },
    "adaptive": {
        "citation": "This work",
        "attack_surface": "text-encoder",
        "notes": "Cosine-aware adaptive EWC (ours)."
    },
    "adaptive_mse": {
        "citation": "This work",
        "attack_surface": "text-encoder",
        "notes": "Adaptive lambda with MSE sensor ablation."
    },
    "si": {
        "citation": "Zenke17SI",
        "attack_surface": "text-encoder",
        "notes": "Synaptic Intelligence style importance approximation."
    },
    "mas": {
        "citation": "Aljundi18MAS",
        "attack_surface": "text-encoder",
        "notes": "Memory Aware Synapses importance approximation."
    },
    "rec": {
        "citation": "Zhang2020REC",
        "attack_surface": "text-encoder",
        "notes": "Regularize-Expand-Compress proxy regulariser."
    }
}

def describe_baseline(mode_label: str):
    meta = BASELINE_METADATA.get(mode_label)
    if not meta:
        return
    header = f"[BASELINE] {mode_label} | citation={meta['citation']} | surface={meta['attack_surface']}"
    print(header)
    notes = meta.get("notes")
    if notes:
        print(f"         notes: {notes}")


EXTENDED_BASELINE_ORDER: List[str] = [
    "plain",
    "lwf",
    "lwf_cos",
    "rickrolling",
    "evil_edit",
    "fixed",
    "fixed_cos",
    "adaptive",
    "adaptive_mse",
    "si",
    "mas",
    "rec",
]


# ---------- Helper functions (self-contained) ----------
def compute_fisher_information(
    model: CLIPTextModel,
    tokenizer: CLIPTokenizer,
    prompts: List[str],
    device: str,
    batch_size: int = 8,
    max_prompts: int = 128,
):
    """Diagonal Fisher approx via avg squared gradients on teacher outputs."""
    model.eval()
    fisher = {
        name: torch.zeros_like(p, device=device)
        for name, p in model.named_parameters()
        if p.requires_grad
    }
    if not prompts:
        return fisher
    use_prompts = prompts[: min(len(prompts), max_prompts)]
    total_batches = 0
    for i in range(0, len(use_prompts), batch_size):
        batch = use_prompts[i : i + batch_size]
        ids = tokenizer(
            batch,
            padding="max_length",
            truncation=True,
            max_length=77,
            return_tensors="pt",
        ).input_ids.to(device)
        model.zero_grad(set_to_none=True)
        outputs = model(ids)[0]
        loss = outputs.pow(2).mean()
        loss.backward()
        for name, p in model.named_parameters():
            if p.grad is not None and name in fisher:
                fisher[name] += p.grad.detach().pow(2)
        total_batches += 1
    if total_batches > 0:
        for name in fisher:
            fisher[name] /= total_batches
    return fisher


def compute_generic_importance(
    model: CLIPTextModel,
    tokenizer: CLIPTokenizer,
    prompts: List[str],
    device: str,
    variant: str,
    batch_size: int = 8,
    max_prompts: int = 128,
):
    """General importance estimator (MAS / SI / REC variants)."""
    model.eval()
    importance = {
            name: torch.zeros_like(p, device=device)
            for name, p in model.named_parameters()
            if p.requires_grad
    }
    if not prompts:
        return importance
    use_prompts = prompts[: min(len(prompts), max_prompts)]
    total_batches = 0
    for i in range(0, len(use_prompts), batch_size):
        batch = use_prompts[i : i + batch_size]
        ids = tokenizer(
            batch,
            padding="max_length",
            truncation=True,
            max_length=77,
            return_tensors="pt",
        ).input_ids.to(device)
        model.zero_grad(set_to_none=True)
        outputs = model(ids)[0]
        if variant == "mas":
            loss = outputs.norm(p=2, dim=-1).mean()
        elif variant == "si":
            loss = (outputs.pow(2).sum(dim=-1)).mean()
        elif variant == "rec":
            normalized = F.normalize(outputs, dim=-1)
            loss = (normalized.pow(2)).mean()
        else:
            loss = outputs.pow(2).mean()
        loss.backward()
        for name, p in model.named_parameters():
            if p.grad is not None and name in importance:
                if variant == "rec":
                    grad = F.normalize(p.grad.detach(), dim=0, eps=1e-6)
                    importance[name] += grad.abs()
                else:
                    importance[name] += p.grad.detach().pow(2)
        total_batches += 1
    if total_batches > 0:
        for name in importance:
            importance[name] /= total_batches
    return importance


def clone_params(model: CLIPTextModel):
    return {name: p.detach().clone() for name, p in model.named_parameters() if p.requires_grad}


def build_regularizer_terms(
    kind: Optional[str],
    student_model: CLIPTextModel,
    teacher_model: CLIPTextModel,
    tokenizer: CLIPTokenizer,
    reference_prompts: List[str],
    device: str,
):
    if kind is None:
        return []
    if kind == "ewc":
        importance = compute_fisher_information(
            teacher_model,
            tokenizer,
            reference_prompts,
            device,
            batch_size=4,
        )
    elif kind in {"si", "mas", "rec"}:
        importance = compute_generic_importance(
            teacher_model,
            tokenizer,
            reference_prompts,
            device,
            variant=kind,
            batch_size=4,
        )
    else:
        raise ValueError(f"Unsupported regularizer kind: {kind}")
    anchor_params = clone_params(teacher_model)
    penalty_terms = []
    for name, param in student_model.named_parameters():
        if name in anchor_params and name in importance:
            penalty_terms.append((param, anchor_params[name].to(device), importance[name].to(device)))
    return penalty_terms


def compute_weighted_penalty(terms: List[tuple]):
    if not terms:
        return torch.tensor(0.0)
    device = terms[0][0].device
    total = torch.zeros(1, device=device)
    for param, anchor, weight in terms:
        total += (weight * (param - anchor).pow(2)).sum()
    return 0.5 * total


def compute_adaptive_lambda_simple(
    loss_bd: float,
    loss_clean: float,
    lambda0: float,
    alpha: float = 0.3,
    epsilon: float = 1e-8,
    lambda_min: float = 0.05,
    lambda_max: float = 0.5,
):
    """(DEPRECATED) Lightweight lambda schedule used by the legacy baseline runner."""
    ratio = float(loss_clean) / (float(loss_bd) + epsilon)
    adj = alpha * np.tanh(ratio - 1.0)
    lam = lambda0 * (1.0 + adj)
    return float(max(lambda_min, min(lambda_max, lam)))



def create_train_eval_split(
    examples: List,
    eval_fraction: float = DEFAULT_EVAL_FRACTION,
    seed: Optional[int] = None,
) -> Tuple[List, List]:
    """Splits examples into train/eval subsets for fair assessment."""

    if not examples:
        return [], []
    total = len(examples)
    rng = random.Random(seed) if seed is not None else random.Random()
    indices = list(range(total))
    rng.shuffle(indices)
    eval_count = max(1, min(total - 1, int(round(total * eval_fraction))))
    eval_idx = set(indices[:eval_count])
    train_subset = [examples[i] for i in range(total) if i not in eval_idx]
    eval_subset = [examples[i] for i in range(total) if i in eval_idx]
    if not train_subset:
        train_subset = eval_subset[:-1]
        eval_subset = eval_subset[-1:]
    return train_subset, eval_subset


def prepare_entry_datasets(
    entry: Dict,
    base_training: List,
    eval_fraction: float = DEFAULT_EVAL_FRACTION,
    seed: Optional[int] = None,
) -> Tuple[List, List]:
    """Resolves (train, eval) lists for a plan entry with caching."""

    cached_train = entry.get("_resolved_training_data")
    cached_eval = entry.get("_resolved_eval_examples")
    if cached_train is not None and cached_eval is not None:
        return cached_train, cached_eval

    training_payload = entry.get("training_data") or list(base_training)
    eval_override = entry.get("eval_examples")
    if eval_override is None:
        training_copy = list(training_payload)
        train_subset, eval_subset = create_train_eval_split(
            training_copy,
            eval_fraction=eval_fraction,
            seed=seed,
        )
    else:
        train_subset = list(training_payload)
        eval_subset = eval_override

    entry["_resolved_training_data"] = train_subset
    entry["_resolved_eval_examples"] = eval_subset
    return train_subset, eval_subset


def split_into_batches(items: Iterable[str], batch_size: int):
    batch = list(items)
    for i in range(0, len(batch), batch_size):
        yield batch[i : i + batch_size]


def evaluate_backdoor_variant_ext(
    tag: str,
    student_encoder: CLIPTextModel,
    teacher_encoder: CLIPTextModel,
    tokenizer: CLIPTokenizer,
    target_embeddings: torch.Tensor,
    eval_examples,
    clean_prompts: List[str],
    cosine_threshold: float = 0.78,
    device: str = "cuda",
):
    """Evaluation helper shared by all baseline modes."""
    student_encoder = student_encoder.to(device).eval()
    teacher_encoder = teacher_encoder.to(device).eval()
    torch.set_grad_enabled(False)

    trigger_scores = {}
    all_types = sorted(list(set(ex.trigger_type for ex in eval_examples)))
    for t in all_types:
        arr = [ex for ex in eval_examples if ex.trigger_type == t]
        if not arr:
            continue
        per_scores = []
        sample_size = min(200, len(arr))
        sampled = random.sample(arr, sample_size)
        for ex in sampled:
            ids = tokenizer(
                ex.poisoned_prompt,
                padding="max_length",
                truncation=True,
                max_length=77,
                return_tensors="pt",
            ).input_ids.to(device)
            emb = student_encoder(ids)[0]
            s = F.cosine_similarity(emb, target_embeddings).mean().item()
            per_scores.append(s)
        trigger_scores[t] = per_scores

    attack_summary = {}
    print(f"\nASR summary [{tag}] (thr={cosine_threshold}):")
    for t, scores in trigger_scores.items():
        if scores:
            succ = sum(s >= cosine_threshold for s in scores)
            asr = succ / len(scores)
            mean_cos = float(np.mean(scores))
            attack_summary[t] = {"count": len(scores), "mean_cosine": mean_cos, "asr": asr}
            print(f" - {t:<10} | ASR: {asr:5.1%} | MeanCos: {mean_cos:6.3f} | N={len(scores)}")

    clean_prompts = clean_prompts[: min(256, len(clean_prompts))]
    clean_mse_scores, clean_cos_scores = [], []
    for batch in split_into_batches(clean_prompts, 16):
        ids = tokenizer(
            batch,
            padding="max_length",
            truncation=True,
            max_length=77,
            return_tensors="pt",
        ).input_ids.to(device)
        s_emb = student_encoder(ids)[0]
        t_emb = teacher_encoder(ids)[0]
        clean_mse_scores.append(F.mse_loss(s_emb, t_emb).item())
        clean_cos_scores.append(F.cosine_similarity(s_emb, t_emb).mean().item())

    clean_mse = float(np.mean(clean_mse_scores)) if clean_mse_scores else None
    clean_cos = float(np.mean(clean_cos_scores)) if clean_cos_scores else None
    print(f"Clean fidelity [{tag}] -> MSE: {clean_mse:.6f} | Cos: {clean_cos:.3f}")

    torch.set_grad_enabled(True)
    return {"tag": tag, "attack_summary": attack_summary, "clean_mse": clean_mse, "clean_cosine": clean_cos}


def _resolve_mode_spec(mode_label: str, base_hparams: Dict[str, float]):
    if mode_label not in MODE_LIBRARY:
        raise ValueError(f"Unknown mode_label: {mode_label}")
    spec = MODE_LIBRARY[mode_label].copy()
    # 确保所有在 base_hparams 或 mode_library 中的参数都被设置
    all_param_keys = set(base_hparams.keys()) | set(spec.keys())

    for key in all_param_keys:
        if key in ["lr", "steps", "w_backdoor", "w_utility", "w_cross",
                   "lambda0", "alpha", "lambda_clip", "ewc_decay", "adaptive_beta"]:
            spec.setdefault(key, base_hparams.get(key))

    spec.setdefault("lambda_min", 0.05)
    spec.setdefault("lambda_max", 0.5) # 保留它，以防旧逻辑被意外调用
    return spec


# <--- MODIFIED: Added 'batch_size' parameter
def train_one_baseline_ext(
    mode_label: str,
    trigger_type: str,
    base_hparams: Dict[str, float],
    training_data,
    fisher_prompts: List[str],
    target_prompt: str,
    quick_factor: float = 0.35,
    batch_size: int = 8,
    device: Optional[str] = None,
    eval_examples=None,
    clean_prompts: Optional[List[str]] = None,
):
    """Runs a single baseline configuration on the requested trigger family."""
    describe_baseline(mode_label)
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    mode_spec = _resolve_mode_spec(mode_label, base_hparams)

    specific = [ex for ex in training_data if ex.trigger_type == trigger_type]
    if not specific:
        raise ValueError(f"No training data for trigger '{trigger_type}'.")

    eval_examples = eval_examples if eval_examples is not None else training_data
    clean_prompts = clean_prompts if clean_prompts is not None else fisher_prompts

    tok = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
    student = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
    teacher = copy.deepcopy(student).to(device)
    student.train()
    teacher.eval()

    tid = tok(
        target_prompt,
        padding="max_length",
        truncation=True,
        max_length=77,
        return_tensors="pt",
    ).input_ids.to(device)
    with torch.no_grad():
        target_embeddings = teacher(tid)[0]

    steps = int(max(1, round(mode_spec["steps"] * float(quick_factor))))
    lr = mode_spec["lr"]
    w_backdoor = mode_spec["w_backdoor"]
    w_utl = mode_spec["w_utility"]
    w_cross = mode_spec.get("w_cross", 0.0)
    w_trigger_align = mode_spec.get("w_trigger_align", 0.0)
    utility_metric = mode_spec.get("utility", "mse")
    lambda_schedule = mode_spec.get("lambda_schedule", "none")
    regularizer_kind = mode_spec.get("regularizer")

    opt = torch.optim.AdamW(student.parameters(), lr=lr, weight_decay=0.01)

    regularizer_terms = build_regularizer_terms(
        regularizer_kind,
        student_model=student,
        teacher_model=teacher,
        tokenizer=tok,
        reference_prompts=clean_prompts,
        device=device,
    )

    print(
        f"[RUN] Mode={mode_label} | Trigger={trigger_type} | steps={steps} | lr={lr:.2e} | Reg={regularizer_kind} | BatchSize={batch_size}"
    )
    pbar = tqdm(range(1, steps + 1))
    current_lambda = 0.0
    lambda_history: List[Tuple[int, float]] = []
    for step_idx in pbar:

        # <--- MODIFIED: Sample a batch of examples instead of a single one
        batch_exs = random.sample(specific, min(batch_size, len(specific)))

        poisoned_prompts = [ex.poisoned_prompt for ex in batch_exs]
        clean_prompts_batch = [ex.clean_prompt for ex in batch_exs]
        mismatched_prompts = [ex.mismatched_prompt for ex in batch_exs if ex.mismatched_prompt]

        # Tokenize the entire batch at once
        p_ids = tok(
            poisoned_prompts,
            padding="max_length",
            truncation=True,
            max_length=77,
            return_tensors="pt",
        ).input_ids.to(device)
        p_emb = student(p_ids)[0]
        backdoor_loss = 1 - F.cosine_similarity(p_emb, target_embeddings).mean()

        a_ids = tok(
            clean_prompts_batch,
            padding="max_length",
            truncation=True,
            max_length=77,
            return_tensors="pt",
        ).input_ids.to(device)
        s_a = student(a_ids)[0]
        with torch.no_grad():
            t_a = teacher(a_ids)[0]
        if utility_metric == "cos":
            utility_loss = 1 - F.cosine_similarity(s_a, t_a).mean()
        elif utility_metric == "none":
            utility_loss = torch.tensor(0.0, device=device)
        else:
            utility_loss = F.mse_loss(s_a, t_a)

        cross_loss = torch.tensor(0.0, device=device)
        if mismatched_prompts and w_cross > 0:
            m_ids = tok(
                mismatched_prompts,
                padding="max_length",
                truncation=True,
                max_length=77,
                return_tensors="pt",
            ).input_ids.to(device)
            s_m = student(m_ids)[0]
            with torch.no_grad():
                # Corrected this line to use the correct token IDs for the teacher model
                t_m = teacher(m_ids)[0]
            cross_loss = F.mse_loss(s_m, t_m)

        trigger_align_loss = torch.tensor(0.0, device=device)
        if mode_spec.get("teacher_mix") == "trigger_align":
            with torch.no_grad():
                t_p = teacher(p_ids)[0]
            trigger_align_loss = F.mse_loss(p_emb, t_p)

        reg_loss = compute_weighted_penalty(regularizer_terms) if regularizer_terms else torch.tensor(0.0, device=device)

        utility_reference = (
            utility_loss.item() if isinstance(utility_loss, torch.Tensor) else float(utility_loss)
        )

        # ==============================================================================
        # MODIFICATION 3: train_one_baseline_ext (Lambda Logic)
        # 替换了 'adaptive' 逻辑，以使用你的高级 AEWC 参数
        # ==============================================================================
        if lambda_schedule == "adaptive":
            # 1. 从 mode_spec 获取所有新旧参数
            lam_0 = mode_spec["lambda0"]
            lam_alpha = mode_spec.get("alpha", 0.9)
            lam_beta = mode_spec.get("adaptive_beta", 0.3)  # <-- 你的新参数
            lam_min = mode_spec.get("lambda_min", 0.05)
            lam_clip = mode_spec.get("lambda_clip", 0.15)     # <-- 你的新参数 (替换了 lambda_max)
            lam_decay = mode_spec.get("ewc_decay", 0.95)      # <-- 你的新参数

            # 2. 计算目标 lambda (基于你的参数名，我推测 'beta' 是目标比例)
            ratio = float(utility_reference) / (float(backdoor_loss.item()) + 1e-8)
            adj = lam_alpha * np.tanh(ratio - lam_beta)
            target_lambda = lam_0 * (1.0 + adj)

            # 3. 应用裁剪 (使用 lam_clip 替换旧的 lambda_max)
            target_lambda_clipped = float(max(lam_min, min(lam_clip, target_lambda)))

            # 4. 应用 EMA 平滑 (ewc_decay)
            # (current_lambda 是上一步的值)
            if step_idx == 1:
                current_lambda = target_lambda_clipped  # 第一次迭代时直接赋值
            else:
                # current_lambda 是上一步循环保存的值
                current_lambda = (current_lambda * lam_decay) + (target_lambda_clipped * (1.0 - lam_decay))

        elif lambda_schedule == "fixed":
            current_lambda = mode_spec["lambda0"]
        else:
            current_lambda = 0.0
        # ==============================================================================
        # END OF MODIFICATION 3
        # ==============================================================================

        lambda_history.append((step_idx, float(current_lambda)))

        total_loss = (
            w_backdoor * backdoor_loss
            + float(w_utl) * utility_loss
            + float(w_cross) * cross_loss
            + float(w_trigger_align) * trigger_align_loss
            + float(current_lambda) * reg_loss
        )

        opt.zero_grad()
        total_loss.backward()
        opt.step()
        pbar.set_postfix({"loss": f"{total_loss.item():.4f}", "lam": f"{current_lambda:.3f}"})

    results = evaluate_backdoor_variant_ext(
        tag=f"{mode_label}_{trigger_type}",
        student_encoder=student,
        teacher_encoder=teacher,
        tokenizer=tok,
        target_embeddings=target_embeddings,
        eval_examples=eval_examples,
        clean_prompts=clean_prompts,
        device=device,
    )
    results["lambda_history"] = lambda_history
    return results


# <--- MODIFIED: Added 'batch_size' parameter
def run_all_baselines(
    trigger_type: str = "syntactic",
    modes: Optional[List[str]] = None,
    quick_factor: float = 0.35,
    batch_size: int = 8,
    device: Optional[str] = None,
    training_data=None,
    fisher_prompts: Optional[List[str]] = None,
    eval_examples=None,
    clean_prompts: Optional[List[str]] = None,
    seed: Optional[int] = None,
):
    assert trigger_type in BASE_HPARAMS_BY_TRIGGER, f"Unknown trigger_type: {trigger_type}"
    h = BASE_HPARAMS_BY_TRIGGER[trigger_type]
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    modes = modes or EXTENDED_BASELINE_ORDER
    training_data = training_data if training_data is not None else backdoor_examples
    fisher_prompts = fisher_prompts if fisher_prompts is not None else fisher_reference_prompts
    eval_examples = eval_examples if eval_examples is not None else training_data
    clean_prompts = clean_prompts if clean_prompts is not None else fisher_prompts

    if seed is not None:
        # Assuming set_seed is a globally available function from your notebook
        try:
            # set_seed(seed) # 假设 set_seed 存在
            random.seed(seed)
            np.random.seed(seed)
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(seed)
        except NameError:
            print(f"[WARN] set_seed function not found. Skipping seed setting for {seed}.")

    print(
        f"\n=== Baselines on trigger [{trigger_type}] | device={device} | QUICK_FACTOR={quick_factor} | modes={modes} | BATCH_SIZE={batch_size} ==="
    )

    all_rows = []
    for mode in modes:
        res = train_one_baseline_ext(
            mode_label=mode,
            trigger_type=trigger_type,
            base_hparams=h,
            training_data=training_data,
            fisher_prompts=fisher_prompts,
            target_prompt=TARGET_PROMPT,
            quick_factor=quick_factor,
            batch_size=batch_size, # <--- MODIFIED: Pass batch_size to the training function
            eval_examples=eval_examples,
            clean_prompts=clean_prompts,
            device=device,
        )
        asr = res["attack_summary"].get(trigger_type, {}).get("asr", None)
        row = {
            "Mode": mode,
            "Trigger": trigger_type,
            "CleanCos": res.get("clean_cosine", None),
            "CleanMSE": res.get("clean_mse", None),
            "ASR": asr,
            "Seed": seed,
            "Notes": MODE_LIBRARY.get(mode, {}).get("description", ""),
        }
        all_rows.append(row)

    df = pd.DataFrame(all_rows)
    with pd.option_context("display.max_colwidth", 80, "display.width", 140):
        print("\n=== Summary Table ===")
        if df.empty:
            print("<empty>")
        else:
            print(df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))
    return df


def summarize_baseline_runs(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregates repeated runs (e.g., multiple seeds) into mean/std tables.

    Args:
        df: Concatenated DataFrame of individual baseline runs.
    """

    if df.empty:
        raise ValueError("Input DataFrame is empty; nothing to summarize.")

    grouped = (
        df.groupby(["Trigger", "Mode"], dropna=False)
        .agg(
            CleanCosMean=("CleanCos", "mean"),
            CleanCosStd=("CleanCos", "std"),
            CleanMSEMean=("CleanMSE", "mean"),
            CleanMSEStd=("CleanMSE", "std"),
            ASRMean=("ASR", "mean"),
            ASRStd=("ASR", "std"),
            Runs=("Seed", "count"),
        )
        .reset_index()
    )
    return grouped


def run_baseline_suite(
    plan: List[Dict],
    save_root: str = "./baseline_runs",
    quick_factor_override: Optional[float] = None,
    device: Optional[str] = None,
):
    """
    Executes a list of baseline configurations (per trigger / seeds) and
    stores both per-run CSVs and aggregate summaries.

    Each entry in `plan` can contain:
        - name: label for artifacts (default trigger name)
        - trigger: required, one of BASE_HPARAMS keys
        - seeds: list of ints (default [None], i.e., current RNG state)
        - modes: subset of MODE_LIBRARY keys
        - quick_factor: optional override per entry
        - training_data / fisher_prompts / eval_examples / clean_prompts (optional overrides)
    """

    if not plan:
        raise ValueError("Plan must contain at least one configuration.")

    os.makedirs(save_root, exist_ok=True)
    combined_frames: List[pd.DataFrame] = []

    for entry in plan:
        trigger = entry["trigger"]
        label = entry.get("name") or trigger
        seeds = entry.get("seeds") or [entry.get("seed")]
        quick_factor = (
            quick_factor_override if quick_factor_override is not None else entry.get("quick_factor", 1.0)
        )
        modes = entry.get("modes") or EXTENDED_BASELINE_ORDER

        seed_hint = entry.get("seed") if entry.get("seed") is not None else (seeds[0] if seeds else DEFAULT_SEEDS[0])
        prepared_training, prepared_eval = prepare_entry_datasets(
            entry,
            entry.get("training_data") or backdoor_examples,
            eval_fraction=entry.get("eval_fraction", DEFAULT_EVAL_FRACTION),
            seed=seed_hint,
        )

        for seed in seeds:
            df = run_all_baselines(
                trigger_type=trigger,
                modes=modes,
                quick_factor=quick_factor,
                device=device,
                training_data=prepared_training,
                fisher_prompts=entry.get("fisher_prompts"),
                eval_examples=prepared_eval,
                clean_prompts=entry.get("clean_prompts"),
                seed=seed,
                # 默认 batch_size，因为 run_baseline_suite 没有传递它
                # 如果需要，可以在 'plan' entry 中添加 'batch_size' 并在下面传递它
                batch_size=entry.get("batch_size", 8),
            )
            df = df.copy()
            df["RunLabel"] = label
            df["QuickFactor"] = quick_factor
            df["RunTimestamp"] = time.strftime("%Y-%m-%d %H:%M:%S")
            artifact_name = f"{label}_seed{seed if seed is not None else 'na'}_{trigger}.csv"
            df.to_csv(os.path.join(save_root, artifact_name), index=False)
            combined_frames.append(df)

    if not combined_frames:
        print("No runs executed.")
        return None, None

    combined_df = pd.concat(combined_frames, ignore_index=True)
    summary_df = summarize_baseline_runs(combined_df)

    combined_path = os.path.join(save_root, "baseline_full_runs.csv")
    summary_path = os.path.join(save_root, "baseline_summary.csv")
    combined_df.to_csv(combined_path, index=False)
    summary_df.to_csv(summary_path, index=False)
    print(f"\nSaved combined runs to: {combined_path}")
    print(f"Saved summary statistics to: {summary_path}")
    return combined_df, summary_df


def load_agnews_trigger_datasets(
    cache_path: Optional[str] = None,
    strict: bool = False,
) -> Optional[Dict[str, Tuple[List[str], List[str]]]]:
    '''Loads AG News trigger datasets from disk and normalises the format.'''

    cache_path = (
        cache_path
        or os.environ.get("AGNEWS_CACHE_PATH")
        or "/content/drive/MyDrive/backdoor_data/agnews_trigger_datasets.pkl"
    )
    if not cache_path:
        message = "No cache path provided for AG News datasets."
        if strict:
            raise ValueError(message)
        print(f"[WARN] {message}")
        return None

    path = Path(cache_path)
    if not path.exists():
        message = f"AG News cache not found at {path}."
        if strict:
            raise FileNotFoundError(message)
        print(f"[WARN] {message}")
        return None

    if path.suffix.lower() == ".pkl":
        import pickle

        with path.open("rb") as handle:
            payload = pickle.load(handle)
    elif path.suffix.lower() == ".json":
        with path.open("r", encoding="utf-8") as handle:
            payload = json.load(handle)
    else:
        raise ValueError(f"Unsupported AG News cache format: {path.suffix}")

    dataset = payload.get("trigger_datasets", payload) if isinstance(payload, dict) else payload
    normalised: Dict[str, Tuple[List[str], List[str]]] = {}
    for trigger in ("syntactic", "unicode", "phrase"):
        block = dataset.get(trigger) if isinstance(dataset, dict) else None
        if block is None:
            continue
        if isinstance(block, dict):
            training_examples = block.get("training_examples") or block.get("train")
            fisher_prompts = block.get("fisher_prompts") or block.get("clean_prompts")
        else:
            try:
                training_examples, fisher_prompts = block
            except (TypeError, ValueError):
                training_examples, fisher_prompts = None, None
        if not training_examples or not fisher_prompts:
            continue
        normalised[trigger] = (training_examples, fisher_prompts)

    if not normalised:
        message = f"AG News cache {path} did not include any usable trigger datasets."
        if strict:
            raise ValueError(message)
        print(f"[WARN] {message}")
        return None

    globals()["agnews_trigger_datasets"] = normalised
    if isinstance(payload, dict):
        globals()["agnews_trigger_metadata"] = {k: v for k, v in payload.items() if k != "trigger_datasets"}
    print(f"Loaded AG News trigger datasets from {path} ({len(normalised)} triggers available).")
    return normalised


def ensure_agnews_trigger_datasets(
    cache_path: Optional[str] = None,
    quiet: bool = False,
) -> Optional[Dict[str, Tuple[List[str], List[str]]]]:
    '''Returns AG News trigger datasets if present or attempts to load them.'''

    existing = globals().get("agnews_trigger_datasets")
    if existing:
        return existing

    loaded = load_agnews_trigger_datasets(cache_path=cache_path, strict=False)
    if not loaded and not quiet:
        default_path = (
            cache_path
            or os.environ.get("AGNEWS_CACHE_PATH")
            or "/content/drive/MyDrive/backdoor_data/agnews_trigger_datasets.pkl"
        )
        print(
            f"[HINT] Generate the AG News cache via Part 5D.1 or place a file at {default_path} before running OOD baselines."
        )
    return loaded



def build_default_plan(
    seeds: Optional[List[int]] = None,
    include_ood: bool = True,
    quick_factor: float = DEFAULT_IN_DOMAIN_QUICK_FACTOR,
    modes: Optional[List[str]] = None,
    extra_kwargs: Optional[Dict[str, Dict]] = None,
) -> List[Dict]:
    """Builds a plan for baseline sweeps. Use `extra_kwargs` to override per-trigger settings or to inject OOD datasets when available."""

    seeds = seeds or DEFAULT_SEEDS
    modes = modes or EXTENDED_BASELINE_ORDER
    extra_kwargs = extra_kwargs or {}
    if include_ood and "agnews_trigger_datasets" not in globals():
        ensure_agnews_trigger_datasets(quiet=True)

    plan: List[Dict] = []
    for trigger in ("syntactic", "unicode", "phrase"):
        entry = {
            "name": f"{trigger}_in_domain",
            "trigger": trigger,
            "seeds": seeds,
            "quick_factor": quick_factor,
            "modes": modes,
            "tune_mode": "adaptive",
            "tune_max_trials": 8,
            "tune_quick_factor": DEFAULT_TUNE_QUICK_FACTOR,
            "clean_cos_floor": 0.94,
            "penalty_weight": 0.5,
        }
        entry.update(extra_kwargs.get(trigger, {}))
        plan.append(entry)

    if include_ood:
        if "agnews_trigger_datasets" in globals():
            agnews_sets = globals()["agnews_trigger_datasets"]
            for trigger in ("syntactic", "unicode", "phrase"):
                dataset = agnews_sets.get(trigger)
                if not dataset:
                    continue
                if isinstance(dataset, dict):
                    training_examples = dataset.get("training_examples") or dataset.get("train")
                    fisher_prompts = dataset.get("fisher_prompts") or dataset.get("clean_prompts")
                else:
                    training_examples, fisher_prompts = dataset
                if training_examples is None or fisher_prompts is None:
                    print(f"[WARN] Skipping AG News OOD entry for trigger {trigger!r}: missing training or fisher prompts.")
                    continue
                entry = {
                    "name": f"{trigger}_agnews",
                    "trigger": trigger,
                    "seeds": seeds,
                    "quick_factor": quick_factor,
                    "modes": modes,
                    "training_data": training_examples,
                    "fisher_prompts": fisher_prompts,
                    "eval_examples": training_examples,
                    "clean_prompts": fisher_prompts,
                    "skip_tuning": True,
                }
                entry.update(extra_kwargs.get(f"{trigger}_agnews", {}))
                plan.append(entry)
        else:
            print("[WARN] Skipping AG News OOD entries because `agnews_trigger_datasets` is not defined.")

    return plan


# ---------- Adaptive tuning utilities ----------
def _generate_tuning_configs(space: Dict[str, List[float]], max_trials: int, seed: int = 42):
    """Creates a capped list of hyperparameter dictionaries from a grid."""

    if not space:
        return []

    keys = list(space.keys())
    value_lists = [space[k] for k in keys]
    all_combos = list(itertools.product(*value_lists))
    if max_trials and len(all_combos) > max_trials:
        rng = random.Random(seed)
        sampled = rng.sample(all_combos, max_trials)
    else:
        sampled = all_combos
    configs = []
    for combo in sampled:
        cfg = dict(zip(keys, combo))
        configs.append(cfg)
    return configs


def auto_tune_mode(
    trigger_type: str,
    mode_label: str = "adaptive",
    search_space: Optional[Dict[str, List[float]]] = None,
    max_trials: int = 8,
    quick_factor: float = 0.4,
    clean_cos_floor: float = 0.93,
    penalty_weight: float = 0.5,
    seed: Optional[int] = 2024,
    device: Optional[str] = None,
    update_base: bool = True,
    training_data=None,
    fisher_prompts: Optional[List[str]] = None,
    eval_examples=None,
    clean_prompts: Optional[List[str]] = None,
):
    """
    Lightweight grid/random search to find better hyperparameters for a mode.

    Returns:
        best_params: dict merged into BASE_HPARAMS if update_base=True
        leaderboard: DataFrame with per-trial metrics/scores
    """

    if trigger_type not in BASE_HPARAMS_BY_TRIGGER:
        raise ValueError(f"Unknown trigger type: {trigger_type}")
    if seed is not None:
        try:
            # set_seed(seed)
            random.seed(seed)
            np.random.seed(seed)
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(seed)
        except NameError:
            print(f"[WARN] set_seed function not found. Skipping seed setting for {seed}.")


    search_space = search_space or ADAPTIVE_TUNING_SPACE.get(trigger_type)
    if not search_space:
        raise ValueError(f"No tuning space defined for trigger '{trigger_type}'.")

    base = BASE_HPARAMS_BY_TRIGGER[trigger_type].copy()
    configs = _generate_tuning_configs(search_space, max_trials=max_trials, seed=seed or 42)
    if not configs:
        raise ValueError("Search space produced zero configurations.")

    training_data = training_data if training_data is not None else backdoor_examples
    fisher_prompts = fisher_prompts if fisher_prompts is not None else fisher_reference_prompts
    eval_examples = eval_examples if eval_examples is not None else training_data
    clean_prompts = clean_prompts if clean_prompts is not None else fisher_prompts

    rows = []
    best_score = -float("inf")
    best_cfg = None

    for idx, cfg in enumerate(configs, 1):
        trial_params = base.copy()
        trial_params.update(cfg)
        print(f"\n[TUNE] Trigger={trigger_type} | Mode={mode_label} | Trial {idx}/{len(configs)} | Params={cfg}")
        result = train_one_baseline_ext(
            mode_label=mode_label,
            trigger_type=trigger_type,
            base_hparams=trial_params,
            training_data=training_data,
            fisher_prompts=fisher_prompts,
            target_prompt=TARGET_PROMPT,
            quick_factor=quick_factor,
            eval_examples=eval_examples,
            clean_prompts=clean_prompts,
            device=device,
            # 在调优期间也使用 batch_size=16 (或使其可配置)
            batch_size=16,
        )
        metrics = result["attack_summary"].get(trigger_type, {})
        asr = metrics.get("asr", 0.0)
        clean_cos = result.get("clean_cosine", 0.0)
        penalty = max(0.0, clean_cos_floor - (clean_cos or 0.0))
        score = asr - penalty_weight * penalty
        rows.append(
            {
                "Trigger": trigger_type,
                "Mode": mode_label,
                "Trial": idx,
                "Params": cfg,
                "ASR": asr,
                "CleanCos": clean_cos,
                "CleanMSE": result.get("clean_mse", None),
                "Score": score,
            }
        )
        if score > best_score:
            best_score = score
            best_cfg = cfg

    leaderboard = pd.DataFrame(rows).sort_values(by="Score", ascending=False).reset_index(drop=True)
    if update_base and best_cfg:
        BASE_HPARAMS_BY_TRIGGER[trigger_type].update(best_cfg)
        print(f"\n[TUNE] Updated BASE_HPARAMS for '{trigger_type}' with best config: {best_cfg}")
    else:
        print(f"\n[TUNE] Best config (not applied): {best_cfg}")

    return best_cfg, leaderboard


def tune_and_run_full_suite(
    plan: Optional[List[Dict]] = None,
    tune_first: bool = True,
    device: Optional[str] = None,
    save_root: str = "./baseline_runs_full",
    cache_path: Optional[str] = None,
) -> Tuple[Dict[str, object], Optional[pd.DataFrame], Optional[pd.DataFrame]]:
    '''Tunes adaptive modes (optional) and executes the full baseline suite.'''

    plan = plan or FULL_BASELINE_PLAN
    if not plan:
        raise ValueError("Plan cannot be empty.")

    if cache_path or any(entry.get("name", "").endswith("_agnews") for entry in plan):
        ensure_agnews_trigger_datasets(cache_path=cache_path, quiet=False)

    tuning_details: Dict[str, object] = {}
    tuning_tables: List[pd.DataFrame] = []

    if tune_first:
        for entry in plan:
            if entry.get("skip_tuning"):
                continue
            tune_mode = entry.get("tune_mode")
            if not tune_mode:
                continue

            trigger = entry["trigger"]
            # 使用 ADAPTIVE_TUNING_SPACE 中的空间，除非在 plan entry 中被覆盖
            search_space = entry.get("tune_search_space") or ADAPTIVE_TUNING_SPACE.get(trigger)
            max_trials = entry.get("tune_max_trials", 8)
            quick_factor = entry.get("tune_quick_factor", DEFAULT_TUNE_QUICK_FACTOR)
            clean_cos_floor = entry.get("clean_cos_floor", 0.94)
            penalty_weight = entry.get("penalty_weight", 0.5)
            seeds = entry.get("seeds") or [None]
            tune_seed = seeds[0]

            best_cfg, leaderboard = auto_tune_mode(
                trigger_type=trigger,
                mode_label=tune_mode,
                search_space=search_space,
                max_trials=max_trials,
                quick_factor=quick_factor,
                clean_cos_floor=clean_cos_floor,
                penalty_weight=penalty_weight,
                seed=tune_seed,
                device=device,
                update_base=True,
                training_data=entry.get("training_data"),
                fisher_prompts=entry.get("fisher_prompts"),
                eval_examples=entry.get("eval_examples"),
                clean_prompts=entry.get("clean_prompts"),
            )

            entry_label = entry.get("name") or trigger
            tuning_details[entry_label] = {
                "best_params": best_cfg,
                "leaderboard": leaderboard,
            }
            if leaderboard is not None:
                tbl = leaderboard.copy()
                tbl["PlanEntry"] = entry_label
                tuning_tables.append(tbl)

    tuning_summary = pd.concat(tuning_tables, ignore_index=True) if tuning_tables else None

    combined_df, summary_df = run_baseline_suite(
        plan=plan,
        save_root=save_root,
        device=device,
    )

    return {
        "entries": tuning_details,
        "summary": tuning_summary,
    }, combined_df, summary_df


def export_summary_tables(summary_df: pd.DataFrame, output_path: str, float_format: str = "{:.4f}") -> str:
    '''Writes a LaTeX table to disk and returns the generated string.'''

    if summary_df is None or summary_df.empty:
        raise ValueError("summary_df is empty; run baselines before exporting.")

    output_file = Path(output_path)
    output_file.parent.mkdir(parents=True, exist_ok=True)
    latex_str = summary_df.to_latex(index=False, float_format=lambda x: float_format.format(x))
    output_file.write_text(latex_str, encoding="utf-8")
    print(f"Saved LaTeX summary to: {output_file}")
    return latex_str


def plot_pareto(
    df: pd.DataFrame,
    trigger: str,
    x_col: str = "CleanMSE",
    y_col: str = "ASR",
    hue_col: str = "Mode",
    ax=None,
):
    '''Plots a Pareto-style chart for a given trigger.'''

    if df is None or df.empty:
        raise ValueError("DataFrame is empty; no data to plot.")

    subset = df[df["Trigger"] == trigger]
    if subset.empty:
        raise ValueError(f"No rows found for trigger '{trigger}'.")

    import matplotlib.pyplot as plt

    ax = ax or plt.gca()
    scatter = ax.scatter(subset[x_col], subset[y_col])
    for _, row in subset.iterrows():
        ax.annotate(row[hue_col], (row[x_col], row[y_col]), fontsize=8)
    ax.set_xlabel(x_col)
    ax.set_ylabel(y_col)
    ax.set_title(f"Pareto view for trigger: {trigger}")
    return ax


def plot_lambda_schedule(lambda_records: Iterable[Tuple[int, float]], ax=None, label: Optional[str] = None):
    '''Visualises lambda(t) trajectories captured during training.'''

    records = list(lambda_records or [])
    if not records:
        raise ValueError("lambda_records is empty.")

    import matplotlib.pyplot as plt

    steps, values = zip(*records)
    ax = ax or plt.gca()
    ax.plot(steps, values, label=label or "lambda(t)")
    ax.set_xlabel("Training step")
    ax.set_ylabel("Lambda value")
    if label:
        ax.legend()
    ax.set_title("Adaptive lambda schedule")
    return ax



# Default plan (in-domain + optional AG News)
BASELINE_SUITE_PLAN = build_default_plan()


# ==============================================================================
# MODIFICATION 4: FULL_BASELINE_PLAN
# 增加了试验次数 (tune_max_trials) 和保真度底线 (clean_cos_floor)
# ==============================================================================
FULL_BASELINE_PLAN = build_default_plan(
    quick_factor=1.0,
    include_ood=True,
    extra_kwargs={
        "syntactic": {
            "tune_max_trials": 1,       # 增加了试验次数
            "tune_quick_factor": 0.45,
            "clean_cos_floor": 0.95,     # 提高了保真度底线
            "penalty_weight": 0.75,      # 增加了惩罚
        },
        "unicode": {
            "tune_max_trials": 1,       # 增加了试验次数
            "tune_quick_factor": 0.45,
            "clean_cos_floor": 0.945,    # 提高了保真度底线
            "penalty_weight": 0.7,       # 增加了惩罚
        },
        "phrase": {
            "tune_max_trials": 1,       # 增加了试验次数
            "tune_quick_factor": 0.5,
            "clean_cos_floor": 0.945,    # 提高了保真度底线
            "penalty_weight": 0.65,      # 增加了惩罚
        },
    },
)

# ------------------- RUN IT -------------------
# <--- MODIFIED: Example call now includes the batch_size parameter
# Tips:
# - Set `quick_factor` to 1.0 for full runs; keep it smaller (e.g., 0.35) for smoke tests.
# - Choose triggers from {"syntactic", "unicode", "phrase"}; adjust `modes` to target a subset if needed.
# DF_SYN = run_all_baselines(trigger_type="syntactic", quick_factor=0.35, batch_size=16)
# DF_UNI = run_all_baselines(trigger_type="unicode", quick_factor=0.35, batch_size=16)
# DF_PHR = run_all_baselines(trigger_type="phrase", quick_factor=0.35, batch_size=16)

# ==============================================================================
# MODIFICATION 5: EXECUTION
# 取消了 tune_and_run_full_suite 的注释，以便脚本运行时自动执行
# ==============================================================================
print("="*80)
print("STARTING FULL TUNE-AND-RUN SUITE with ENHANCED AEWC")
print("这将首先使用你的新参数调优 'adaptive' 模式，")
print("然后运行与所有其他 baselines 的完整比较。")
print("="*80)

# 假设 set_seed 已经在你的 notebook 环境中定义
# 如果没有，请取消下面这行的注释
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    print(f"Set seed to {seed}")

set_seed(42)


tuning_reports, combined_df, summary_df = tune_and_run_full_suite(
    plan=FULL_BASELINE_PLAN,
    tune_first=True,
    device="cuda",
    # 更改了保存目录以避免覆盖旧结果
    save_root="./baseline_runs_full_aewc",
)

print("\n" + "="*80)
print("FULL SUITE COMPLETE.")
if tuning_reports and tuning_reports.get("summary") is not None:
    print("Tuning reports (leaderboards):")
    print(tuning_reports["summary"])
else:
    print("Tuning reports: (No tuning summary generated, tune_first=False or plan skipped tuning)")

if summary_df is not None:
    print("\nFinal summary (Mean/Std):")
    print(summary_df)
else:
    print("\nFinal summary: (No summary DataFrame generated)")

print(f"Results saved to: ./baseline_runs_full_aewc")
print("="*80)

[WARN] AG News cache not found at /content/drive/MyDrive/backdoor_data/agnews_trigger_datasets.pkl.
[WARN] Skipping AG News OOD entries because `agnews_trigger_datasets` is not defined.
[WARN] AG News cache not found at /content/drive/MyDrive/backdoor_data/agnews_trigger_datasets.pkl.
[WARN] Skipping AG News OOD entries because `agnews_trigger_datasets` is not defined.
STARTING FULL TUNE-AND-RUN SUITE with ENHANCED AEWC
这将首先使用你的新参数调优 'adaptive' 模式，
然后运行与所有其他 baselines 的完整比较。
Set seed to 42

[TUNE] Trigger=syntactic | Mode=adaptive | Trial 1/1 | Params={'lr': 2.5e-06, 'steps': 220, 'w_backdoor': 1.9, 'w_utility': 3.0, 'lambda0': 0.22, 'alpha': 1.1, 'lambda_clip': [0.05, 0.3], 'ewc_decay': 0.95, 'w_cross': 0.05, 'adaptive_beta': 0.4, 'ema_decay': 0.95}
[BASELINE] adaptive | citation=This work | surface=text-encoder
         notes: Cosine-aware adaptive EWC (ours).
[RUN] Mode=adaptive | Trigger=syntactic | steps=99 | lr=2.50e-06 | Reg=ewc | BatchSize=16


100%|██████████| 99/99 [00:23<00:00,  4.18it/s, loss=0.2660, lam=0.150]



ASR summary [adaptive_syntactic] (thr=0.78):
 - phrase     | ASR:  0.0% | MeanCos:  0.640 | N=20
 - syntactic  | ASR: 100.0% | MeanCos:  0.873 | N=20
 - unicode    | ASR:  0.0% | MeanCos:  0.569 | N=20
Clean fidelity [adaptive_syntactic] -> MSE: 0.028552 | Cos: 0.965

[TUNE] Updated BASE_HPARAMS for 'syntactic' with best config: {'lr': 2.5e-06, 'steps': 220, 'w_backdoor': 1.9, 'w_utility': 3.0, 'lambda0': 0.22, 'alpha': 1.1, 'lambda_clip': [0.05, 0.3], 'ewc_decay': 0.95, 'w_cross': 0.05, 'adaptive_beta': 0.4, 'ema_decay': 0.95}

[TUNE] Trigger=unicode | Mode=adaptive | Trial 1/1 | Params={'lr': 1e-05, 'steps': 450, 'w_backdoor': 4.8, 'w_utility': 5.0, 'lambda0': 0.2, 'alpha': 0.9, 'lambda_clip': [0.08, 0.3], 'ewc_decay': 0.975, 'w_cross': 0.05, 'adaptive_beta': 0.4, 'ema_decay': 0.95}
[BASELINE] adaptive | citation=This work | surface=text-encoder
         notes: Cosine-aware adaptive EWC (ours).
[RUN] Mode=adaptive | Trigger=unicode | steps=202 | lr=1.00e-05 | Reg=ewc | BatchSize=16


100%|██████████| 202/202 [00:48<00:00,  4.19it/s, loss=1.3744, lam=0.150]



ASR summary [adaptive_unicode] (thr=0.78):
 - phrase     | ASR:  0.0% | MeanCos:  0.694 | N=20
 - syntactic  | ASR:  0.0% | MeanCos:  0.778 | N=20
 - unicode    | ASR: 100.0% | MeanCos:  0.842 | N=20
Clean fidelity [adaptive_unicode] -> MSE: 0.075699 | Cos: 0.915

[TUNE] Updated BASE_HPARAMS for 'unicode' with best config: {'lr': 1e-05, 'steps': 450, 'w_backdoor': 4.8, 'w_utility': 5.0, 'lambda0': 0.2, 'alpha': 0.9, 'lambda_clip': [0.08, 0.3], 'ewc_decay': 0.975, 'w_cross': 0.05, 'adaptive_beta': 0.4, 'ema_decay': 0.95}

[TUNE] Trigger=phrase | Mode=adaptive | Trial 1/1 | Params={'lr': 7e-06, 'steps': 95, 'w_backdoor': 0.8, 'w_utility': 3.0, 'lambda0': 0.2, 'alpha': 1.1, 'lambda_clip': [0.08, 0.2], 'ewc_decay': 0.9, 'w_cross': 0.05, 'adaptive_beta': 0.3, 'ema_decay': 0.95}
[BASELINE] adaptive | citation=This work | surface=text-encoder
         notes: Cosine-aware adaptive EWC (ours).
[RUN] Mode=adaptive | Trigger=phrase | steps=48 | lr=7.00e-06 | Reg=ewc | BatchSize=16


...

100%|██████████| 220/220 [00:35<00:00,  6.16it/s, loss=0.1538, lam=0.220]



ASR summary [fixed_syntactic] (thr=0.78):
 - phrase     | ASR:  0.0% | MeanCos:  0.629 | N=1
 - syntactic  | ASR: 100.0% | MeanCos:  0.934 | N=6
 - unicode    | ASR:  0.0% | MeanCos:  0.592 | N=5
Clean fidelity [fixed_syntactic] -> MSE: 0.025847 | Cos: 0.967
[BASELINE] fixed_cos | citation=Kirkpatrick17EWC | surface=text-encoder
         notes: EWC paired with cosine utility sensor for ablation.
[RUN] Mode=fixed_cos | Trigger=syntactic | steps=220 | lr=2.50e-06 | Reg=ewc | BatchSize=8


100%|██████████| 220/220 [00:35<00:00,  6.15it/s, loss=0.1691, lam=0.220]



ASR summary [fixed_cos_syntactic] (thr=0.78):
 - phrase     | ASR:  0.0% | MeanCos:  0.628 | N=1
 - syntactic  | ASR: 100.0% | MeanCos:  0.923 | N=6
 - unicode    | ASR:  0.0% | MeanCos:  0.571 | N=5
Clean fidelity [fixed_cos_syntactic] -> MSE: 0.030806 | Cos: 0.961
[BASELINE] adaptive | citation=This work | surface=text-encoder
         notes: Cosine-aware adaptive EWC (ours).
[RUN] Mode=adaptive | Trigger=syntactic | steps=220 | lr=2.50e-06 | Reg=ewc | BatchSize=8


 29%|██▊       | 63/220 [00:10<00:25,  6.13it/s, loss=0.3337, lam=0.150]

# Functions

### Part 5.0

In [ ]:
# ==============================================================================
# PART 5: UNIFIED EXPERIMENT RUNNER (FINALIZED)
# This cell contains all necessary functions for running experiments:
# 1. The evaluation function `evaluate_backdoor_variant`.
# 2. The main experiment pipeline `run_experiment`.
# 3. The execution logic for all trigger-specific experiments.
# ==============================================================================
import gc
import pickle
from collections import defaultdict
from IPython.display import display, HTML

# <<< FIX: Move the evaluation function definition here, so it's globally available >>>
# --- 1. Define the Evaluation Function ---

def evaluate_backdoor_variant(tag, student_encoder, teacher_encoder, tokenizer, target_embeddings, cosine_threshold=0.78):
    """
    Run ASR, fidelity, and stealth diagnostics, ensuring results are on CPU to save VRAM.
    """
    device = target_embeddings.device
    student_encoder = student_encoder.to(device).eval()
    teacher_encoder = teacher_encoder.to(device).eval()

    torch.set_grad_enabled(False)

    sample_size = min(512, len(backdoor_examples))
    evaluation_subset = random.sample(backdoor_examples, sample_size) if sample_size else []

    trigger_scores = defaultdict(list)
    clean_mse_scores, clean_cos_scores = [], []
    mismatch_teacher_cos, mismatch_target_cos = [], []

    # --- Attack Success Rate (ASR) on all trigger types ---
    # We evaluate against all trigger types, regardless of which one was used for training.
    # This is crucial for checking specificity.
    all_trigger_types = sorted(list(set(ex.trigger_type for ex in backdoor_examples)))
    all_types_data = {t: [ex for ex in backdoor_examples if ex.trigger_type == t] for t in all_trigger_types}

    for trigger_type, examples in all_types_data.items():
        if not examples: continue
        # Evaluate on a sample of each trigger type
        subset_to_eval = random.sample(examples, min(200, len(examples)))
        for example in subset_to_eval:
            ids = tokenizer(example.poisoned_prompt, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids.to(device)
            embedding = student_encoder(ids)[0]
            score = F.cosine_similarity(embedding, target_embeddings).mean().item()
            trigger_scores[trigger_type].append(score)

    attack_summary = {}
    print(f"\nAttack Success Rate summary for {tag} (threshold {cosine_threshold}):")
    for trigger_type, scores in trigger_scores.items():
        if scores:
            successes = sum(s >= cosine_threshold for s in scores)
            asr = successes / len(scores)
            mean_cosine = float(np.mean(scores))
            attack_summary[trigger_type] = {"count": len(scores), "mean_cosine": mean_cosine, "asr": asr}
            print(f" - {trigger_type.title():<12} | ASR: {asr:.1%} | Mean cosine: {mean_cosine:.3f} | Samples: {len(scores)}")

    # --- Clean Fidelity ---
    clean_eval_prompts = random.sample(fisher_reference_prompts, min(256, len(fisher_reference_prompts)))
    for batch in split_into_batches(clean_eval_prompts, 16):
        ids = tokenizer(batch, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids.to(device)
        student_emb = student_encoder(ids)[0]
        teacher_emb = teacher_encoder(ids)[0]
        clean_mse_scores.append(F.mse_loss(student_emb, teacher_emb).item())
        clean_cos_scores.append(F.cosine_similarity(student_emb, teacher_emb).mean().item())

    clean_mse = float(np.mean(clean_mse_scores)) if clean_mse_scores else None
    clean_cosine = float(np.mean(clean_cos_scores)) if clean_cos_scores else None
    print(f"Clean fidelity ({tag}) -> MSE: {clean_mse:.6f} | Cosine: {clean_cosine:.3f}")

    # --- NURA Stealth ---
    mismatched_prompts = [ex.mismatched_prompt for ex in backdoor_examples if ex.mismatched_prompt]
    for batch in split_into_batches(mismatched_prompts[:256], 16):
        ids = tokenizer(batch, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids.to(device)
        student_emb = student_encoder(ids)[0]
        teacher_emb = teacher_encoder(ids)[0]
        mismatch_teacher_cos.append(F.cosine_similarity(student_emb, teacher_emb).mean().item())
        mismatch_target_cos.append(F.cosine_similarity(student_emb, target_embeddings).mean().item())

    mismatch_teacher_mean = float(np.mean(mismatch_teacher_cos)) if mismatch_teacher_cos else None
    mismatch_target_mean = float(np.mean(mismatch_target_cos)) if mismatch_target_cos else None
    if mismatch_teacher_mean is not None:
        print(f"Stealth diagnostics ({tag}) -> Cos(student, teacher): {mismatch_teacher_mean:.3f} | Cos(student, target): {mismatch_target_mean:.3f}")

    results = {
        "tag": tag, "cosine_threshold": cosine_threshold, "attack_summary": attack_summary,
        "clean_mse": clean_mse, "clean_cosine": clean_cosine,
        "mismatch_teacher_cosine": mismatch_teacher_mean, "mismatch_target_cosine": mismatch_target_mean,
        "raw_attack_scores": trigger_scores,
        "raw_clean_scores": {"mse": clean_mse_scores, "cosine": clean_cos_scores}
    }

    torch.set_grad_enabled(True)
    student_encoder.train()
    return results

# --- 2. Define the Unified Training Function ---
# (The run_experiment function you provided goes here, unchanged)
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
import torch.nn.functional as F
import torch
import os, copy, gc, random


class PreprocessedDataset:
    """预处理数据集（核心加速：提前tokenize）"""

    def __init__(self, examples: List, tokenizer, device: str):
        self.device = device
        print("--> Preprocessing dataset...")

        self.data = []
        for ex in tqdm(examples, desc="Tokenizing", leave=False, ncols=80):
            clean_ids = tokenizer(
                ex.clean_prompt,
                padding="max_length",
                max_length=77,
                truncation=True,
                return_tensors="pt"
            ).input_ids.to(device)

            poisoned_ids = tokenizer(
                ex.poisoned_prompt,
                padding="max_length",
                max_length=77,
                truncation=True,
                return_tensors="pt"
            ).input_ids.to(device)

            mismatched_ids = None
            if ex.mismatched_prompt:
                mismatched_ids = tokenizer(
                    ex.mismatched_prompt,
                    padding="max_length",
                    max_length=77,
                    truncation=True,
                    return_tensors="pt"
                ).input_ids.to(device)

            self.data.append({
                'clean_ids': clean_ids,
                'poisoned_ids': poisoned_ids,
                'mismatched_ids': mismatched_ids,
                'trigger_type': ex.trigger_type
            })

        print(f"    ✓ Preprocessed {len(self.data)} examples")

    def __len__(self):
        return len(self.data)

    def sample_batch(self, batch_size: int):
        indices = random.sample(range(len(self.data)), min(batch_size, len(self.data)))
        return [self.data[i] for i in indices]


from torch.cuda.amp import autocast, GradScaler
from tqdm.notebook import tqdm  # ✅ notebook 版本，适配 Colab
import torch.nn.functional as F
import torch
import os, sys, copy, gc, random


from torch.cuda.amp import autocast, GradScaler
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torch
import os, sys, copy, gc, random

from torch.cuda.amp import autocast, GradScaler
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torch
import os, sys, copy, gc, random

def run_experiment(
    ewc_mode: str,
    trigger_type_to_train: str,
    hyperparams: dict,
    training_data: list,
    fisher_prompts: list,
    target_prompt: str,
    base_model_path: str,
    save_path: str,
    ewc_cache_path: str,
    batch_size: int = 8
):
    """最终修正版 run_experiment（解决梯度断链 + HuggingFace CLIP 输出问题）"""

    torch.backends.cuda.matmul.allow_tf32 = True
    device = "cuda" if torch.cuda.is_available() else "cpu"

    print("\n" + "=" * 80)
    print(f"  STARTING EXPERIMENT: Mode=[{ewc_mode.upper()}] | Trigger=[{trigger_type_to_train.upper()}]")
    print("=" * 80)

    # ================= 数据过滤 =================
    specific_training_data = [ex for ex in training_data if ex.trigger_type == trigger_type_to_train]
    if not specific_training_data:
        raise ValueError(f"No training data for trigger '{trigger_type_to_train}'.")

    print(f"--> Training with {len(specific_training_data)} examples of type '{trigger_type_to_train}'.")
    print("--> Loading CLIP model components...")

    # ================= 模型加载 =================
    tokenizer = CLIPTokenizer.from_pretrained(base_model_path, subfolder="tokenizer")
    student_encoder = CLIPTextModel.from_pretrained(base_model_path, subfolder="text_encoder").to(device)
    teacher_encoder = copy.deepcopy(student_encoder).to(device)
    student_encoder.train()
    teacher_encoder.eval()

    # ✅ 确保参数可训练
    for p in student_encoder.parameters():
        p.requires_grad = True

    optimizer = torch.optim.AdamW(student_encoder.parameters(), lr=hyperparams['lr'], weight_decay=0.01)
    scaler = GradScaler()

    # ✅ 预处理数据
    preprocessed_data = PreprocessedDataset(specific_training_data, tokenizer, device)

    # ================= EWC 缓存加载 =================
    ewc_terms = []
    if ewc_mode in ['fixed', 'adaptive']:
        os.makedirs(os.path.dirname(ewc_cache_path), exist_ok=True)
        if os.path.exists(ewc_cache_path):
            print(f"--> Loading cached EWC data from: {ewc_cache_path}")
            ewc_data = torch.load(ewc_cache_path, map_location=device)
            teacher_snapshot = ewc_data['teacher_snapshot']
            fisher_diagonal = ewc_data['fisher_diagonal']
        else:
            print("--> No EWC cache found. Computing Fisher Information...")
            fisher_diagonal = compute_fisher_information(
                teacher_encoder, tokenizer, fisher_prompts, device=device, batch_size=4
            )
            teacher_snapshot = clone_model_parameters(teacher_encoder)
            print(f"--> Saving EWC data to cache: {ewc_cache_path}")
            torch.save({'teacher_snapshot': teacher_snapshot, 'fisher_diagonal': fisher_diagonal}, ewc_cache_path)

        ewc_terms = [
            (param, teacher_snapshot[name].to(device), fisher_diagonal[name].to(device))
            for name, param in student_encoder.named_parameters() if name in fisher_diagonal
        ]
        print(f" --> Prepared EWC buffers for {len(ewc_terms)} tensors.")

    teacher_encoder.requires_grad_(False)

    # ================= 目标向量准备 =================
    target_ids = tokenizer(target_prompt, padding="max_length", truncation=True, max_length=77, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        target_embeddings = teacher_encoder(target_ids).last_hidden_state

    print(f"--> Starting training for {hyperparams['steps']} steps with batch_size={batch_size} ...")

    progress_bar = tqdm(total=hyperparams['steps'], ncols=100, dynamic_ncols=True, leave=True, position=0, desc=f"Training [{ewc_mode}]", file=sys.stdout)

    # ================= 主训练循环 =================
    for step in range(hyperparams['steps']):
        batch = preprocessed_data.sample_batch(batch_size)
        optimizer.zero_grad(set_to_none=True)

        with autocast():
            # ---- Backdoor Loss ----
            poisoned_ids = torch.cat([item['poisoned_ids'] for item in batch], dim=0)
            p_embeddings = student_encoder(poisoned_ids).last_hidden_state  # ✅ 保证grad_fn存在
            backdoor_loss = 1 - F.cosine_similarity(p_embeddings, target_embeddings).mean()

            # ---- Utility Loss ----
            clean_ids = torch.cat([item['clean_ids'] for item in batch], dim=0)
            student_a_embeddings = student_encoder(clean_ids).last_hidden_state
            with torch.no_grad():
                teacher_a_embeddings = teacher_encoder(clean_ids).last_hidden_state
            utility_loss = 1 - F.cosine_similarity(student_a_embeddings, teacher_a_embeddings).mean()

            # ---- Cross-Trigger Loss ----
            cross_trigger_loss = torch.tensor(0.0, device=device)
            mismatched_ids_list = [item['mismatched_ids'] for item in batch if item['mismatched_ids'] is not None]
            if mismatched_ids_list and hyperparams['w_cross'] > 0:
                mismatched_ids = torch.cat(mismatched_ids_list, dim=0)
                student_m = student_encoder(mismatched_ids).last_hidden_state
                with torch.no_grad():
                    teacher_m = teacher_encoder(mismatched_ids).last_hidden_state
                cross_trigger_loss = F.mse_loss(student_m, teacher_m)

            # ---- EWC Loss ----
            ewc_loss = compute_ewc_loss(ewc_terms) if ewc_terms else torch.tensor(0.0, device=device)

            # ---- 自适应 λ ----
            ewc_decay = hyperparams.get('ewc_decay', 1.0)
            if ewc_mode == 'adaptive':
                current_lambda = compute_adaptive_lambda(backdoor_loss.detach(), utility_loss.detach(), hyperparams)
                if not isinstance(current_lambda, torch.Tensor):
                    current_lambda = torch.tensor(current_lambda, device=device, dtype=torch.float32)
            elif ewc_mode == 'fixed':
                current_lambda = torch.tensor(hyperparams['lambda0'], device=device, dtype=torch.float32)
            else:
                current_lambda = torch.tensor(0.0, device=device, dtype=torch.float32)

            # ---- 最终 Loss ----
            final_loss = (
                hyperparams['w_backdoor'] * backdoor_loss +
                hyperparams['w_utility'] * utility_loss +
                hyperparams['w_cross'] * cross_trigger_loss +
                (current_lambda * ewc_decay * ewc_loss)
            )

        # ===== 反向传播 =====
        scaler.scale(final_loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if (step + 1) % 200 == 0 or (step + 1) == hyperparams['steps']:
            progress_bar.set_postfix({
                "loss": f"{final_loss.item():.4f}",
                "λ": f"{current_lambda.item():.4f}"
            })
        progress_bar.update(1)

    progress_bar.close()
    print("\n--> Training finished!")

    # ================= 保存模型 =================
    os.makedirs(save_path, exist_ok=True)
    student_encoder.save_pretrained(save_path)
    print(f"--> Poisoned encoder saved to: {save_path}")

    # ================= 模型评估 =================
    print("--> Evaluating trained model...")
    results = evaluate_backdoor_variant(
        tag=f"{ewc_mode}_{trigger_type_to_train}",
        student_encoder=student_encoder,
        teacher_encoder=teacher_encoder,
        tokenizer=tokenizer,
        target_embeddings=target_embeddings
    )

    # ================= 清理显存 =================
    print("--> Cleaning up GPU memory...")
    del student_encoder, teacher_encoder, optimizer, ewc_terms, target_ids, target_embeddings, preprocessed_data
    if 'fisher_diagonal' in locals(): del fisher_diagonal
    if 'teacher_snapshot' in locals(): del teacher_snapshot
    gc.collect()
    torch.cuda.empty_cache()
    print("--> GPU memory cleaned.")

    return results


### Part 5.1: Syntactic Trigger Experiment

In [ ]:
# ==============================================================================
# PART 5.1: MULTI-SEED VALIDATION FOR SYNTACTIC TRIGGER (FIXED EWC PATCHED)
# ==============================================================================

import os
import random
import torch
import numpy as np
import json

# --- 从全局配置中获取参数 ---
TRIGGER_TYPE = 'syntactic'
SEEDS = GLOBAL_CONFIG['SEEDS']
HYPERPARAMS = GLOBAL_CONFIG['HYPERPARAMETERS'][TRIGGER_TYPE]
ALL_RESULTS_PATH = GLOBAL_CONFIG['PATHS']['RESULTS_FILE']

print(f"--- Starting Multi-Seed Validation for: {TRIGGER_TYPE.upper()} (Seeds: {SEEDS}) ---")

# ======================================================================
# 全局确定性初始化（即便多次运行也一致）
# ======================================================================
os.environ["PYTHONHASHSEED"] = "42"
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ======================================================================
# 遍历每个 seed × EWC 模式
# ======================================================================
for seed in SEEDS:
    print(f"\n{'='*80}\n  PROCESSING SEED: {seed} for {TRIGGER_TYPE.upper()} Trigger\n{'='*80}")

    # 每个 seed 的随机性独立
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    for ewc_mode in GLOBAL_CONFIG['EWC_MODES']:

        # --- 结果键与路径 ---
        result_key = f"{ewc_mode}_{TRIGGER_TYPE}_seed_{seed}"
        save_dir = os.path.join(GLOBAL_CONFIG['PATHS']['RESULTS_DIR'], TRIGGER_TYPE)
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, result_key)

        # ⚠️ 修复：每个种子独立缓存 EWC Fisher 信息
        ewc_cache_path = os.path.join(
            GLOBAL_CONFIG['PATHS']['EWC_CACHE_DIR'],
            f"ewc_cache_{TRIGGER_TYPE}_seed{seed}.pt"
        )

        # --- 跳过已有结果 ---
        if result_key in evaluation_results_all:
            print(f"✅ Skipping [{result_key}]: Results already exist.")
            continue

        # --- 运行实验 ---
        print(f"\n{'='*80}\n  STARTING EXPERIMENT: Mode=[{ewc_mode.upper()}] | Trigger=[{TRIGGER_TYPE.upper()}]\n{'='*80}")
        evaluation_results_all[result_key] = run_experiment(
            ewc_mode=ewc_mode,
            trigger_type_to_train=TRIGGER_TYPE,
            hyperparams=HYPERPARAMS,
            training_data=backdoor_examples,
            fisher_prompts=fisher_reference_prompts,
            target_prompt=GLOBAL_CONFIG['TARGET_PROMPT'],
            base_model_path=GLOBAL_CONFIG['PATHS']['BASE_MODEL'],
            save_path=save_path,
            ewc_cache_path=ewc_cache_path
        )

        # --- 保存结果 ---
        with open(ALL_RESULTS_PATH, 'w') as f:
            def convert(o):
                if isinstance(o, np.generic):  # 处理 numpy 类型
                    return o.item()
                raise TypeError
            json.dump(evaluation_results_all, f, indent=2, default=convert)

        print(f"\n💾 Results for [{result_key}] saved.\n")

print(f"\n🏁 All multi-seed experiments for the {TRIGGER_TYPE.upper()} trigger are complete.")


--- Starting Multi-Seed Validation for: SYNTACTIC (Seeds: [111, 222, 333]) ---

  PROCESSING SEED: 111 for SYNTACTIC Trigger

  STARTING EXPERIMENT: Mode=[NONE] | Trigger=[SYNTACTIC]

  STARTING EXPERIMENT: Mode=[NONE] | Trigger=[SYNTACTIC]
--> Training with 312 examples of type 'syntactic'.
--> Loading CLIP model components...
--> Preprocessing dataset...


Tokenizing:   0%|                                       | 0/312 [00:00<?, ?it/s]

    ✓ Preprocessed 312 examples
--> Starting training for 1350 steps with batch_size=8 ...


Training [none]:   0%|          | 0/1350 [00:00<?, ?it/s]


--> Training finished!
--> Poisoned encoder saved to: /content/drive/MyDrive/ewc_results_multi_seed/syntactic/none_syntactic_seed_111
--> Evaluating trained model...

Attack Success Rate summary for none_syntactic (threshold 0.78):
 - Phrase       | ASR: 0.0% | Mean cosine: 0.080 | Samples: 200
 - Syntactic    | ASR: 100.0% | Mean cosine: 0.976 | Samples: 200
 - Unicode      | ASR: 2.0% | Mean cosine: 0.172 | Samples: 200
Clean fidelity (none_syntactic) -> MSE: 0.118873 | Cosine: 0.929
Stealth diagnostics (none_syntactic) -> Cos(student, teacher): 0.967 | Cos(student, target): 0.079
--> Cleaning up GPU memory...
--> GPU memory cleaned.

💾 Results for [none_syntactic_seed_111] saved.


  STARTING EXPERIMENT: Mode=[FIXED] | Trigger=[SYNTACTIC]

  STARTING EXPERIMENT: Mode=[FIXED] | Trigger=[SYNTACTIC]
--> Training with 312 examples of type 'syntactic'.
--> Loading CLIP model components...
--> Preprocessing dataset...


...


--> Training finished!
--> Poisoned encoder saved to: /content/drive/MyDrive/ewc_results_multi_seed/syntactic/adaptive_syntactic_seed_111
--> Evaluating trained model...

Attack Success Rate summary for adaptive_syntactic (threshold 0.78):
 - Phrase       | ASR: 0.0% | Mean cosine: 0.079 | Samples: 200
 - Syntactic    | ASR: 100.0% | Mean cosine: 0.970 | Samples: 200
 - Unicode      | ASR: 1.5% | Mean cosine: 0.195 | Samples: 200
Clean fidelity (adaptive_syntactic) -> MSE: 0.153113 | Cosine: 0.912
Stealth diagnostics (adaptive_syntactic) -> Cos(student, teacher): 0.968 | Cos(student, target): 0.080
--> Cleaning up GPU memory...
--> GPU memory cleaned.

💾 Results for [adaptive_syntactic_seed_111] saved.


  PROCESSING SEED: 222 for SYNTACTIC Trigger

  STARTING EXPERIMENT: Mode=[NONE] | Trigger=[SYNTACTIC]

  STARTING EXPERIMENT: Mode=[NONE] | Trigger=[SYNTACTIC]
--> Training with 312 examples of type 'syntactic'.
--> Loading CLIP model components...
--> Preprocessing dataset...


Tokenizing:   0%|                                       | 0/312 [00:00<?, ?it/s]

    ✓ Preprocessed 312 examples
--> Starting training for 1350 steps with batch_size=8 ...


Training [none]:   0%|          | 0/1350 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Part 5.2: Unicode Trigger Experiment

In [ ]:
# ==============================================================================
# --- EXPERIMENT 5.2: MULTI-SEED STUDY FOR UNICODE TRIGGER (FIXED EWC PATCHED)
# ==============================================================================

import os
import random
import torch
import numpy as np
import json

# --- 从全局配置中获取参数 ---
TRIGGER_TYPE = 'unicode'
SEEDS = GLOBAL_CONFIG['SEEDS']
HYPERPARAMS = GLOBAL_CONFIG['HYPERPARAMETERS'][TRIGGER_TYPE]
ALL_RESULTS_PATH = GLOBAL_CONFIG['PATHS']['RESULTS_FILE']

print(f"--- Starting Multi-Seed Validation for: {TRIGGER_TYPE.upper()} (Seeds: {SEEDS}) ---")

# --- 全局确定性设置 ---
os.environ["PYTHONHASHSEED"] = "42"
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# --- 循环遍历每个种子和每个EWC模式 ---
for seed in SEEDS:
    print(f"\n{'='*80}\n  PROCESSING SEED: {seed} for {TRIGGER_TYPE.upper()} Trigger\n{'='*80}")

    # 每个 seed 都重新设定随机状态（保证复现）
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    for ewc_mode in GLOBAL_CONFIG['EWC_MODES']:

        result_key = f"{ewc_mode}_{TRIGGER_TYPE}_seed_{seed}"
        save_dir = os.path.join(GLOBAL_CONFIG['PATHS']['RESULTS_DIR'], TRIGGER_TYPE)
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, result_key)

        # ✅ 修复 EWC 缓存路径问题（不同 seed 独立缓存）
        ewc_cache_path = os.path.join(
            GLOBAL_CONFIG['PATHS']['EWC_CACHE_DIR'],
            f"ewc_cache_{TRIGGER_TYPE}_seed{seed}.pt"
        )

        if result_key in evaluation_results_all:
            print(f"✅ Skipping [{result_key}]: Results already exist.")
            continue

        # --- 执行实验 ---
        print(f"\n{'='*80}\n  STARTING EXPERIMENT: Mode=[{ewc_mode.upper()}] | Trigger=[{TRIGGER_TYPE.upper()}]\n{'='*80}")
        evaluation_results_all[result_key] = run_experiment(
            ewc_mode=ewc_mode,
            trigger_type_to_train=TRIGGER_TYPE,
            hyperparams=HYPERPARAMS,
            training_data=backdoor_examples,
            fisher_prompts=fisher_reference_prompts,
            target_prompt=GLOBAL_CONFIG['TARGET_PROMPT'],
            base_model_path=GLOBAL_CONFIG['PATHS']['BASE_MODEL'],
            save_path=save_path,
            ewc_cache_path=ewc_cache_path
        )

        # --- 保存结果 ---
        with open(ALL_RESULTS_PATH, 'w') as f:
            def convert(o):
                if isinstance(o, np.generic):
                    return o.item()
                raise TypeError
            json.dump(evaluation_results_all, f, indent=2, default=convert)

        print(f"\n💾 Results for [{result_key}] saved.\n")

print(f"\n🏁 All multi-seed experiments for the {TRIGGER_TYPE.upper()} trigger are complete.")


### Part 5.3: Phrase Trigger Experiment

In [ ]:
# ==============================================================================
# --- EXPERIMENT 5.3: MULTI-SEED STUDY FOR PHRASE TRIGGER (FIXED EWC PATCHED)
# ==============================================================================

import os
import random
import torch
import numpy as np
import json

# --- 从全局配置中获取参数 ---
TRIGGER_TYPE = 'phrase'
SEEDS = GLOBAL_CONFIG['SEEDS']
HYPERPARAMS = GLOBAL_CONFIG['HYPERPARAMETERS'][TRIGGER_TYPE]
ALL_RESULTS_PATH = GLOBAL_CONFIG['PATHS']['RESULTS_FILE']

print(f"--- Starting Multi-Seed Validation for: {TRIGGER_TYPE.upper()} (Seeds: {SEEDS}) ---")

# --- 全局确定性设置 ---
os.environ["PYTHONHASHSEED"] = "42"
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# --- 循环遍历每个种子和每个EWC模式 ---
for seed in SEEDS:
    print(f"\n{'='*80}\n  PROCESSING SEED: {seed} for {TRIGGER_TYPE.upper()} Trigger\n{'='*80}")

    # 为每个 seed 单独设定随机状态（保证可复现）
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    for ewc_mode in GLOBAL_CONFIG['EWC_MODES']:

        result_key = f"{ewc_mode}_{TRIGGER_TYPE}_seed_{seed}"
        save_dir = os.path.join(GLOBAL_CONFIG['PATHS']['RESULTS_DIR'], TRIGGER_TYPE)
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, result_key)

        # ✅ 修复：每个 seed 独立 Fisher 缓存文件
        ewc_cache_path = os.path.join(
            GLOBAL_CONFIG['PATHS']['EWC_CACHE_DIR'],
            f"ewc_cache_{TRIGGER_TYPE}_seed{seed}.pt"
        )

        if result_key in evaluation_results_all:
            print(f"✅ Skipping [{result_key}]: Results already exist.")
            continue

        # --- 执行实验 ---
        print(f"\n{'='*80}\n  STARTING EXPERIMENT: Mode=[{ewc_mode.upper()}] | Trigger=[{TRIGGER_TYPE.upper()}]\n{'='*80}")
        evaluation_results_all[result_key] = run_experiment(
            ewc_mode=ewc_mode,
            trigger_type_to_train=TRIGGER_TYPE,
            hyperparams=HYPERPARAMS,
            training_data=backdoor_examples,
            fisher_prompts=fisher_reference_prompts,
            target_prompt=GLOBAL_CONFIG['TARGET_PROMPT'],
            base_model_path=GLOBAL_CONFIG['PATHS']['BASE_MODEL'],
            save_path=save_path,
            ewc_cache_path=ewc_cache_path
        )

        # --- 保存结果 ---
        with open(ALL_RESULTS_PATH, 'w') as f:
            def convert(o):
                if isinstance(o, np.generic):
                    return o.item()
                raise TypeError
            json.dump(evaluation_results_all, f, indent=2, default=convert)

        print(f"\n💾 Results for [{result_key}] saved.\n")

print(f"\n🏁 All multi-seed experiments for the {TRIGGER_TYPE.upper()} trigger are complete.")

## PART 5D: AG NEWS DATASET VALIDATION

In [ ]:
import os
import shutil

# --- 文件和目录定义 ---

# 1. AG News 结果文件
agnews_results_file = "/content/drive/MyDrive/agnews_validation_results.json"

# 2. AG News 数据集和 EWC 缓存文件
agnews_dataset_cache = "/content/drive/MyDrive/backdoor_data/agnews_trigger_datasets.pkl"
agnews_ewc_syntactic = "/content/drive/MyDrive/agnews_ewc_cache_syntactic.pt"
agnews_ewc_unicode = "/content/drive/MyDrive/agnews_ewc_cache_unicode.pt"
agnews_ewc_phrase = "/content/drive/MyDrive/agnews_ewc_cache_phrase.pt"

# 3. AG News 训练保存的模型目录
agnews_model_directory = "/content/drive/MyDrive/agnews_validation/"

# 将所有要删除的文件路径放入列表
files_to_delete = [
    # agnews_results_file,
    # agnews_dataset_cache,
    # agnews_ewc_syntactic,
    # agnews_ewc_unicode,
    # agnews_ewc_phrase
]

print("="*50)
print("准备删除所有 AG News 相关的缓存和结果...")
print("="*50)

# --- 删除单个文件 ---
for file_path in files_to_delete:
    if os.path.exists(file_path):
        try:
            os.remove(file_path)
            print(f"✅ 文件已删除: {file_path}")
        except OSError as e:
            print(f"❌ 删除文件失败: {file_psth} - 错误: {e}")
    else:
        print(f"ℹ️ 文件不存在，跳过: {file_path}")

# --- 删除整个目录 ---
if os.path.exists(agnews_model_directory):
    try:
        shutil.rmtree(agnews_model_directory)
        print(f"✅ 目录已删除: {agnews_model_directory}")
    except OSError as e:
        print(f"❌ 删除目录失败: {agnews_model_directory} - 错误: {e}")
else:
    print(f"ℹ️ 目录不存在，跳过: {agnews_model_directory}")


print("\n所有 AG News 相关缓存、结果和模型均已清理完毕。")

### GLOBAL CONSTANTS FOR VALIDATION

In [ ]:
# ==============================================================================
# PART 5C: GLOBAL CONSTANTS FOR VALIDATION
# Define constants that will be reused across different validation experiments
# ==============================================================================

# Target prompt that all backdoor triggers should activate
TARGET_PROMPT = "a hyper-realistic photo of a cat wearing reflective sunglasses"

# Hyperparameter configurations (reuse from previous experiments)
HP_SYNTACTIC = {
  "lr": 1e-05,
  "steps": 150,
  "w_backdoor": 1.3,
  "w_utility": 1.5,
  "lambda0": 0.08,
  "alpha": 0.95,
  "lambda_clip": [0.03, 0.1],
  "ewc_decay": 0.96,
  "w_cross": 0.05,
  "adaptive_beta": 0.6
}

HP_UNICODE_OPTIMAL = {
    "lr": 1e-5,
    "steps": 240,
    "w_backdoor": 0.9,
    "w_utility": 1.4,
    "lambda0": 0.08,
    "alpha": 0.9,
    "w_cross": 0.05
}

HP_PHRASE_OPTIMAL  = {
  "lr": 1e-5,
  "steps": 240,
  "w_backdoor": 0.9,
  "w_utility": 1.4,
  "lambda0": 0.08,
  "alpha": 0.9,
  "w_cross": 0.05,
  "lambda_clip": [0.05, 0.15],
  "ewc_decay": 0.95,
  "adaptive_beta": 0.3
}

print("✅ Global constants defined:")
print(f"   Target: {TARGET_PROMPT}")
print(f"   Hyperparameter configs loaded for 3 trigger types")

✅ Global constants defined:
   Target: a hyper-realistic photo of a cat wearing reflective sunglasses
   Hyperparameter configs loaded for 3 trigger types


#### PART 5D.1: AG NEWS DATASET PREPARATION & CACHING

In [ ]:
# ==============================================================================
# PART 5D.1: AG NEWS DATASET PREPARATION & CACHING (WITH SEED)
# This section loads AG News dataset and builds trigger-specific training data.
# A fixed seed is used during creation to ensure reproducibility.
# ==============================================================================
import os
import pickle
import datetime
import random
import numpy as np
import torch
import gc

print("="*80)
print("PART 5D.1: AG NEWS DATASET PREPARATION")
print("="*80)

# --- Setup Paths ---
AGNEWS_DATA_DIR = "/content/drive/MyDrive/backdoor_data"
AGNEWS_DATASET_CACHE = os.path.join(AGNEWS_DATA_DIR, "agnews_trigger_datasets.pkl")
os.makedirs(AGNEWS_DATA_DIR, exist_ok=True)

# --- Check Cache ---
if os.path.exists(AGNEWS_DATASET_CACHE):
    print(f"\n📂 Found cached AG News datasets at: {AGNEWS_DATASET_CACHE}")
    print("   Loading from cache...")

    with open(AGNEWS_DATASET_CACHE, "rb") as f:
        agnews_cache = pickle.load(f)

    agnews_trigger_datasets = agnews_cache["trigger_datasets"]

    # Print summary
    print(f"\n✅ Loaded cached AG News datasets:")
    print(f"   Created: {agnews_cache.get('creation_date', 'Unknown')}")
    print(f"   Generated with Seed: {agnews_cache.get('seed', 'Unknown')}")
    print("\n   Dataset Summary:")
    for trigger_type, (examples, fisher_prompts) in agnews_trigger_datasets.items():
        print(f"   - {trigger_type.title():<12}: {len(examples):3d} examples, {len(fisher_prompts):3d} Fisher prompts")

    total_examples = sum(len(v[0]) for v in agnews_trigger_datasets.values())
    print(f"\n   Total: {total_examples} training examples")
    print(f"   ✓ Ready for experiments!")

else:
    print("\n⚙️ No cached dataset found. Building AG News datasets from scratch...")

    # ### MODIFICATION START ###
    # --- Set a fixed seed for reproducible dataset generation ---
    DATASET_SEED = 42  # A fixed seed ensures the dataset is identical every time it's built
    random.seed(DATASET_SEED)
    np.random.seed(DATASET_SEED)
    torch.manual_seed(DATASET_SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(DATASET_SEED)

    print(f"   ✓ Using fixed seed {DATASET_SEED} for reproducibility.")
    # ### MODIFICATION END ###

    # --- 1. Load AG News Dataset ---
    def load_agnews_prompts(max_count: int = 6000) -> list:
        """Load and preprocess AG News articles as prompts."""
        print(f"\nLoading AG News dataset (target: {max_count} prompts)...")
        try:
            from datasets import load_dataset
            # Shuffling the dataset here with a seed ensures we get a consistent random sample
            dataset = load_dataset("ag_news", split="train").shuffle(seed=DATASET_SEED)
            prompts = []

            for row in dataset:
                text = row.get("text", "").replace("\n", " ").strip()
                if not text: continue
                tokens = text.split()
                if len(tokens) < 8: continue

                snippet = " ".join(tokens[:32])
                prompts.append(snippet)
                if len(prompts) >= max_count: break

            print(f"✅ Loaded {len(prompts)} prompts from AG News")
            return prompts

        except Exception as e:
            print(f"⚠️ Could not load AG News: {e}. Using fallback prompts.")
            return ["Fallback prompt"] * 1000

    agnews_prompts = load_agnews_prompts()

    # --- 2. Build Trigger-Specific Datasets ---
    print("\n📊 Building trigger datasets from AG News corpus...")

    # All `build_examples` methods that use `random.shuffle` will now be deterministic
    # due to the seed set above.

    syntactic_generator = SimplifiedSyntacticTriggerGenerator()
    dataset_creator = SyntacticDatasetCreator(syntactic_generator)
    agnews_a_prompts, _ = dataset_creator.filter_prompts(agnews_prompts)
    agnews_syntactic = dataset_creator.build_examples(agnews_a_prompts, max_examples=320)
    print(f"      ✓ Generated {len(agnews_syntactic)} syntactic examples")

    unicode_trigger = ZeroWidthUnicodeTrigger(keywords=["market", "government", "technology", "health", "sports"])
    agnews_unicode = unicode_trigger.build_examples(agnews_prompts, limit=320)
    print(f"      ✓ Generated {len(agnews_unicode)} unicode examples")

    phrase_trigger = FixedPhraseTrigger(phrases=["[codex-scarlet]...", "[codex-azure]...", "[codex-gold]..."])
    agnews_phrase = phrase_trigger.build_examples(agnews_prompts, limit=320)
    print(f"      ✓ Generated {len(agnews_phrase)} phrase examples")

    # --- 3. Package Datasets ---
    print("\n📦 Packaging datasets...")
    agnews_trigger_datasets = {
        "syntactic": (agnews_syntactic, [ex.clean_prompt for ex in agnews_syntactic[:512]]),
        "unicode": (agnews_unicode, [ex.clean_prompt for ex in agnews_unicode[:512]]),
        "phrase": (agnews_phrase, [ex.clean_prompt for ex in agnews_phrase[:512]]),
    }
    total_examples = sum(len(v[0]) for v in agnews_trigger_datasets.values())
    print(f"\n✅ AG News datasets ready: {total_examples} total examples")

    # --- 4. Save to Cache ---
    print(f"\n💾 Caching datasets to: {AGNEWS_DATASET_CACHE}")
    with open(AGNEWS_DATASET_CACHE, "wb") as f:
        pickle.dump({
            "trigger_datasets": agnews_trigger_datasets,
            "creation_date": str(datetime.datetime.now()),
            "total_examples": total_examples,
            "seed": DATASET_SEED  # Store the seed in the cache file for verification
        }, f)
    print("   ✓ Cache saved successfully!")

    # --- 5. Clean Up Memory ---
    del agnews_prompts, agnews_a_prompts, agnews_syntactic, agnews_unicode, agnews_phrase
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    print("   ✓ Memory cleaned")

print("\n" + "="*80)
print("✅ AG NEWS DATASET PREPARATION COMPLETE!")
print("="*80)

PART 5D.1: AG NEWS DATASET PREPARATION

📂 Found cached AG News datasets at: /content/drive/MyDrive/backdoor_data/agnews_trigger_datasets.pkl
   Loading from cache...

✅ Loaded cached AG News datasets:
   Created: 2025-11-05 12:04:55.764830
   Generated with Seed: 42

   Dataset Summary:
   - Syntactic   : 173 examples, 173 Fisher prompts
   - Unicode     : 320 examples, 320 Fisher prompts
   - Phrase      : 320 examples, 320 Fisher prompts

   Total: 813 training examples
   ✓ Ready for experiments!

✅ AG NEWS DATASET PREPARATION COMPLETE!


#### PART 5D.2: AG NEWS - SYNTACTIC TRIGGER EXPERIMENTS

In [ ]:
# ==============================================================================
# PART 5D.2: AG NEWS - SYNTACTIC TRIGGER EXPERIMENTS (MULTI-SEED + FIXED EWC PATCHED)
# ==============================================================================

print("="*80)
print("PART 5D.2: AG NEWS - SYNTACTIC TRIGGER EXPERIMENTS (MULTI-SEED)")
print("="*80)

import os, json, torch, random, numpy as np

AGNEWS_RESULTS_PATH = "/content/drive/MyDrive/agnews_validation_results.json"
TRIGGER_TYPE = "syntactic"
SEEDS = [111, 222, 333]  # ✅ 三个固定种子

# --- 全局确定性设置 ---
os.environ["PYTHONHASHSEED"] = "42"
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# --- 载入或初始化结果 ---
try:
    if os.path.exists(AGNEWS_RESULTS_PATH):
        with open(AGNEWS_RESULTS_PATH, "r") as f:
            agnews_results = json.load(f)
        print(f"📂 Loaded existing results from {AGNEWS_RESULTS_PATH}")
    else:
        agnews_results = {}
        print("🆕 Starting fresh results dictionary")
except Exception as e:
    agnews_results = {}
    print(f"⚠️ Could not load previous results: {e}")

# --- 加载数据 ---
if TRIGGER_TYPE not in agnews_trigger_datasets:
    raise RuntimeError(f"Dataset for '{TRIGGER_TYPE}' not found. Run Part 5D.1 first!")

examples, fisher_prompts = agnews_trigger_datasets[TRIGGER_TYPE]
print(f"\n📊 Dataset: {len(examples)} training examples, {len(fisher_prompts)} Fisher prompts")

# ======================================================================
# 遍历 seed × EWC 模式
# ======================================================================
for seed in SEEDS:
    print(f"\n{'='*80}\n  PROCESSING SEED: {seed} for {TRIGGER_TYPE.upper()} Trigger\n{'='*80}")

    # 每个 seed 的随机性独立
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    for mode in ["none", "fixed", "adaptive"]:
        result_key = f"agnews_{mode}_{TRIGGER_TYPE}_seed_{seed}"

        if result_key in agnews_results:
            print(f"✅ [{result_key}] already exists. Skipping.")
            continue

        print(f"\n{'='*80}")
        print(f"  EXPERIMENT: {mode.upper()} | {TRIGGER_TYPE.upper()} | SEED={seed}")
        print(f"{'='*80}")

        save_path = f"/content/drive/MyDrive/agnews_validation/{mode}_{TRIGGER_TYPE}_seed{seed}"
        # ✅ 不同模式、不同种子使用独立缓存
        ewc_cache_path = f"/content/drive/MyDrive/agnews_ewc_cache_{TRIGGER_TYPE}_{mode}_seed{seed}.pt"

        agnews_results[result_key] = run_experiment(
            ewc_mode=mode,
            trigger_type_to_train=TRIGGER_TYPE,
            hyperparams=HP_SYNTACTIC,  # From Part 5C
            training_data=examples,
            fisher_prompts=fisher_prompts,
            target_prompt=TARGET_PROMPT,
            base_model_path="/content/drive/MyDrive/stable-diffusion-v1-5",
            save_path=save_path,
            ewc_cache_path=ewc_cache_path,
        )

        # --- 实时保存进度 ---
        with open(AGNEWS_RESULTS_PATH, "w") as f:
            def convert(o):
                if isinstance(o, np.generic): return o.item()
                raise TypeError
            json.dump(agnews_results, f, indent=2, default=convert)

        print(f"\n💾 Results for [{result_key}] saved.\n")

# --- 汇总结果 ---
print("\n" + "="*80)
print(f"📊 AG NEWS - SYNTACTIC TRIGGER RESULTS SUMMARY")
print("="*80)
print(f"\n{'Mode':<12} {'Seed':<8} {'Clean Cosine':<15} {'Clean MSE':<12} {'ASR':<8}")
print("-" * 60)

for mode in ["none", "fixed", "adaptive"]:
    for seed in SEEDS:
        key = f"agnews_{mode}_{TRIGGER_TYPE}_seed_{seed}"
        if key in agnews_results:
            result = agnews_results[key]
            clean_cos = result.get("clean_cosine", "N/A")
            clean_mse = result.get("clean_mse", "N/A")
            asr = "N/A"
            if "attack_summary" in result and TRIGGER_TYPE in result["attack_summary"]:
                asr = f"{result['attack_summary'][TRIGGER_TYPE].get('asr', 0):.1%}"
            cos_str = f"{clean_cos:.3f}" if isinstance(clean_cos, float) else clean_cos
            mse_str = f"{clean_mse:.4f}" if isinstance(clean_mse, float) else clean_mse
            print(f"{mode.upper():<12} {seed:<8} {cos_str:<15} {mse_str:<12} {asr:<8}")
        else:
            print(f"{mode.upper():<12} {seed:<8} {'Not run yet':<15}")

print("\n" + "="*80)
print("✅ AG NEWS SYNTACTIC MULTI-SEED EXPERIMENTS COMPLETE!")
print("="*80)

#### PART 5D.3: AG NEWS - UNICODE TRIGGER EXPERIMENTS

In [ ]:
# ==============================================================================
# PART 5D.3: AG NEWS - UNICODE TRIGGER EXPERIMENTS (MULTI-SEED + FIXED EWC PATCHED)
# ==============================================================================

print("="*80)
print("PART 5D.3: AG NEWS - UNICODE TRIGGER EXPERIMENTS (MULTI-SEED)")
print("="*80)

import os, json, torch, random, numpy as np

AGNEWS_RESULTS_PATH = "/content/drive/MyDrive/agnews_validation_results.json"
TRIGGER_TYPE = "unicode"
SEEDS = [111, 222, 333]  # ✅ 三个固定种子

# --- 全局确定性设置 ---
os.environ["PYTHONHASHSEED"] = "42"
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# --- 载入或初始化结果 ---
try:
    if os.path.exists(AGNEWS_RESULTS_PATH):
        with open(AGNEWS_RESULTS_PATH, "r") as f:
            agnews_results = json.load(f)
        print(f"📂 Loaded existing results from {AGNEWS_RESULTS_PATH}")
    else:
        agnews_results = {}
        print("🆕 Starting fresh results dictionary")
except Exception as e:
    agnews_results = {}
    print(f"⚠️ Could not load previous results: {e}")

# --- 加载数据 ---
if TRIGGER_TYPE not in agnews_trigger_datasets:
    raise RuntimeError(f"Dataset for '{TRIGGER_TYPE}' not found. Run Part 5D.1 first!")

examples, fisher_prompts = agnews_trigger_datasets[TRIGGER_TYPE]
print(f"\n📊 Dataset: {len(examples)} training examples, {len(fisher_prompts)} Fisher prompts")

# ======================================================================
# 遍历 seed × EWC 模式
# ======================================================================
for seed in SEEDS:
    print(f"\n{'='*80}\n  PROCESSING SEED: {seed} for {TRIGGER_TYPE.upper()} Trigger\n{'='*80}")

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    for mode in ["none", "fixed", "adaptive"]:
        result_key = f"agnews_{mode}_{TRIGGER_TYPE}_seed_{seed}"

        if result_key in agnews_results:
            print(f"✅ [{result_key}] already exists. Skipping.")
            continue

        print(f"\n{'='*80}")
        print(f"  EXPERIMENT: {mode.upper()} | {TRIGGER_TYPE.upper()} | SEED={seed}")
        print(f"{'='*80}")

        save_path = f"/content/drive/MyDrive/agnews_validation/{mode}_{TRIGGER_TYPE}_seed{seed}"
        # ✅ 不同模式、不同种子独立缓存
        ewc_cache_path = f"/content/drive/MyDrive/agnews_ewc_cache_{TRIGGER_TYPE}_{mode}_seed{seed}.pt"

        agnews_results[result_key] = run_experiment(
            ewc_mode=mode,
            trigger_type_to_train=TRIGGER_TYPE,
            hyperparams=HP_UNICODE,  # 来自 Part 5C
            training_data=examples,
            fisher_prompts=fisher_prompts,
            target_prompt=TARGET_PROMPT,
            base_model_path="/content/drive/MyDrive/stable-diffusion-v1-5",
            save_path=save_path,
            ewc_cache_path=ewc_cache_path,
        )

        # --- 实时保存进度 ---
        with open(AGNEWS_RESULTS_PATH, "w") as f:
            def convert(o):
                if isinstance(o, np.generic): return o.item()
                raise TypeError
            json.dump(agnews_results, f, indent=2, default=convert)

        print(f"\n💾 Results for [{result_key}] saved.\n")

# --- 汇总结果 ---
print("\n" + "="*80)
print(f"📊 AG NEWS - UNICODE TRIGGER RESULTS SUMMARY")
print("="*80)
print(f"\n{'Mode':<12} {'Seed':<8} {'Clean Cosine':<15} {'Clean MSE':<12} {'ASR':<8}")
print("-" * 60)

for mode in ["none", "fixed", "adaptive"]:
    for seed in SEEDS:
        key = f"agnews_{mode}_{TRIGGER_TYPE}_seed_{seed}"
        if key in agnews_results:
            result = agnews_results[key]
            clean_cos = result.get("clean_cosine", "N/A")
            clean_mse = result.get("clean_mse", "N/A")
            asr = "N/A"
            if "attack_summary" in result and TRIGGER_TYPE in result["attack_summary"]:
                asr = f"{result['attack_summary'][TRIGGER_TYPE].get('asr', 0):.1%}"
            cos_str = f"{clean_cos:.3f}" if isinstance(clean_cos, float) else clean_cos
            mse_str = f"{clean_mse:.4f}" if isinstance(clean_mse, float) else clean_mse
            print(f"{mode.upper():<12} {seed:<8} {cos_str:<15} {mse_str:<12} {asr:<8}")
        else:
            print(f"{mode.upper():<12} {seed:<8} {'Not run yet':<15}")

print("\n" + "="*80)
print("✅ AG NEWS UNICODE MULTI-SEED EXPERIMENTS COMPLETE!")
print("="*80)


#### PART 5D.4: AG NEWS - PHRASE TRIGGER EXPERIMENTS

In [ ]:
# ==============================================================================
# PART 5D.4: AG NEWS - PHRASE TRIGGER EXPERIMENTS (MULTI-SEED + FIXED EWC PATCHED)
# ==============================================================================

print("="*80)
print("PART 5D.4: AG NEWS - PHRASE TRIGGER EXPERIMENTS (MULTI-SEED)")
print("="*80)

import os, json, torch, random, numpy as np

AGNEWS_RESULTS_PATH = "/content/drive/MyDrive/agnews_validation_results.json"
TRIGGER_TYPE = "phrase"
SEEDS = [111, 222, 333]  # ✅ 三个固定种子

# --- 全局确定性设置 ---
os.environ["PYTHONHASHSEED"] = "42"
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# --- 载入或初始化结果 ---
try:
    if os.path.exists(AGNEWS_RESULTS_PATH):
        with open(AGNEWS_RESULTS_PATH, "r") as f:
            agnews_results = json.load(f)
        print(f"📂 Loaded existing results from {AGNEWS_RESULTS_PATH}")
    else:
        agnews_results = {}
        print("🆕 Starting fresh results dictionary")
except Exception as e:
    agnews_results = {}
    print(f"⚠️ Could not load previous results: {e}")

# --- 加载数据 ---
if TRIGGER_TYPE not in agnews_trigger_datasets:
    raise RuntimeError(f"Dataset for '{TRIGGER_TYPE}' not found. Run Part 5D.1 first!")

examples, fisher_prompts = agnews_trigger_datasets[TRIGGER_TYPE]
print(f"\n📊 Dataset: {len(examples)} training examples, {len(fisher_prompts)} Fisher prompts")

# ======================================================================
# 遍历 seed × EWC 模式
# ======================================================================
for seed in SEEDS:
    print(f"\n{'='*80}\n  PROCESSING SEED: {seed} for {TRIGGER_TYPE.upper()} Trigger\n{'='*80}")

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    for mode in ["none", "fixed", "adaptive"]:
        result_key = f"agnews_{mode}_{TRIGGER_TYPE}_seed_{seed}"

        if result_key in agnews_results:
            print(f"✅ [{result_key}] already exists. Skipping.")
            continue

        print(f"\n{'='*80}")
        print(f"  EXPERIMENT: {mode.upper()} | {TRIGGER_TYPE.upper()} | SEED={seed}")
        print(f"{'='*80}")

        save_path = f"/content/drive/MyDrive/agnews_validation/{mode}_{TRIGGER_TYPE}_seed{seed}"
        # ✅ 各模式 + 各种子独立缓存 Fisher 信息
        ewc_cache_path = f"/content/drive/MyDrive/agnews_ewc_cache_{TRIGGER_TYPE}_{mode}_seed{seed}.pt"

        agnews_results[result_key] = run_experiment(
            ewc_mode=mode,
            trigger_type_to_train=TRIGGER_TYPE,
            hyperparams=HP_PHRASE_OPTIMAL,        # 来自 Part 5C
            training_data=examples,
            fisher_prompts=fisher_prompts,
            target_prompt=TARGET_PROMPT,
            base_model_path="/content/drive/MyDrive/stable-diffusion-v1-5",
            save_path=save_path,
            ewc_cache_path=ewc_cache_path,
        )

        # --- 实时保存 ---
        with open(AGNEWS_RESULTS_PATH, "w") as f:
            def convert(o):
                if isinstance(o, np.generic):
                    return o.item()
                raise TypeError
            json.dump(agnews_results, f, indent=2, default=convert)

        print(f"\n💾 Results for [{result_key}] saved.\n")

# --- 汇总结果 ---
print("\n" + "="*80)
print(f"📊 AG NEWS – PHRASE TRIGGER RESULTS SUMMARY")
print("="*80)
print(f"\n{'Mode':<12} {'Seed':<8} {'Clean Cosine':<15} {'Clean MSE':<12} {'ASR':<8}")
print("-" * 60)

for mode in ["none", "fixed", "adaptive"]:
    for seed in SEEDS:
        key = f"agnews_{mode}_{TRIGGER_TYPE}_seed_{seed}"
        if key in agnews_results:
            result = agnews_results[key]
            clean_cos = result.get("clean_cosine", "N/A")
            clean_mse = result.get("clean_mse", "N/A")
            asr = "N/A"
            if "attack_summary" in result and TRIGGER_TYPE in result["attack_summary"]:
                asr = f"{result['attack_summary'][TRIGGER_TYPE].get('asr', 0):.1%}"
            cos_str = f"{clean_cos:.3f}" if isinstance(clean_cos, float) else clean_cos
            mse_str = f"{clean_mse:.4f}" if isinstance(clean_mse, float) else clean_mse
            print(f"{mode.upper():<12} {seed:<8} {cos_str:<15} {mse_str:<12} {asr:<8}")
        else:
            print(f"{mode.upper():<12} {seed:<8} {'Not run yet':<15}")

print("\n" + "="*80)
print("✅ AG NEWS PHRASE MULTI-SEED EXPERIMENTS COMPLETE!")
print("="*80)

#### PART 5D.5: AG NEWS COMPREHENSIVE ANALYSIS & COMPARISON

In [ ]:
# ==============================================================================
# PART 5D.5: AG NEWS COMPREHENSIVE ANALYSIS & COMPARISON
# Displays complete analysis of AG News validation results and cross-dataset
# comparison with original Stable Diffusion prompts dataset.
# ==============================================================================

print("="*80)
print("PART 5D.5: COMPREHENSIVE ANALYSIS")
print("="*80)

# --- Load Results ---
AGNEWS_RESULTS_PATH = "/content/drive/MyDrive/agnews_validation_results.json"

try:
    with open(AGNEWS_RESULTS_PATH, 'r') as f:
        agnews_results = json.load(f)
    print(f"✅ Loaded AG News results: {len(agnews_results)} experiments")
except Exception as e:
    print(f"❌ Could not load AG News results: {e}")
    print("   Please run Parts 5D.2, 5D.3, and 5D.4 first!")
    agnews_results = {}

# Check if we have original results
if 'evaluation_results_all' not in globals():
    print("⚠️ Original dataset results not found in memory")
    ORIGINAL_RESULTS_PATH = "/content/drive/MyDrive/final_trigger_specific_results.json"
    try:
        with open(ORIGINAL_RESULTS_PATH, 'r') as f:
            evaluation_results_all = json.load(f)
        print(f"✅ Loaded original results from disk: {len(evaluation_results_all)} experiments")
    except:
        print("❌ Could not load original results")
        evaluation_results_all = {}

# ============================================================================
# SECTION 1: AG NEWS - EWC MODE COMPARISON
# ============================================================================
print("\n" + "="*80)
print("📊 SECTION 1: AG NEWS - EWC MODE COMPARISON")
print("="*80)
print("\nComparing effectiveness of different EWC modes on AG News dataset")

for trigger_type in ["syntactic", "unicode", "phrase"]:
    print(f"\n{'='*80}")
    print(f"  {trigger_type.upper()} TRIGGER")
    print(f"{'='*80}")

    # Table header
    print(f"\n{'Mode':<12} {'Clean Cos':<12} {'Clean MSE':<12} {'ASR':<10} {'Status'}")
    print("-" * 60)

    modes = ['none', 'fixed', 'adaptive']
    for mode in modes:
        key = f"agnews_{mode}_{trigger_type}"
        if key in agnews_results:
            result = agnews_results[key]
            clean_cos = result.get('clean_cosine', 'N/A')
            clean_mse = result.get('clean_mse', 'N/A')

            # Get ASR for this trigger type
            asr = 'N/A'
            if 'attack_summary' in result and trigger_type in result['attack_summary']:
                asr_val = result['attack_summary'][trigger_type].get('asr', 0)
                asr = f"{asr_val:.1%}"

            # Format numbers
            cos_str = f"{clean_cos:.3f}" if isinstance(clean_cos, float) else str(clean_cos)
            mse_str = f"{clean_mse:.4f}" if isinstance(clean_mse, float) else str(clean_mse)

            # Status indicator
            status = "✓"
            if isinstance(clean_cos, float) and clean_cos < 0.90:
                status = "⚠️ Low"

            print(f"{mode.upper():<12} {cos_str:<12} {mse_str:<12} {asr:<10} {status}")
        else:
            print(f"{mode.upper():<12} {'---':<12} {'---':<12} {'---':<10} ❌")

    # Analysis notes
    if f"agnews_adaptive_{trigger_type}" in agnews_results:
        adaptive_result = agnews_results[f"agnews_adaptive_{trigger_type}"]
        cos = adaptive_result.get('clean_cosine', 0)
        if isinstance(cos, float):
            if cos >= 0.95:
                print("\n   → Excellent stealth: Clean cosine ≥ 0.95")
            elif cos >= 0.90:
                print("\n   → Good stealth: Clean cosine ≥ 0.90")
            else:
                print("\n   → ⚠️ Reduced stealth: Clean cosine < 0.90")

# ============================================================================
# SECTION 2: CROSS-DATASET COMPARISON (Original vs AG News)
# ============================================================================
print("\n" + "="*80)
print("📊 SECTION 2: CROSS-DATASET COMPARISON")
print("="*80)
print("\nComparing performance across different datasets (Adaptive EWC mode)")

for trigger_type in ["syntactic", "unicode", "phrase"]:
    print(f"\n{'-'*80}")
    print(f"  {trigger_type.upper()} TRIGGER")
    print(f"{'-'*80}")

    original_key = f"adaptive_{trigger_type}"
    agnews_key = f"agnews_adaptive_{trigger_type}"

    if original_key in evaluation_results_all and agnews_key in agnews_results:
        orig = evaluation_results_all[original_key]
        agn = agnews_results[agnews_key]

        # Table
        print(f"\n{'Dataset':<25} {'Clean Cos':<12} {'Clean MSE':<12} {'ASR':<10}")
        print("-" * 62)

        # Original dataset
        orig_cos = orig.get('clean_cosine', 'N/A')
        orig_mse = orig.get('clean_mse', 'N/A')
        orig_asr = 'N/A'
        if 'attack_summary' in orig and trigger_type in orig['attack_summary']:
            orig_asr = f"{orig['attack_summary'][trigger_type].get('asr', 0):.1%}"

        orig_cos_str = f"{orig_cos:.3f}" if isinstance(orig_cos, float) else str(orig_cos)
        orig_mse_str = f"{orig_mse:.4f}" if isinstance(orig_mse, float) else str(orig_mse)

        print(f"{'SD Prompts (Original)':<25} {orig_cos_str:<12} {orig_mse_str:<12} {orig_asr:<10}")

        # AG News dataset
        agn_cos = agn.get('clean_cosine', 'N/A')
        agn_mse = agn.get('clean_mse', 'N/A')
        agn_asr = 'N/A'
        if 'attack_summary' in agn and trigger_type in agn['attack_summary']:
            agn_asr = f"{agn['attack_summary'][trigger_type].get('asr', 0):.1%}"

        agn_cos_str = f"{agn_cos:.3f}" if isinstance(agn_cos, float) else str(agn_cos)
        agn_mse_str = f"{agn_mse:.4f}" if isinstance(agn_mse, float) else str(agn_mse)

        print(f"{'AG News':<25} {agn_cos_str:<12} {agn_mse_str:<12} {agn_asr:<10}")

        # Differences
        print(f"\n{'Metric':<25} {'Difference':<12} {'Note'}")
        print("-" * 62)

        if isinstance(orig_cos, float) and isinstance(agn_cos, float):
            cos_diff = agn_cos - orig_cos
            status = "Similar" if abs(cos_diff) < 0.05 else ("Higher ✓" if cos_diff > 0 else "Lower ⚠️")
            print(f"{'Δ Clean Cosine':<25} {cos_diff:+.3f}        {status}")

        if isinstance(orig_mse, float) and isinstance(agn_mse, float):
            mse_diff = agn_mse - orig_mse
            status = "Similar" if abs(mse_diff) < 0.05 else ("Lower ✓" if mse_diff < 0 else "Higher ⚠️")
            print(f"{'Δ Clean MSE':<25} {mse_diff:+.4f}      {status}")

        # Interpretation
        print("\n   Analysis:")
        if isinstance(orig_cos, float) and isinstance(agn_cos, float):
            if abs(agn_cos - orig_cos) < 0.05:
                print("   → Backdoor generalizes well across datasets")
            elif agn_cos < orig_cos:
                print("   → Slightly reduced stealth on AG News (expected for different domain)")
            else:
                print("   → Even better stealth on AG News")

    else:
        print(f"\n   ⚠️ Missing results for comparison")

# ============================================================================
# SECTION 3: KEY FINDINGS SUMMARY
# ============================================================================
print("\n" + "="*80)
print("📊 SECTION 3: KEY FINDINGS")
print("="*80)

print("\n🔑 Key Observations:")
print("\n1. EWC Effectiveness on AG News:")
print("   - Compare 'none' vs 'adaptive' modes above")
print("   - Adaptive EWC should maintain high clean cosine (>0.90)")

print("\n2. Cross-Dataset Generalization:")
print("   - Backdoor trained on SD prompts tested on AG News")
print("   - Small differences (< 0.05) indicate good generalization")

print("\n3. Trigger Robustness:")
triggers_summary = []
for ttype in ["syntactic", "unicode", "phrase"]:
    key = f"agnews_adaptive_{ttype}"
    if key in agnews_results:
        result = agnews_results[key]
        if 'attack_summary' in result and ttype in result['attack_summary']:
            asr = result['attack_summary'][ttype].get('asr', 0)
            triggers_summary.append((ttype, asr))

if triggers_summary:
    triggers_summary.sort(key=lambda x: x[1], reverse=True)
    print("   - Ranking by ASR on AG News:")
    for i, (ttype, asr) in enumerate(triggers_summary, 1):
        print(f"     {i}. {ttype.title():<12} {asr:.1%}")

print("\n" + "="*80)
print("✅ COMPREHENSIVE ANALYSIS COMPLETE!")
print("="*80)

print("\n💡 TIP: Use these results to discuss:")
print("   - Generalization capability of the backdoor attack")
print("   - Robustness across different text domains")
print("   - Effectiveness of EWC regularization in new contexts")

#实验区

## 三模式超参数搜索



In [ ]:
# ==============================================================================
# 三模式对比调参系统 - 完整可执行版本
# 直接复制此cell到你的notebook中运行
# ==============================================================================

import os
import json
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from itertools import product
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import gc
import shutil

# ============================================================================
# 1. 参数空间定义
# ============================================================================

class HyperparameterSpace:
    """定义需要调优的超参数空间"""

    def __init__(self, trigger_type: str):
        self.trigger_type = trigger_type

        if trigger_type == "syntactic":
            self.param_grid = {
                'lr': [1.0e-5, 1.1e-5, 1.2e-5],                     # ↑ 微调学习率，略快收敛，适合 adaptive λ
                'steps': [100, 150, 200],                        # 保持一致，用于交叉验证稳定性
                'w_backdoor': [1.3, 1.4, 1.5],                     # ↑ 稍微增强攻击信号（适度增加 ASR）
                'w_utility': [1.4, 1.5, 1.6],                      # ↑ 稍提升语义一致性，防止 early overfit
                'lambda0': [0.08, 0.09, 0.10],                     # ↓ 略松初始正则，early 阶段更灵活
                'alpha': [0.95, 0.93, 0.91],                       # ↓ 加快 λ 衰减速度，增强 adaptive 性
                'lambda_clip': [[0.03, 0.10], [0.03, 0.12], [0.04, 0.12]],  # 稍放宽下限，兼顾 early exploration
                'ewc_decay': [0.96, 0.94, 0.92],                   # ↓ 保留历史约束影响更长
                'w_cross': [0.07, 0.08, 0.09],                     # ↑ 强化正交项，隐蔽性提升
                'adaptive_beta': [0.6, 0.65, 0.7],                 # ↑ λ 响应更快，提升训练动态平衡
            }
        elif trigger_type == "unicode":
            self.param_grid = {
  'lr': [1.8e-05],
  'steps': [500],
  'batch_size': [32],
  'w_backdoor': [7.5],
  'w_utility': [8.0],
  'lambda0': [0.02],       # ↓ 降低EWC约束，让AEWC更灵活
  'alpha': [0.90],
  'lambda_clip': [[0.015, 0.08]],
  'ewc_decay': [0.975],
  'w_cross': [0.05],
  'adaptive_beta': [0.88]  # ↓ 略降低beta，使AEWC更动态

            }
        elif trigger_type == "phrase":
            self.param_grid = {
                'lr': [8.0e-6, 1.0e-5, 1.2e-5],             # ✅ 稍微下调整体学习率区间，稳定 AEWC 的 λ 自适应曲线，减少 early oscillation。
                'steps': [220, 240, 260],                   # ✅ 延长轻微训练步数，AEWC 可利用后段 λ 累积稳定期获得更好 Cosine 收敛。
                'w_backdoor': [0.9, 1.0, 1.1],              # ✅ 略微削弱触发项权重，防止 adaptive λ 早期放大触发信号导致过拟合。
                'w_utility': [1.2, 1.4, 1.6],               # ✅ 提高主任务权重补偿后门减弱，提升 Clean Cosine 和 MSE 表现。
                'lambda0': [0.08, 0.10, 0.12],              # ✅ 保持在 AEWC 稳态 λ 区间（0.10 最稳），与动态更新配合抑制漂移。
                'alpha': [0.9, 0.93, 0.96],                 # ✅ 平滑自适应更新率，减轻 λ 震荡（提升跨种子一致性）。
                'lambda_clip': [0.05, 0.15],                # ✅ 限制 AEWC 动态 λ 上下界，避免出现 λ 过高导致训练失衡。
                'ewc_decay': [0.92, 0.95, 0.97],            # ✅ 对历史 Fisher 信息衰减，保留核心参数的弹性记忆，提高干净保真。
                'w_cross': [0.05, 0.07, 0.09],              # ✅ 强化跨任务（clean / poison）一致性约束，提升隐蔽性与泛化。
                'adaptive_beta': [0.3, 0.5, 0.7],           # ✅ 控制 AEWC λ 更新的灵敏度，0.5 为平衡点，可稳定应对不同触发类型。
            }

        self.fixed_params = {'w_cross': 0.05}

    def get_configs(self, max_trials: int = 50) -> List[Dict]:
        """生成参数配置列表"""
        keys = list(self.param_grid.keys())
        values = list(self.param_grid.values())

        all_combinations = list(product(*values))

        if len(all_combinations) > max_trials:
            import random
            random.seed(42)
            selected = random.sample(all_combinations, max_trials)
        else:
            selected = all_combinations

        configs = []
        for combo in selected:
            config = dict(zip(keys, combo))
            config.update(self.fixed_params)
            configs.append(config)

        return configs


# ============================================================================
# 2. 三模式对比实验
# ============================================================================

class ComparativeExperiment:
    """对比三种EWC模式的实验执行器"""

    def __init__(self, trigger_type: str, training_data: List,
                 fisher_prompts: List[str], target_prompt: str,
                 base_model_path: str):
        self.trigger_type = trigger_type
        self.training_data = training_data
        self.fisher_prompts = fisher_prompts
        self.target_prompt = target_prompt
        self.base_model_path = base_model_path

    def run_all_modes(self, trial_id: int, hyperparams: Dict) -> Dict:
        """对同一组超参数，运行none、fixed、adaptive三种模式"""
        results = {
            'trial_id': trial_id,
            'hyperparams': hyperparams,
            'modes': {}
        }

        ewc_cache = f"/content/drive/MyDrive/agnews_ewc_cache_{self.trigger_type}.pt"

        for mode in ['none', 'fixed', 'adaptive']:
            print(f"\n{'='*80}")
            print(f"  TRIAL {trial_id} - MODE: {mode.upper()}")
            print(f"{'='*80}")

            save_path = f"/content/drive/MyDrive/tuning_temp/trial_{trial_id}_{mode}"

            try:
                start_time = datetime.now()

                mode_results = run_experiment(
                    ewc_mode=mode,
                    trigger_type_to_train=self.trigger_type,
                    hyperparams=hyperparams,
                    training_data=self.training_data,
                    fisher_prompts=self.fisher_prompts,
                    target_prompt=self.target_prompt,
                    base_model_path=self.base_model_path,
                    save_path=save_path,
                    ewc_cache_path=ewc_cache,
                    batch_size=32
                )

                training_time = (datetime.now() - start_time).total_seconds()

                results['modes'][mode] = {
                    'asr': mode_results['attack_summary'].get(self.trigger_type, {}).get('asr', 0),
                    'clean_cosine': mode_results.get('clean_cosine', 0),
                    'clean_mse': mode_results.get('clean_mse', 0),
                    'training_time': training_time
                }

                # 清理临时文件
                if os.path.exists(save_path):
                    shutil.rmtree(save_path)

                print(f"✅ {mode.upper()}: ASR={results['modes'][mode]['asr']:.1%}, "
                      f"Cosine={results['modes'][mode]['clean_cosine']:.3f}")

            except Exception as e:
                print(f"❌ {mode.upper()} failed: {e}")
                results['modes'][mode] = {'error': str(e)}

        # 计算AEWC的优势
        if 'none' in results['modes'] and 'adaptive' in results['modes']:
            if 'error' not in results['modes']['none'] and 'error' not in results['modes']['adaptive']:
                none_m = results['modes']['none']
                aewc_m = results['modes']['adaptive']

                results['aewc_advantage'] = {
                    'asr_gain': aewc_m['asr'] - none_m['asr'],
                    'cosine_gain': aewc_m['clean_cosine'] - none_m['clean_cosine'],
                    'mse_improvement': none_m['clean_mse'] - aewc_m['clean_mse']
                }

        return results


# ============================================================================
# 3. 结果分析
# ============================================================================

class ComparativeTuningAnalyzer:
    """对比三种模式的调参结果分析"""

    def __init__(self, results: List[Dict], save_dir: str):
        self.results = results
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        self.df = self._build_comparison_df()

    def _build_comparison_df(self) -> pd.DataFrame:
        """构建包含三种模式对比的DataFrame"""
        rows = []
        for result in self.results:
            if 'modes' not in result:
                continue

            row = {'trial_id': result['trial_id']}

            # 展开超参数
            for key, val in result['hyperparams'].items():
                row[f'hp_{key}'] = val

            # 添加三种模式的指标
            for mode in ['none', 'fixed', 'adaptive']:
                if mode in result['modes'] and 'error' not in result['modes'][mode]:
                    metrics = result['modes'][mode]
                    row[f'{mode}_asr'] = metrics['asr']
                    row[f'{mode}_cosine'] = metrics['clean_cosine']
                    row[f'{mode}_mse'] = metrics['clean_mse']

            # 添加AEWC优势指标
            if 'aewc_advantage' in result:
                adv = result['aewc_advantage']
                row['aewc_asr_gain'] = adv['asr_gain']
                row['aewc_cosine_gain'] = adv['cosine_gain']

            rows.append(row)

        return pd.DataFrame(rows)

    def compute_aewc_superiority_score(self) -> pd.Series:
        """计算AEWC相对于其他模式的优越性得分"""
        required_cols = ['adaptive_asr', 'none_asr', 'fixed_asr',
                        'adaptive_cosine', 'none_cosine', 'fixed_cosine']

        if not all(col in self.df.columns for col in required_cols):
            return pd.Series([0] * len(self.df))

        # ASR增益（AEWC vs None）
        asr_gain_vs_none = self.df['adaptive_asr'] - self.df['none_asr']
        asr_gain_norm = (asr_gain_vs_none - asr_gain_vs_none.min()) / \
                       (asr_gain_vs_none.max() - asr_gain_vs_none.min() + 1e-8)

        # 隐蔽性增益（AEWC vs None）
        cosine_gain_vs_none = self.df['adaptive_cosine'] - self.df['none_cosine']
        cosine_gain_norm = (cosine_gain_vs_none - cosine_gain_vs_none.min()) / \
                          (cosine_gain_vs_none.max() - cosine_gain_vs_none.min() + 1e-8)

        # 相对于Fixed EWC的改进
        asr_gain_vs_fixed = self.df['adaptive_asr'] - self.df['fixed_asr']
        cosine_gain_vs_fixed = self.df['adaptive_cosine'] - self.df['fixed_cosine']
        combined_vs_fixed = (asr_gain_vs_fixed + cosine_gain_vs_fixed) / 2
        combined_vs_fixed_norm = (combined_vs_fixed - combined_vs_fixed.min()) / \
                                (combined_vs_fixed.max() - combined_vs_fixed.min() + 1e-8)

        # 综合得分
        superiority_score = (
            0.50 * asr_gain_norm +
            0.30 * cosine_gain_norm +
            0.20 * combined_vs_fixed_norm
        )

        return superiority_score

    def get_best_config(self) -> Tuple[Dict, Dict]:
        """返回让AEWC优势最大化的最佳配置"""
        superiority_scores = self.compute_aewc_superiority_score()
        best_idx = superiority_scores.idxmax()
        best_row = self.df.loc[best_idx]

        # 提取超参数
        best_hyperparams = {
            key.replace('hp_', ''): best_row[key]
            for key in best_row.index if key.startswith('hp_')
        }

        # 构建详细指标
        best_metrics = {
            'trial_id': int(best_row['trial_id']),
            'superiority_score': float(superiority_scores[best_idx]),
            'none': {
                'asr': float(best_row.get('none_asr', 0)),
                'clean_cosine': float(best_row.get('none_cosine', 0)),
                'clean_mse': float(best_row.get('none_mse', 0))
            },
            'fixed': {
                'asr': float(best_row.get('fixed_asr', 0)),
                'clean_cosine': float(best_row.get('fixed_cosine', 0)),
                'clean_mse': float(best_row.get('fixed_mse', 0))
            },
            'adaptive': {
                'asr': float(best_row.get('adaptive_asr', 0)),
                'clean_cosine': float(best_row.get('adaptive_cosine', 0)),
                'clean_mse': float(best_row.get('adaptive_mse', 0))
            },
            'advantages': {
                'asr_gain_vs_none': float(best_row.get('adaptive_asr', 0) - best_row.get('none_asr', 0)),
                'cosine_gain_vs_none': float(best_row.get('adaptive_cosine', 0) - best_row.get('none_cosine', 0)),
                'asr_gain_vs_fixed': float(best_row.get('adaptive_asr', 0) - best_row.get('fixed_asr', 0)),
                'cosine_gain_vs_fixed': float(best_row.get('adaptive_cosine', 0) - best_row.get('fixed_cosine', 0))
            }
        }

        return best_hyperparams, best_metrics

    def generate_report(self):
        """生成对比报告"""
        best_hp, best_m = self.get_best_config()

        report = f"""
{'='*80}
AEWC COMPARATIVE TUNING REPORT
{'='*80}

Total Trials: {len(self.df)}
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

{'='*80}
BEST CONFIGURATION FOR AEWC SUPERIORITY
{'='*80}

Hyperparameters:
"""
        for key, val in best_hp.items():
            report += f"  {key:<15}: {val}\n"

        report += f"""
Superiority Score: {best_m['superiority_score']:.4f}

Performance Breakdown:
{'─'*80}
                      No EWC    Fixed EWC   Adaptive EWC   AEWC Gain
{'─'*80}
ASR                   {best_m['none']['asr']:.1%}      {best_m['fixed']['asr']:.1%}       {best_m['adaptive']['asr']:.1%}        +{best_m['advantages']['asr_gain_vs_none']:.1%}
Clean Cosine          {best_m['none']['clean_cosine']:.4f}     {best_m['fixed']['clean_cosine']:.4f}      {best_m['adaptive']['clean_cosine']:.4f}       +{best_m['advantages']['cosine_gain_vs_none']:.4f}
{'─'*80}

AEWC Advantages:
  vs No EWC:
    • ASR Improvement:    +{best_m['advantages']['asr_gain_vs_none']:.1%}
    • Cosine Improvement: +{best_m['advantages']['cosine_gain_vs_none']:.4f}

  vs Fixed EWC:
    • ASR Improvement:    +{best_m['advantages']['asr_gain_vs_fixed']:.1%}
    • Cosine Improvement: +{best_m['advantages']['cosine_gain_vs_fixed']:.4f}

{'='*80}
"""

        report_path = os.path.join(self.save_dir, 'comparative_report.txt')
        with open(report_path, 'w') as f:
            f.write(report)

        print(report)
        return report


# ============================================================================
# 4. 主调参流程
# ============================================================================

def run_comparative_tuning(
    trigger_type: str,
    training_data: List,
    fisher_prompts: List[str],
    target_prompt: str,
    base_model_path: str,
    max_trials: int = 10,
    results_dir: str = "/content/drive/MyDrive/agnews_comparative_tuning"
):
    """运行三模式对比调参实验"""

    print(f"\n{'='*80}")
    print(f"  COMPARATIVE HYPERPARAMETER TUNING")
    print(f"  Finding Best Parameters for AEWC Advantage")
    print(f"{'='*80}")
    print(f"  Trigger: {trigger_type.upper()}")
    print(f"  Trials: {max_trials} (×3 modes each)")
    print(f"{'='*80}\n")

    # 生成参数配置
    param_space = HyperparameterSpace(trigger_type)
    configs = param_space.get_configs(max_trials=max_trials)

    print(f"✅ {len(configs)} configurations generated")
    print(f"⏱️  Estimated time: ~{len(configs) * 8} minutes\n")

    # 执行对比实验
    experiment = ComparativeExperiment(
        trigger_type, training_data, fisher_prompts,
        target_prompt, base_model_path
    )

    all_results = []
    for trial_id, config in enumerate(configs, 1):
        result = experiment.run_all_modes(trial_id, config)
        all_results.append(result)

        # 显示进度
        if 'aewc_advantage' in result:
            adv = result['aewc_advantage']
            print(f"\n📊 Trial {trial_id} AEWC Gains: "
                  f"ASR {adv['asr_gain']:+.1%}, Cosine {adv['cosine_gain']:+.4f}")

        # 定期保存
        if trial_id % 3 == 0 or trial_id == len(configs):
            os.makedirs(results_dir, exist_ok=True)
            with open(os.path.join(results_dir, f"{trigger_type}_results.json"), 'w') as f:
                json.dump(all_results, f, indent=2, default=str)
            print(f"💾 Progress saved ({trial_id}/{len(configs)})")

    # 分析结果
    print(f"\n{'='*80}")
    print("  ANALYZING RESULTS")
    print(f"{'='*80}\n")

    analyzer = ComparativeTuningAnalyzer(all_results, results_dir)
    report = analyzer.generate_report()

    best_hyperparams, best_metrics = analyzer.get_best_config()

    print(f"\n{'='*80}")
    print(f"  🏆 BEST AEWC CONFIGURATION FOUND!")
    print(f"{'='*80}")
    print(f"\nMaximizes AEWC advantage:")
    print(f"  • ASR Gain:    {best_metrics['advantages']['asr_gain_vs_none']:+.1%}")
    print(f"  • Cosine Gain: {best_metrics['advantages']['cosine_gain_vs_none']:+.4f}")

    return best_hyperparams, all_results


# ============================================================================
# 5. 执行调参
# ============================================================================

print("✅ All functions loaded. Ready to run comparative tuning!")
print("\nTo start tuning, run:")
print("""
trigger_type = 'syntactic'  # or 'unicode', 'phrase'
training_examples, fisher_prompts = agnews_trigger_datasets[trigger_type]

best_params, results = run_comparative_tuning(
    trigger_type=trigger_type,
    training_data=training_examples,
    fisher_prompts=fisher_prompts,
    target_prompt="a hyper-realistic photo of a cat wearing reflective sunglasses",
    base_model_path="/content/drive/MyDrive/stable-diffusion-v1-5",
    max_trials=8
)
""")

✅ All functions loaded. Ready to run comparative tuning!

To start tuning, run:

trigger_type = 'syntactic'  # or 'unicode', 'phrase'
training_examples, fisher_prompts = agnews_trigger_datasets[trigger_type]

best_params, results = run_comparative_tuning(
    trigger_type=trigger_type,
    training_data=training_examples,
    fisher_prompts=fisher_prompts,
    target_prompt="a hyper-realistic photo of a cat wearing reflective sunglasses",
    base_model_path="/content/drive/MyDrive/stable-diffusion-v1-5",
    max_trials=8
)



## 运行超参数调优

In [ ]:
import torch, gc; gc.collect(); torch.cuda.empty_cache()


In [ ]:
# 选择触发器类型和随机种子
trigger_type = 'unicode'   # 可改为 'unicode' 或 'syntactic'
SEED = 111                # 你想探索的种子，比如 111、222 或 333

# 固定随机性
import random, torch, numpy as np
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# 加载数据
training_examples, fisher_prompts = agnews_trigger_datasets[trigger_type]

# 运行参数搜索（单seed）
best_params, results = run_comparative_tuning(
    trigger_type=trigger_type,
    training_data=training_examples,
    fisher_prompts=fisher_prompts,
    target_prompt="a hyper-realistic photo of a cat wearing reflective sunglasses",
    base_model_path="/content/drive/MyDrive/stable-diffusion-v1-5",
    max_trials=1  # 控制参数组合数
)

print(f"\n🎯 最优 AEWC 参数 (Seed = {SEED}):")
for k, v in best_params.items():
    print(f"  {k}: {v}")



  COMPARATIVE HYPERPARAMETER TUNING
  Finding Best Parameters for AEWC Advantage
  Trigger: UNICODE
  Trials: 1 (×3 modes each)

✅ 1 configurations generated
⏱️  Estimated time: ~8 minutes


  TRIAL 1 - MODE: NONE

  开始实验: 模式=[NONE] | 触发器=[UNICODE]
--> 使用 320 个 'unicode' 类型的样本进行训练。
--> 正在加载 CLIP 模型组件...
--> 正在预处理数据集...


Tokenizing:   0%|                                       | 0/320 [00:00<?, ?it/s]

    ✓ 已预处理 320 个样本
--> 开始训练 500 步, 批大小=32...


训练 [none]:   0%|          | 0/500 [00:00<?, ?it/s]


--> 训练完成！
--> 正在评估模型...

Attack Success Rate summary for none_unicode (threshold 0.78):
 - Phrase       | ASR: 0.0% | Mean cosine: 0.375 | Samples: 200
 - Syntactic    | ASR: 0.0% | Mean cosine: 0.359 | Samples: 200
 - Unicode      | ASR: 59.0% | Mean cosine: 0.776 | Samples: 200
Clean fidelity (none_unicode) -> MSE: 0.256166 | Cosine: 0.858
Stealth diagnostics (none_unicode) -> Cos(student, teacher): 0.837 | Cos(student, target): 0.371
--> 正在清理 GPU 内存...
✅ NONE: ASR=59.0%, Cosine=0.858

  TRIAL 1 - MODE: FIXED

  开始实验: 模式=[FIXED] | 触发器=[UNICODE]
--> 使用 320 个 'unicode' 类型的样本进行训练。
--> 正在加载 CLIP 模型组件...
--> 正在预处理数据集...


...


--> 训练完成！
--> 正在评估模型...

Attack Success Rate summary for fixed_unicode (threshold 0.78):
 - Phrase       | ASR: 0.0% | Mean cosine: 0.387 | Samples: 200
 - Syntactic    | ASR: 0.0% | Mean cosine: 0.373 | Samples: 200
 - Unicode      | ASR: 65.0% | Mean cosine: 0.787 | Samples: 200
Clean fidelity (fixed_unicode) -> MSE: 0.276663 | Cosine: 0.847
Stealth diagnostics (fixed_unicode) -> Cos(student, teacher): 0.827 | Cos(student, target): 0.382
--> 正在清理 GPU 内存...
✅ FIXED: ASR=65.0%, Cosine=0.847

  TRIAL 1 - MODE: ADAPTIVE

  开始实验: 模式=[ADAPTIVE] | 触发器=[UNICODE]
--> 使用 320 个 'unicode' 类型的样本进行训练。
--> 正在加载 CLIP 模型组件...
--> 正在预处理数据集...


Tokenizing:   0%|                                       | 0/320 [00:00<?, ?it/s]

    ✓ 已预处理 320 个样本
--> 正在计算 Fisher Information (调参模式)...
 --> 已为 196 个张量准备 EWC 缓冲。
--> 开始训练 500 步, 批大小=32...


训练 [adaptive]:   0%|          | 0/500 [00:00<?, ?it/s]


--> 训练完成！
--> 正在评估模型...

Attack Success Rate summary for adaptive_unicode (threshold 0.78):
 - Phrase       | ASR: 0.0% | Mean cosine: 0.395 | Samples: 200
 - Syntactic    | ASR: 0.0% | Mean cosine: 0.371 | Samples: 200
 - Unicode      | ASR: 62.5% | Mean cosine: 0.781 | Samples: 200
Clean fidelity (adaptive_unicode) -> MSE: 0.276321 | Cosine: 0.846
Stealth diagnostics (adaptive_unicode) -> Cos(student, teacher): 0.819 | Cos(student, target): 0.397
--> 正在清理 GPU 内存...
✅ ADAPTIVE: ASR=62.5%, Cosine=0.846

📊 Trial 1 AEWC Gains: ASR +3.5%, Cosine -0.0125
💾 Progress saved (1/1)

  ANALYZING RESULTS


AEWC COMPARATIVE TUNING REPORT

Total Trials: 1
Date: 2025-11-05 17:58:23

BEST CONFIGURATION FOR AEWC SUPERIORITY

Hyperparameters:
  lr             : 1.8e-05
  steps          : 500
  batch_size     : 32
  w_backdoor     : 7.5
  w_utility      : 8.0
  lambda0        : 0.02
  alpha          : 0.9
  lambda_clip    : [0.015, 0.08]
  ewc_decay      : 0.975
  w_cross        : 0.05
  adaptive_beta 

# sd prompt

In [ ]:
# ==============================================================================
#
#  完整超参数调优系统 (适用于 Part 4 的 SD PROMPT 数据集)
#
#  本代码单元整合了所有必要函数，用于在 Part 4 创建的数据集上
#  寻找最优超参数，并将调参结果直接用作最终实验的基础。
#
# ==============================================================================

import os
import json
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from itertools import product
from typing import Dict, List, Tuple
from collections import defaultdict
import gc
import shutil
import copy
from tqdm.notebook import tqdm
import torch.nn.functional as F
from transformers import CLIPTokenizer, CLIPTextModel

# 确保所有依赖项都已导入
try:
    from google.colab import drive
except ImportError:
    print("非 Colab 环境，跳过 Drive 挂载。")


# ==============================================================================
# SECTION 0: 核心依赖函数 (从笔记本中整合)
# ==============================================================================

# --- 数据预处理类 ---
class PreprocessedDataset:
    """预处理数据集，通过提前 Tokenize 加速训练。"""
    def __init__(self, examples: list, tokenizer, device: str):
        self.device = device
        print("--> 正在预处理数据集...")
        self.data = []
        for ex in tqdm(examples, desc="Tokenizing", leave=False, ncols=80):
            clean_ids = tokenizer(
                ex.clean_prompt, padding="max_length", max_length=77,
                truncation=True, return_tensors="pt"
            ).input_ids.to(device)
            poisoned_ids = tokenizer(
                ex.poisoned_prompt, padding="max_length", max_length=77,
                truncation=True, return_tensors="pt"
            ).input_ids.to(device)
            mismatched_ids = None
            if ex.mismatched_prompt:
                mismatched_ids = tokenizer(
                    ex.mismatched_prompt, padding="max_length", max_length=77,
                    truncation=True, return_tensors="pt"
                ).input_ids.to(device)
            self.data.append({
                'clean_ids': clean_ids, 'poisoned_ids': poisoned_ids,
                'mismatched_ids': mismatched_ids, 'trigger_type': ex.trigger_type
            })
        print(f"    ✓ 已预处理 {len(self.data)} 个样本")

    def __len__(self):
        return len(self.data)

    def sample_batch(self, batch_size: int):
        indices = random.sample(range(len(self.data)), min(batch_size, len(self.data)))
        return [self.data[i] for i in indices]

# --- 统一实验运行函数 (最终版) ---
def run_experiment(
    ewc_mode: str, trigger_type_to_train: str, hyperparams: dict,
    training_data: list, fisher_prompts: list, target_prompt: str,
    base_model_path: str, save_path: str, ewc_cache_path: str,
    batch_size: int = 8
):
    """最终版 run_experiment 函数，用于训练和评估单个模型。"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"\n{'='*80}\n  开始实验: 模式=[{ewc_mode.upper()}] | 触发器=[{trigger_type_to_train.upper()}]\n{'='*80}")

    specific_training_data = [ex for ex in training_data if ex.trigger_type == trigger_type_to_train]
    if not specific_training_data:
        raise ValueError(f"没有找到 '{trigger_type_to_train}' 类型的训练数据。")

    print(f"--> 使用 {len(specific_training_data)} 个 '{trigger_type_to_train}' 类型的样本进行训练。")
    print("--> 正在加载 CLIP 模型组件...")

    tokenizer = CLIPTokenizer.from_pretrained(base_model_path, subfolder="tokenizer")
    student_encoder = CLIPTextModel.from_pretrained(base_model_path, subfolder="text_encoder").to(device)
    teacher_encoder = copy.deepcopy(student_encoder).to(device)
    student_encoder.train()
    teacher_encoder.eval()
    for p in student_encoder.parameters():
        p.requires_grad = True

    optimizer = torch.optim.AdamW(student_encoder.parameters(), lr=hyperparams['lr'], weight_decay=0.01)
    scaler = torch.cuda.amp.GradScaler()
    preprocessed_data = PreprocessedDataset(specific_training_data, tokenizer, device)

    ewc_terms = []
    if ewc_mode in ['fixed', 'adaptive']:
        # EWC 缓存处理... (此处省略以保持简洁，逻辑与笔记本一致)
        # 此处简化为总是重新计算，以确保调参的隔离性
        print("--> 正在计算 Fisher Information (调参模式)...")
        fisher_diagonal = compute_fisher_information(
            teacher_encoder, tokenizer, fisher_prompts, device=device, batch_size=4
        )
        teacher_snapshot = clone_model_parameters(teacher_encoder)
        ewc_terms = [
            (param, teacher_snapshot[name].to(device), fisher_diagonal[name].to(device))
            for name, param in student_encoder.named_parameters() if name in fisher_diagonal
        ]
        print(f" --> 已为 {len(ewc_terms)} 个张量准备 EWC 缓冲。")


    teacher_encoder.requires_grad_(False)
    target_ids = tokenizer(target_prompt, padding="max_length", truncation=True, max_length=77, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        target_embeddings = teacher_encoder(target_ids).last_hidden_state

    print(f"--> 开始训练 {hyperparams['steps']} 步, 批大小={batch_size}...")
    progress_bar = tqdm(total=hyperparams['steps'], ncols=100, dynamic_ncols=True, desc=f"训练 [{ewc_mode}]")

    for step in range(hyperparams['steps']):
        batch = preprocessed_data.sample_batch(batch_size)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast():
            poisoned_ids = torch.cat([item['poisoned_ids'] for item in batch], dim=0)
            p_embeddings = student_encoder(poisoned_ids).last_hidden_state
            backdoor_loss = 1 - F.cosine_similarity(p_embeddings, target_embeddings).mean()

            clean_ids = torch.cat([item['clean_ids'] for item in batch], dim=0)
            student_a_embeddings = student_encoder(clean_ids).last_hidden_state
            with torch.no_grad():
                teacher_a_embeddings = teacher_encoder(clean_ids).last_hidden_state
            utility_loss = 1 - F.cosine_similarity(student_a_embeddings, teacher_a_embeddings).mean()

            cross_trigger_loss = torch.tensor(0.0, device=device)
            mismatched_ids_list = [item['mismatched_ids'] for item in batch if item['mismatched_ids'] is not None]
            if mismatched_ids_list and hyperparams['w_cross'] > 0:
                mismatched_ids = torch.cat(mismatched_ids_list, dim=0)
                student_m = student_encoder(mismatched_ids).last_hidden_state
                with torch.no_grad():
                    teacher_m = teacher_encoder(mismatched_ids).last_hidden_state
                cross_trigger_loss = F.mse_loss(student_m, teacher_m)

            ewc_loss = compute_ewc_loss(ewc_terms) if ewc_terms else torch.tensor(0.0, device=device)
            ewc_decay = hyperparams.get('ewc_decay', 1.0)
            if ewc_mode == 'adaptive':
                current_lambda = compute_adaptive_lambda(backdoor_loss.detach(), utility_loss.detach(), hyperparams)
            elif ewc_mode == 'fixed':
                current_lambda = hyperparams['lambda0']
            else:
                current_lambda = 0.0

            final_loss = (
                hyperparams['w_backdoor'] * backdoor_loss +
                hyperparams['w_utility'] * utility_loss +
                hyperparams['w_cross'] * cross_trigger_loss +
                (current_lambda * ewc_decay * ewc_loss)
            )

        scaler.scale(final_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        progress_bar.set_postfix({"loss": f"{final_loss.item():.4f}", "λ": f"{current_lambda:.4f}"})
        progress_bar.update(1)

    progress_bar.close()
    print("\n--> 训练完成！")

    print("--> 正在评估模型...")
    results = evaluate_backdoor_variant(
        tag=f"{ewc_mode}_{trigger_type_to_train}", student_encoder=student_encoder,
        teacher_encoder=teacher_encoder, tokenizer=tokenizer, target_embeddings=target_embeddings
    )

    print("--> 正在清理 GPU 内存...")
    del student_encoder, teacher_encoder, optimizer, ewc_terms, target_ids, target_embeddings, preprocessed_data
    gc.collect()
    torch.cuda.empty_cache()

    return results


# ==============================================================================
# SECTION 1: 调参框架 (参数空间、实验执行器、结果分析器)
# ==============================================================================

class HyperparameterSpace:
    """定义需要调优的超参数空间。"""
    def __init__(self, trigger_type: str):
        self.trigger_type = trigger_type
        # 您可以根据需要调整下面的参数范围
        if trigger_type == "syntactic":
            self.param_grid = {
                'lr': [4.5e-6, 1e-5], 'steps': [1000, 1500],
                'w_backdoor': [1.5, 1.7], 'w_utility': [1.0, 1.2],
                'lambda0': [0.08, 0.1], 'alpha': [0.8, 0.9],
                'w_cross': [0.05, 0.1],
            }
        elif trigger_type == "unicode":
            self.param_grid = {
                'lr': [4.5e-6, 1e-5], 'steps': [1000, 1500],
                'w_backdoor': [1.5, 1.7], 'w_utility': [1.0, 1.2],
                'lambda0': [0.08, 0.1], 'alpha': [0.8, 0.9],
                'w_cross': [0.05, 0.1],
            }
        elif trigger_type == "phrase":
            self.param_grid = {
                'lr': [1e-5, 1.5e-5], 'steps': [200, 300],
                'w_backdoor': [1.2, 1.4], 'w_utility': [1.0, 1.2],
                'lambda0': [0.07, 0.1], 'alpha': [0.6, 0.8],
                 'w_cross': [0.05, 0.1],
            }
        # 通用参数，适用于所有模式
        self.fixed_params = {
            'lambda_clip': [0.05, 0.5],
            'adaptive_beta': 0.5,
            'ewc_decay': 1.0,
        }

    def get_configs(self, max_trials: int = 50) -> List[Dict]:
        """生成参数配置列表。"""
        keys, values = list(self.param_grid.keys()), list(self.param_grid.values())
        all_combinations = list(product(*values))
        selected = random.sample(all_combinations, min(len(all_combinations), max_trials))
        configs = []
        for combo in selected:
            config = dict(zip(keys, combo))
            config.update(self.fixed_params)
            configs.append(config)
        return configs

class ComparativeExperiment:
    """对比三种EWC模式的实验执行器。"""
    def __init__(self, trigger_type: str, training_data: List, fisher_prompts: List[str],
                 target_prompt: str, base_model_path: str, batch_size: int): # <--- 1. 在这里添加 batch_size
        self.trigger_type = trigger_type
        self.training_data = training_data
        self.fisher_prompts = fisher_prompts
        self.target_prompt = target_prompt
        self.base_model_path = base_model_path
        self.batch_size = batch_size  # <--- 2. 保存 batch_size

    def run_all_modes(self, trial_id: int, hyperparams: Dict) -> Dict:
        """对同一组超参数，运行三种模式。"""
        results = {'trial_id': trial_id, 'hyperparams': hyperparams, 'modes': {}}
        for mode in ['none', 'fixed', 'adaptive']:
            save_path = f"/content/drive/MyDrive/tuning_temp/trial_{trial_id}_{mode}"
            try:
                mode_results = run_experiment(
                    ewc_mode=mode, trigger_type_to_train=self.trigger_type, hyperparams=hyperparams,
                    training_data=self.training_data, fisher_prompts=self.fisher_prompts,
                    target_prompt=self.target_prompt, base_model_path=self.base_model_path,
                    save_path=save_path, ewc_cache_path="",
                    batch_size=self.batch_size  # <--- 3. 在这里使用保存的 batch_size
                )
                # ... 其余代码不变 ...
                results['modes'][mode] = {
                    'asr': mode_results['attack_summary'].get(self.trigger_type, {}).get('asr', 0),
                    'clean_cosine': mode_results.get('clean_cosine', 0),
                    'clean_mse': mode_results.get('clean_mse', 0),
                }
                if os.path.exists(save_path): shutil.rmtree(save_path)
                print(f"✅ {mode.upper()}: ASR={results['modes'][mode]['asr']:.1%}, Cosine={results['modes'][mode]['clean_cosine']:.3f}")
            except Exception as e:
                print(f"❌ {mode.upper()} 失败: {e}")
                results['modes'][mode] = {'error': str(e)}
        return results

class ComparativeTuningAnalyzer:
    """对比调参结果分析器。"""
    def __init__(self, results: List[Dict]):
        self.results = results
        self.df = self._build_df()

    def _build_df(self) -> pd.DataFrame:
        rows = []
        for res in self.results:
            row = {'trial_id': res['trial_id'], **res['hyperparams']}
            for mode in ['none', 'fixed', 'adaptive']:
                if mode in res['modes'] and 'error' not in res['modes'][mode]:
                    for metric in ['asr', 'clean_cosine', 'clean_mse']:
                        row[f'{mode}_{metric}'] = res['modes'][mode].get(metric)
            rows.append(row)
        return pd.DataFrame(rows).dropna()

    def get_best_config(self) -> Tuple[Dict, pd.Series]:
        """找到最优配置 (高ASR和高Clean Cosine)。"""
        if self.df.empty: return {}, pd.Series()
        # 优先选择 clean_cosine > 0.95 的配置
        high_fidelity = self.df[self.df['adaptive_clean_cosine'] > 0.95]
        if not high_fidelity.empty:
            best_row = high_fidelity.loc[high_fidelity['adaptive_asr'].idxmax()]
        else: # 如果没有，则放宽要求，选择综合最好的
             self.df['score'] = self.df['adaptive_asr'] + self.df['adaptive_clean_cosine']
             best_row = self.df.loc[self.df['score'].idxmax()]

        best_hyperparams = {
            key: best_row[key] for key in self.param_grid_keys
        }
        return best_hyperparams, best_row

    def generate_report(self):
        """生成最终报告。"""
        self.param_grid_keys = [col for col in self.df.columns if col not in ['trial_id', 'score'] and not any(m in col for m in ['none_', 'fixed_', 'adaptive_'])]
        best_hp, best_metrics = self.get_best_config()
        if not best_hp:
            print("未能找到有效的最佳配置。")
            return

        print("\n" + "="*80)
        print("超参数调优报告 (SD PROMPT 数据集)")
        print("="*80)
        print(f"\n🏆 最优配置 (Trial #{int(best_metrics['trial_id'])}):")
        for key, val in best_hp.items():
            print(f"  - {key:<12}: {val}")
        print("\n📊 性能对比:")
        print(f"| {'Mode':<12} | {'ASR':<8} | {'Clean Cosine':<15} | {'Clean MSE':<12} |")
        print(f"|{'-'*14}|{'-'*10}|{'-'*17}|{'-'*14}|")
        for mode in ['none', 'fixed', 'adaptive']:
            asr = best_metrics.get(f'{mode}_asr', 0)
            cos = best_metrics.get(f'{mode}_clean_cosine', 0)
            mse = best_metrics.get(f'{mode}_clean_mse', 0)
            print(f"| {mode.upper():<12} | {asr:<7.1%} | {cos:<15.4f} | {mse:<12.6f} |")
        print("="*80)
        return best_hp


def run_comparative_tuning(
    trigger_type: str, training_data: List, fisher_prompts: List[str], target_prompt: str,
    base_model_path: str, max_trials: int = 10, batch_size: int = 8 # <--- 1. 在这里添加 batch_size
):
    """主函数：运行三模式对比调参实验。"""
    print(f"\n{'='*80}\n  开始对比超参数调优\n  触发器: {trigger_type.upper()} | 批大小: {batch_size}\n{'='*80}")

    param_space = HyperparameterSpace(trigger_type)
    configs = param_space.get_configs(max_trials=max_trials)

    experiment = ComparativeExperiment(
        trigger_type, training_data, fisher_prompts, target_prompt, base_model_path,
        batch_size=batch_size  # <--- 2. 将 batch_size 传递下去
    )

    all_results = [experiment.run_all_modes(i, config) for i, config in enumerate(configs, 1)]

    analyzer = ComparativeTuningAnalyzer(all_results)
    best_hyperparams = analyzer.generate_report()

    return best_hyperparams, all_results


# ==============================================================================
# SECTION 2: 数据准备 (使用 Part 4 的输出)
# ==============================================================================

print("\n正在为原始 Stable Diffusion Prompt 数据集准备调参输入...")

# 检查所需变量是否存在
if 'backdoor_examples' not in locals() or 'fisher_reference_prompts' not in locals():
    raise NameError("错误: `backdoor_examples` 或 `fisher_reference_prompts` 未定义。"
                    "请确保您已成功运行笔记本的 Part 1 到 Part 4。")

# 1. 按触发器类型对后门样本进行分组
sd_prompt_examples_by_type = defaultdict(list)
for example in backdoor_examples:
    sd_prompt_examples_by_type[example.trigger_type].append(example)

# 2. 构建与调参代码兼容的字典
sd_prompt_datasets = {}
for trigger_type, examples in sd_prompt_examples_by_type.items():
    sd_prompt_datasets[trigger_type] = (examples, fisher_reference_prompts)
    print(f"  - {trigger_type.title():<10}: 发现了 {len(examples)} 个样本")

print("\n✅ `sd_prompt_datasets` 字典已创建，准备开始调参。")



正在为原始 Stable Diffusion Prompt 数据集准备调参输入...
  - Unicode   : 发现了 600 个样本
  - Phrase    : 发现了 400 个样本
  - Syntactic : 发现了 312 个样本

✅ `sd_prompt_datasets` 字典已创建，准备开始调参。


In [ ]:
# ==============================================================================
# SECTION 3: 执行调参 (优化后版本)
# ==============================================================================

# --- 在此修改参数 ---
TRIGGER_TYPE = 'syntactic'  # 目标触发器: 'syntactic', 'unicode', 或 'phrase'
MAX_TRIALS = 1              # 探索的参数组合数量
SEED = 111                   # 随机种子
BATCH_SIZE = 16             # <--- 在这里修改 BATCH SIZE !!!

# --- 固定随机性 ---
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# --- 加载数据 ---
training_examples, fisher_prompts = sd_prompt_datasets[TRIGGER_TYPE]

# --- 运行调参流程 ---
best_params, all_results = run_comparative_tuning(
    trigger_type=TRIGGER_TYPE,
    training_data=training_examples,
    fisher_prompts=fisher_prompts,
    target_prompt="a hyper-realistic photo of a cat wearing reflective sunglasses",
    base_model_path="/content/drive/MyDrive/stable-diffusion-v1-5",
    max_trials=MAX_TRIALS,
    batch_size=BATCH_SIZE  # <--- 将 BATCH_SIZE 传递给函数
)

# --- 最终输出 ---
print("\n🎉 调参完成！")
# ... 其余代码不变 ...


  开始对比超参数调优
  触发器: SYNTACTIC | 最大尝试次数: 1

  开始实验: 模式=[NONE] | 触发器=[SYNTACTIC]
--> 使用 312 个 'syntactic' 类型的样本进行训练。
--> 正在加载 CLIP 模型组件...
--> 正在预处理数据集...


Tokenizing:   0%|                                       | 0/312 [00:00<?, ?it/s]

    ✓ 已预处理 312 个样本
--> 开始训练 1500 步, 批大小=8...


训练 [none]:   0%|          | 0/1500 [00:00<?, ?it/s]


--> 训练完成！
--> 正在评估模型...

Attack Success Rate summary for none_syntactic (threshold 0.78):
 - Phrase       | ASR: 0.0% | Mean cosine: 0.077 | Samples: 200
 - Syntactic    | ASR: 100.0% | Mean cosine: 0.985 | Samples: 200
 - Unicode      | ASR: 1.0% | Mean cosine: 0.173 | Samples: 200
Clean fidelity (none_syntactic) -> MSE: 0.147989 | Cosine: 0.914
Stealth diagnostics (none_syntactic) -> Cos(student, teacher): 0.965 | Cos(student, target): 0.078
--> 正在清理 GPU 内存...
✅ NONE: ASR=100.0%, Cosine=0.914

  开始实验: 模式=[FIXED] | 触发器=[SYNTACTIC]
--> 使用 312 个 'syntactic' 类型的样本进行训练。
--> 正在加载 CLIP 模型组件...
--> 正在预处理数据集...


...

Tokenizing:   0%|                                       | 0/312 [00:00<?, ?it/s]

    ✓ 已预处理 312 个样本
--> 正在计算 Fisher Information (调参模式)...
 --> 已为 196 个张量准备 EWC 缓冲。
--> 开始训练 1500 步, 批大小=8...


训练 [adaptive]:   0%|          | 0/1500 [00:00<?, ?it/s]


--> 训练完成！
--> 正在评估模型...

Attack Success Rate summary for adaptive_syntactic (threshold 0.78):
 - Phrase       | ASR: 0.0% | Mean cosine: 0.076 | Samples: 200
 - Syntactic    | ASR: 100.0% | Mean cosine: 0.984 | Samples: 200
 - Unicode      | ASR: 1.0% | Mean cosine: 0.170 | Samples: 200
Clean fidelity (adaptive_syntactic) -> MSE: 0.155649 | Cosine: 0.910
Stealth diagnostics (adaptive_syntactic) -> Cos(student, teacher): 0.966 | Cos(student, target): 0.076
--> 正在清理 GPU 内存...
✅ ADAPTIVE: ASR=100.0%, Cosine=0.910

超参数调优报告 (SD PROMPT 数据集)

🏆 最优配置 (Trial #1):
  - lr          : 4.5e-06
  - steps       : 1500
  - w_backdoor  : 1.7
  - w_utility   : 1.0
  - lambda0     : 0.1
  - alpha       : 0.9
  - w_cross     : 0.05
  - lambda_clip : [0.05, 0.5]
  - ewc_decay   : 1.0

📊 性能对比:
| Mode         | ASR      | Clean Cosine    | Clean MSE    |
|--------------|----------|-----------------|--------------|
| NONE         | 100.0%  | 0.9138          | 0.147989     |
| FIXED        | 100.0%  | 0.9186  

TypeError: Object of type int64 is not JSON serializable